In [1]:
from numpy import random
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib
import random as rand
import math
import statistics
import numpy as np
from scipy.stats import uniform
from sklearn import preprocessing
import time
from sklearn.cluster import AffinityPropagation
from numpy import unique
from sklearn.cluster import MeanShift
from scipy.stats import poisson
from fbm import FBM
from datetime import datetime

In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from numpy import absolute
from numpy import mean as m
from numpy import std

In [3]:
#for clustering
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import Birch
from sklearn.cluster import SpectralClustering
from sklearn.mixture import GaussianMixture

In [4]:
# Classifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [5]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

if I use this for distributing the resource of vnf among the micro vnfs. Run this defination for each vnf and get the info of how many micro vnfs can be formed.
for example the vnf 0 requested resource is 5, and the requested resources are divided into 3,2. so we can say that we need three microvnfs for that vnf.
This will provide two things,  a way to say each vnf can be divded into so and so micro services, which is basically the info provide by the service provides.
secondly, we will know how much resource each micro vnfs are requested for and from there we can try to implement the concept of decompostions.


In [6]:
extra_note = 'Introducing Traffic'
trial = 'Trial_DDQN_100'
model = 'DDQN'
pr_HL = '6'

## Deep Q Learning

### Linear DQN

In [7]:
import os
import torch as T
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as TT
from collections import namedtuple, deque

from torch.autograd import Variable


In [8]:
def truncate(number, decimals=0):
    """
    Returns a value truncated to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer.")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more.")
    elif decimals == 0:
        return math.trunc(number)

    factor = 10.0 ** decimals
    return math.trunc(number * factor) / factor

In [9]:
class Trail():
    def __init__(self, num_episode, num_sfc, vNetwork_service_er, vNetwork_service_details_er, max_vnf_resource,
                 vnf_per_service, vnf_node_resources, vnf_link_resources, mean, var):
        
        self.num_episode = num_episode
        self.num_sfc = num_sfc
        self.vNetwork_service_er = vNetwork_service_er
        self.vNetwork_service_details_er = vNetwork_service_details_er
        self.vnf_node_resources = vnf_node_resources
        self.vnf_link_resources= vnf_link_resources
        self.vnf_per_service = vnf_per_service
        #print(self.vnf_per_service)
        self.overall_episode_serviceclass = []
        
        for episode in range(self.num_episode): #1 Runtime = 200 episode
            #print("episode",episode)
            self.episode = episode
            service =[]
            service_details =[]
            rand_seed_collection =[]
            vnf_per_episode = []
            self.episode_serviceclass = []
            norm_QoS_policy_val = []
            env_priority = []
            
            # Generating seeds to control the isolated nodes
            seed_array = [1,3,4,7,8,9,10,13,14,16,18,21,22,25,26,28,29,30,31,32,38,39,41,42,43,44,45,47,48,49,50,51,54,58,59,60,62,63,65,66,67,69,71,73,75,76,77,78,79,80,85,86,88,89,90,91,94,96,98,99,100 ]
            
            # QoS metric
            #Flow behaviour: each SFC can be either elastic or inelastic flow. 
            elastic_flow, inelastic_flow = 0, 1
            dict_flow_behviour = {}
            flo_mu, flo_sigma = 0.5, 0.1
            normal_flow_distribution = np.random.normal(flo_mu, flo_sigma, self.num_sfc)
            int_normal_flow_distribution = np.rint(normal_flow_distribution)
            #print(normal_flow_distribution)
            #print(int_normal_flow_distribution)
            #normalizing
            norm_flow_distribution = preprocessing.normalize(np.reshape(int_normal_flow_distribution, (1, self.num_sfc)))
            #print("norm_flow_distribution: ", norm_flow_distribution)
                                           
            #Create the bins and histogram
            #count, bins, ignored = plt.hist(normal_flow_distribution, 20) 
            #Plot the distribution curve
            #plt.plot(bins, 1/(flo_sigma * np.sqrt(2 * np.pi)) * np.exp( - (bins - flo_mu)**2 / (2 * flo_sigma**2) ), linewidth=3, color='y')
            #plt.show()
            
            # QoS Rating: G.1010 rating 
            # Delay: Responsive(~ 1s), Interactive(<< 1s), Timely(~10s), non_critical/Background(>> 10s)  
            dict_QoSRating = {}
            qos_mu, qos_sigma = 2, 0.8
            normal_QoSRating_distribution     = np.random.normal(qos_mu, qos_sigma, self.num_sfc) 
            int_normal_QoSRating_distribution = np.rint(normal_QoSRating_distribution)
            #print(normal_QoSRating_distribution)
            #print("QoS Rating: ", int_normal_QoSRating_distribution)
            #normalizing
            norm_QoSRating = preprocessing.normalize(np.reshape(int_normal_QoSRating_distribution, (1, self.num_sfc)))
            #print("norm_QoSRating: ", norm_QoSRating)

            #common 
            mu, sigma =  4, 10             
            # Jitter  # (1 microsec (0.001 ms) to 30 ms)
            arr_seed = [215, 283, 284, 414, 531, 702, 717, 763, 813]
            seed = rand.choices(arr_seed)
            np.random.seed(seed[0])
            normal_jitter_distribution = abs(np.random.normal(mu, sigma, self.num_sfc))
            
            #print("normal_jitter_distribution: ", normal_jitter_distribution)
            #normalizing
            norm_jitter_distribution = preprocessing.normalize(np.reshape(normal_jitter_distribution, (1, self.num_sfc)))
            #print("norm_jitter_distribution: ", norm_jitter_distribution)

            # Delay: range: 0.1 ms to 30ms: URLLC to Best Effort
            arr_seed = [11, 13, 27, 31, 58, 76, 78, 87, 91, 119] 
            seed = rand.choices(arr_seed)
            np.random.seed(seed[0])
            normal_delay_distribution = abs(np.random.normal(mu, sigma, self.num_sfc))
            #normal_delay_distribution = truncate(normal_delay_distribution,2)
            #print("normal_delay_distribution: ", normal_delay_distribution)
            # normalised this delay 
            norm_normal_delay_distribution = preprocessing.normalize(np.reshape(normal_delay_distribution, (1, self.num_sfc)))
            #print("norm_normal_delay_distribution: ", norm_normal_delay_distribution)
            
            #Reliability: range:  1-10^-9 to 1-10^-2    # not using this 03052022        
            r_mu, r_sigma = 5, 1.5
            arr_seed = [0, 1, 2, 3, 4, 6, 7, 9, 11, 12, 13, 18]
            seed = rand.choices(arr_seed)
            np.random.seed(seed[0])
            #normal_reliability_distribution = abs(np.random.normal(r_mu, r_sigma, self.num_sfc))
            #("normal_reliability_distribution: ", normal_reliability_distribution)
            #norm_reliability_distribution = preprocessing.normalize(np.reshape(normal_reliability_distribution, (1, self.num_sfc)))
            #print("norm_reliability_distribution: ", norm_reliability_distribution)
            #print(max(normal_reliability_distribution), min(normal_reliability_distribution))
            #normal_reliability_distribution =  truncate(normal_reliability_distribution,2)
            
            # waiting time
            wt_mu, wt_sigma = 5, 10
            normal_watingtime_distribution = abs(np.random.normal(wt_mu, wt_sigma, self.num_sfc))
            #normal_watingtime_distribution = truncate(normal_watingtime_distribution,2)
            #print("normal_watingtime_distribution: ",  normal_watingtime_distribution)
            #print(max(normal_watingtime_distribution), min(normal_watingtime_distribution))
            #count, bins, ignored = plt.hist(normal_watingtime_distribution, num_sfc) 
            #Plot the distribution curve
            #plt.plot(bins, 1/(wt_sigma * np.sqrt(2 * np.pi)) * np.exp( - (bins - wt_mu)**2 / (2 * wt_sigma**2) ), linewidth=3, color='y')
            #plt.show()
            
            # normalised this delay 
            norm_watingtime_distribution = preprocessing.normalize(np.reshape(normal_watingtime_distribution, (1, self.num_sfc)))
            #print("norm_watingtime_distribution: ", norm_watingtime_distribution)
            #print("--> ",max(norm_watingtime_distribution[0]), min(norm_watingtime_distribution[0]))

            
            #Packet Loss: Each service can tolerate 0%- <5% packet Loss depending upon the application or service type.
            pl_mu, pl_sigma = 2, 1
            normal_packetloss_distribution = abs(np.random.normal(pl_mu, pl_sigma, self.num_sfc)) 
            # (0% to less than 10%) packetloss per serivce 
            #print("normal_packetloss_distribution: ", normal_packetloss_distribution)
            norm_packetloss_distribution = preprocessing.normalize(np.reshape(normal_packetloss_distribution, (1, self.num_sfc)))
            #print("norm_packetloss_distribution: ", norm_packetloss_distribution)
            
            for i in range(self.num_sfc):
                data = [truncate(1-norm_normal_delay_distribution[0][i],2), #delay
                        #norm_flow_distribution[0][i], # flow dist.
                        truncate(1-norm_jitter_distribution[0][i],2), #jitter
                        truncate(1-norm_packetloss_distribution[0][i],2), #packetloss
                        #truncate(1-norm_watingtime_distribution[0][i],2) #wating time 
                       ]
                data = np.array(data,dtype='float64')
                #print(data)
                sum_data = sum(data)
                #print(sum_data)
                
                norm_QoS_policy_val.append(sum_data)
                #print("")
               
            #print("norm_QoS_policy_val: ", type(norm_QoS_policy_val))
            #print("")
            #print("MAX: ", max(norm_QoS_policy_val))

            env_priority = Normalization(norm_QoS_policy_val, max(norm_QoS_policy_val), min(norm_QoS_policy_val))
            #preprocessing.normalize(np.reshape(norm_QoS_policy_val, (1, self.num_sfc)))
            #print(env_priority)
            
            #print("norm_QoS_policy_val: ", norm_QoS_policy_val)
            #print("SFC, flow behaviour, delay, jitter, Packetloss, reliability, wating time, env priority")
            
            #reliability
            overall_pow = [1,2,3,4,5,6,7,8,9]
            rel_factor, rel_pow, rel_norm_val = [], [], []
            for y in overall_pow:
                a = (1 - (pow(10, -y)))*100
                norm_val = (Normalization(y, max(overall_pow), min(overall_pow)))
                rel_factor.append(a)
                rel_pow.append(y)
                rel_norm_val.append(norm_val)

            #print(rel_factor, rel_pow, rel_norm_val)
            
            #for each episode generating services.
            for sfc in range(self.num_sfc): # number of VNF-FGs # time-step =100
                #print("Service", sfc)
                self.sfc = sfc
                requested_services = []
                requested_nodes    = []
                
                #self.vnf_per_service     = []
                rand_seed =  rand.choice(seed_array)
                #print("Seed", rand_seed)
                rand_seed_collection.append(rand_seed)
                #n = self.vnf_per_service #2to 10 #rand.randrange(2) #no.of nodes=VNF 
                #self.vnf_per_service = n
                #print(vnf_per_service)
                #vnf_per_episode.append(self.vnf_per_service)
                
                #reliability
                y = rand.choice(overall_pow)
                reliability_factor   = rel_factor[y-1]
                reliability_pow      = rel_pow[y-1]
                reliability_norm_val = rel_norm_val[y-1]
                #print(reliability_factor, rel_pow[y-1], rel_norm_val[y-1])
                
                #Assigning 
                # Flow Behaviour
                #print(int_normal_flow_distribution[self.sfc])  
                if int_normal_flow_distribution[self.sfc] == 1:
                    dict_flow_behviour[self.sfc] = "inelastic_flow"
                else: 
                    dict_flow_behviour[self.sfc] = "elastic_flow"
                #print(dict_flow_behviour[self.sfc])
                
                # QoS Rating: Initilization of this QoS rating based on the threshold delay
                #if normal_delay_distribution[self.sfc] >>= 10:
                
                #if int_normal_QoSRating_distribution[self.sfc] == 1:
                #    dict_QoSRating[self.sfc] = "Responsive" 
                #elif int_normal_QoSRating_distribution[self.sfc] == 2:
                #    dict_QoSRating[self.sfc] = "Interactive"
                #    dict_flow_behviour[self.sfc] = "inelastic_flow" 
                #elif int_normal_QoSRating_distribution[self.sfc] == 3:
                #    dict_QoSRating[self.sfc] = "Timely"        
                #else:
                 #   dict_QoSRating[self.sfc] = "non-critical"
                 #   dict_flow_behviour[self.sfc] = "elastic_flow"
                #print(dict_QoSRating)

                eps_value = 0.3
                p = (1 + eps_value)*(math.log(self.vnf_per_service)/self.vnf_per_service) #prob of connectivity
                #print(p)
                # Generating Graphs
                # E-R graph
                self.er_graph_seed = nx.erdos_renyi_graph(n= self.vnf_per_service, p = p, seed=rand_seed, directed =True)
                #nx.draw(self.er_graph_seed, with_labels=True) 
                #plt.show()
                #flow_behaviour = dict_flow_behviour[self.sfc]
                #QoSrating = dict_QoSRating[self.sfc]
                
                sfc_serviceclass = [self.sfc, 
                                    int_normal_flow_distribution[self.sfc], 
                                    dict_flow_behviour[self.sfc],
                                    #dict_QoSRating[self.sfc], 
                                    truncate(normal_delay_distribution[self.sfc],2),
                                    truncate(normal_jitter_distribution[self.sfc],2),
                                    truncate(normal_packetloss_distribution[self.sfc],2),
                                    reliability_factor,
                                    truncate(normal_watingtime_distribution[self.sfc],2),
                                    env_priority[self.sfc]
                                   ]
            
                #print("sfc_serviceclass: ", sfc_serviceclass)
                self.episode_serviceclass.append(sfc_serviceclass)
                
                for vnf in self.er_graph_seed.nodes():
                    self.vnf = vnf
                    #print("VNF", vnf)
                    mu, sigma = mean, var
                    normal_cpu_res = np.rint(np.random.normal(mu, sigma, 1))
                    self.cpu_req   = normal_cpu_res[0]
                    #print("self.cpu_req", self.cpu_req)
                    if self.cpu_req > max_vnf_resource:
                        print("CPU greater than max:", self.cpu_req)
                    
                    self.er_graph_seed.add_node(vnf,
                                                episode = self.episode,
                                                sfc     = self.sfc,
                                                numVNF  = self.vnf_per_service,
                                                vnf     = self.vnf,
                                                cpu_req = self.cpu_req,
                                                delay           = normal_delay_distribution[self.sfc], #delay
                                                norm_delay      = 1-norm_normal_delay_distribution[0][self.sfc], #rule 1
                                                jitter          = normal_jitter_distribution[self.sfc],
                                                norm_jitter     = 1-norm_jitter_distribution[0][self.sfc], #rule 2
                                                packetloss      = normal_packetloss_distribution[self.sfc],
                                                norm_packetloss = 1-norm_packetloss_distribution[0][self.sfc], #rule 3
                                                reliability     = reliability_norm_val,
                                                reliability_factor = reliability_factor, 
                                                flow_behaviour  = dict_flow_behviour[self.sfc],  
                                                int_flow_behaviour = int_normal_flow_distribution[self.sfc],
                                                norm_flow       = norm_flow_distribution[0][self.sfc], #rule 4
                                                theo_wating_time = normal_watingtime_distribution[self.sfc],
                                                norm_waiting_time = 1-norm_watingtime_distribution[0][self.sfc], #rule 5
                                                norm_QoS_policy_val = norm_QoS_policy_val[self.sfc],
                                                env_priority   = env_priority[self.sfc],
                                                QoSRating      = None, #using clustering 
                                                app_category   = None, #using clustering
                                                priority_level = None, #Using clustering    
                                                priority_class = None,
                                                microvnf_done_status = None,
                                                numMicroVNF = None, #self.decompose_candidate[0],
                                                microvnf_res_component = None, #self.decompose_candidate[1],
                                                decompose_candidate = None, #self.decompose_candidate[2]
                                                emb_status = None,
                                                sfc_deployment_time = None,
                                                traffic_packets = None, 
                                                traffic_load = None,
                                                classifier_class = None
                                                
                                               )
                                
                self.vnf_node_resources = list(self.er_graph_seed.nodes(data=True))
                #print(self.vnf_node_resources)

                self.num_of_vl = nx.number_of_edges(self.er_graph_seed)

                link_count = 0
                for link in self.er_graph_seed.edges():
                    #print(link)

                    #INITIALIZATION RESOURCE
                    # Allocating the bandwidth resouce.
                    self.er_graph_seed.add_edge(link[0],link[1],
                                                src_vnf  = link[0],
                                                dest_vnf = link[1],
                                                #src_microvnf = src_microvnf,
                                                #dest_microvnf = dest_microvnf,
                                                episode = self.episode,
                                                sfc = self.sfc,
                                                numVNF = self.vnf_per_service,
                                                numvl = self.num_of_vl,
                                                link_num = link_count,
                                                bw  = rand.randrange(1, 20),  #Mbps
                                                latency = rand.randrange(1,5) #ms #uniform or normalizations #check this out from the literatue 
                                               )
                    link_count += 1
                self.vnf_link_resources= list(self.er_graph_seed.edges(data=True))
                #print(self.vnf_link_resources)
                requested_services.append(self.er_graph_seed.nodes) 
                requested_nodes.append(self.vnf_node_resources)   
                requested_services.append(self.er_graph_seed.edges)
                requested_nodes.append(self.vnf_link_resources) 
                service.append(requested_services)
                service_details.append(requested_nodes)
            
            self.vNetwork_service_er.append(service)
            self.vNetwork_service_details_er.append(service_details)
            self.overall_episode_serviceclass.append(self.episode_serviceclass)
        #print("here",self.vNetwork_service_details_er)
        #print("")
                
    def update_vnf(self, episode, sfc, vnf, decompose_candidate, microvnf_done_status):
        count_epi = 0
        for epi in self.vNetwork_service_details_er:
            if episode == count_epi:
                #print("vnf", vnf, epi[sfc][0][vnf])
                epi[sfc][0][vnf][1]['microvnf_done_status'] = microvnf_done_status
                epi[sfc][0][vnf][1]['numMicroVNF'] =  decompose_candidate[0]
                epi[sfc][0][vnf][1]['microvnf_res_component'] =  decompose_candidate[1]
                epi[sfc][0][vnf][1]['decompose_candidate'] =  decompose_candidate[2]
                #print("==vnf", vnf, epi[sfc][0][vnf][1])  
                #print("decompose_candidate[1]", decompose_candidate[1])
            count_epi += 1
            
    def reset(self, episode, sfc, vnf):
        #print("in rest function")
        count_epi = 0
        for epi in self.vNetwork_service_details_er:
            if episode == count_epi:
                #print("vnf", vnf, epi[sfc][0][vnf])
                epi[sfc][0][vnf][1]['microvnf_done_status'] = None
                epi[sfc][0][vnf][1]['numMicroVNF'] =  None
                epi[sfc][0][vnf][1]['microvnf_res_component'] =  None
                epi[sfc][0][vnf][1]['decompose_candidate'] =  None
                
                #print("==vnf", vnf, epi[sfc][0][vnf][1])  
            count_epi += 1
            
    def priority_update(self, episode, sfc, vnf_per_service, priority_level, priority_class):
        self.episode = episode
        self.sfc = sfc
        self.vnf_per_service = vnf_per_service
        self.priority_level = priority_level
        self.priority_class = priority_class
        for vnf in range(self.vnf_per_service):
            self.vNetwork_service_details_er[self.episode][self.sfc][0][vnf][1]['priority_level'] = self.priority_level
            self.vNetwork_service_details_er[self.episode][self.sfc][0][vnf][1]['priority_class'] = self.priority_class  
            
            
    def emb_update(self, episode, sfc, vnf_per_service, emb_status, sfc_deployment_time):
        self.episode = episode
        self.sfc = sfc
        self.vnf_per_service = vnf_per_service
        self.emb_status = emb_status
        self.sfc_deployment_time = sfc_deployment_time
        #print("emb updated: ", episode, sfc, vnf_per_service, emb_status, sfc_deployment_time)
        for vnf in range(self.vnf_per_service):
            self.vNetwork_service_details_er[self.episode][self.sfc][0][vnf][1]['emb_status'] = emb_status
            self.vNetwork_service_details_er[self.episode][self.sfc][0][vnf][1]['sfc_deployment_time'] = sfc_deployment_time
    
    def traffic_update(self, episode, deployed_sfc, vnf_per_service, epi_traffic_packets_dict, epi_traffic_load_dict):
        self.episode = episode
        self.deployed_sfc = deployed_sfc
        self.vnf_per_service = vnf_per_service
        self.epi_traffic_packets_dict = epi_traffic_packets_dict
        self.epi_traffic_load_dict = epi_traffic_load_dict        
        for vnf in range(self.vnf_per_service):
            self.vNetwork_service_details_er[self.episode][self.deployed_sfc][0][vnf][1]['traffic_packets'] = self.epi_traffic_packets_dict[self.deployed_sfc]
            self.vNetwork_service_details_er[self.episode][self.deployed_sfc][0][vnf][1]['traffic_load'] = self.epi_traffic_load_dict[self.deployed_sfc]
                  

    def classifier_class_update(self, episode, sfc, vnf_per_service, traffic_pred, pred_num, pred_traffic_packet_per_sfc):
        #print("updating Classifer class")
        self.episode = episode
        self.sfc = sfc
        self.vnf_per_service = vnf_per_service
        self.traffic_pred = traffic_pred
        self.pred_num = pred_num
        self.pred_traffic_packet_per_sfc = pred_traffic_packet_per_sfc
        for vnf in range(self.vnf_per_service):
            self.vNetwork_service_details_er[self.episode][self.sfc][0][vnf][1]['classifier_class'] = self.traffic_pred
            self.vNetwork_service_details_er[self.episode][self.sfc][0][vnf][1]['traffic_packets'] = self.pred_num
            self.vNetwork_service_details_er[self.episode][self.sfc][0][vnf][1]['traffic_load'] = self.pred_traffic_packet_per_sfc
                  
        
    def overall_reset(self, num_episode, num_sfc, vnf_per_service):
        self.num_episode = num_episode
        self.num_sfc = num_sfc
        self.vnf_per_service = vnf_per_service
        for epi in self.vNetwork_service_details_er:
            for sfc in range(num_sfc):
                for vnf in range(vnf_per_service):
                    #print("vnf", vnf, epi[sfc][0][vnf])
                    epi[sfc][0][vnf][1]['microvnf_done_status'] = None
                    epi[sfc][0][vnf][1]['numMicroVNF'] =  None
                    epi[sfc][0][vnf][1]['microvnf_res_component'] =  None
                    epi[sfc][0][vnf][1]['decompose_candidate'] =  None
                    epi[sfc][0][vnf][1]['emb_status'] = None
                    epi[sfc][0][vnf][1]['sfc_deployment_time'] = None
                    epi[sfc][0][vnf][1]['traffic_packets'] = None 
                    epi[sfc][0][vnf][1]['traffic_load'] = None
                    epi[sfc][0][vnf][1]['classifier_class'] = None



In [10]:
# Hardcoded parameters
## Parameters
Algorithm = ['Algorithm: DDQN+ Target Networks + Experience Replay']
Hidden_layer, Neurons_per_HiddenLayer,drop_prob, Opimizer = 6, 300, 0, 'Optimizer: Adam' #Neural Network
learning_rate, gamma,  eps_min = 0.0001, 0.99, 0.01 #Learning = 0.001#
threshold_delay = 100 # Latency 
amount_local_reward, amount_local_penalty = 1000, -100 #Local Rewards
mem_size, batch_size, replace  = 500000, 32, 10000 # Memory size

alpha, beta, tau = 0.0001, 0.001, 0.001

num_episode, num_sfc = 1 ,50 #e and num of SFC
tr_mem_size, tr_batch_size = 10000, 32
test_mem_size, test_batch_size = 500, 128
acc_theshold_limit, n_splits, n_repeat = 0.8, 2, 2
vnf_per_service = 5 #[5, 10, 15]
max_vnf_resource = 5 #2
mean, var = 3, 0.4
overall_sfc = num_episode * num_sfc

threshold_percentage = 800 #80 # #abve 20% of resource the decompose is activated
threshold_decompose = (max_vnf_resource*threshold_percentage)/100
print(threshold_decompose)
eps_value = 0.3

full_nodal_avail = 1
threshold_nodal_decomp_percentage =  0 # 10 #
threshold_nodal_decomp = (full_nodal_avail*threshold_nodal_decomp_percentage)/100
print(threshold_nodal_decomp)


traffic_batch_size = 32
n_clusters = 2

classifier_models ={}

40.0
0.0


In [11]:
eps_dec = 1e-4 #3,4,5,6,7
print(eps_dec)

0.0001


In [12]:
# added on 12th April 2022
def Normalization(data, data_max, data_min):
    norm_data = ((data - data_min) / (data_max - data_min))
    return norm_data

In [13]:
global_services =Trail(num_episode= num_episode, 
                          num_sfc =num_sfc, 
                          max_vnf_resource = max_vnf_resource, 
                          vnf_per_service = vnf_per_service,
                          vNetwork_service_er =[], 
                          vNetwork_service_details_er=[],
                          vnf_link_resources = [], 
                          vnf_node_resources=[], 
                          mean = mean, 
                          var = var
                         )

### DDPG

In [14]:
# Critic Network
class CriticNetwork(nn.Module):
    def __init__(self, beta, n_state, fc_unit, n_actions, name):
        
        #print("Critic Network")
        super(CriticNetwork, self).__init__()
        
        self.beta      = beta
        self.n_actions = n_actions
        self.n_state   = n_state
        self.fc_unit   = fc_unit
        #self.drop      = nn.Dropout(p=drop_prob)
        self.name      = name
        
        self.fc1 = nn.Linear(*self.n_state, self.fc_unit)
        self.fc2 = nn.Linear(self.fc_unit, self.fc_unit)
        self.fc3 = nn.Linear(self.fc_unit, self.fc_unit) #hidden 3
        self.fc4 = nn.Linear(self.fc_unit, self.fc_unit) #hidden 4
        self.fc5 = nn.Linear(self.fc_unit, self.fc_unit) #hidden 5      
        self.fc6 = nn.Linear(self.fc_unit, self.fc_unit) #hidden 6
        
        #Layer Normalization
        #self.ln1 = nn.LayerNorm(self.fc_unit)
        #self.ln2 = nn.LayerNorm(self.fc_unit)
        #self.bn3 = nn.LayerNorm(self.fc_unit)
        #self.bn4 = nn.LayerNorm(self.fc_unit)
        #self.bn5 = nn.LayerNorm(self.fc_unit)
        #self.bn6 = nn.LayerNorm(self.fc_unit)
        
        #Batch Normalization
        self.bn1 = nn.BatchNorm1d(self.fc_unit)
        self.bn2 = nn.BatchNorm1d(self.fc_unit)
        self.bn3 = nn.BatchNorm1d(self.fc_unit)
        self.bn4 = nn.BatchNorm1d(self.fc_unit)
        self.bn5 = nn.BatchNorm1d(self.fc_unit)
        self.bn6 = nn.BatchNorm1d(self.fc_unit)
        
        self.action_value  = nn.Linear(self.n_actions, self.fc_unit)
        self.q             = nn.Linear(self.fc_unit, 1)
        
        f1 = 1./np.sqrt(self.fc1.weight.data.size()[0])
        self.fc1.weight.data.uniform_(-f1, f1)
        self.fc1.bias.data.uniform_(-f1, f1)

        f2 = 1./np.sqrt(self.fc2.weight.data.size()[0])
        self.fc2.weight.data.uniform_(-f2, f2)
        self.fc2.bias.data.uniform_(-f2, f2)
        
        f3 = 1./np.sqrt(self.fc3.weight.data.size()[0])
        self.fc3.weight.data.uniform_(-f3, f3)
        self.fc3.bias.data.uniform_(-f3, f3)
        
        f4 = 1./np.sqrt(self.fc4.weight.data.size()[0])
        self.fc4.weight.data.uniform_(-f4, f4)
        self.fc4.bias.data.uniform_(-f4, f4)
        
        f5 = 1./np.sqrt(self.fc5.weight.data.size()[0])
        self.fc5.weight.data.uniform_(-f5, f5)
        self.fc5.bias.data.uniform_(-f5, f5)
        
        f6 = 1./np.sqrt(self.fc6.weight.data.size()[0])
        self.fc6.weight.data.uniform_(-f6, f6)
        self.fc6.bias.data.uniform_(-f6, f6)

        f7 = 0.003
        self.q.weight.data.uniform_(-f7, f7)
        self.q.bias.data.uniform_(-f7, f7)

        f8 = 1./np.sqrt(self.action_value.weight.data.size()[0])
        self.action_value.weight.data.uniform_(-f8, f8)
        self.action_value.bias.data.uniform_(-f8, f8)

        self.optimizer = optim.Adam(self.parameters(), lr=self.beta, weight_decay=0.01)
        self.device = T.device('cpu') 
        self.to(self.device)

    def forward(self, state, action):
        #print("==> Critic action: ", action)
        layer = self.fc1(state)
        layer = self.bn1(layer)
        layer = F.relu(layer)
        
        layer = self.fc2(layer)
        layer = self.bn2(layer)
        layer = F.relu(layer)
        
        layer = self.fc3(layer)
        layer = self.bn3(layer)
        layer = F.relu(layer)
        
        layer = self.fc4(layer)
        layer = self.bn4(layer)
        layer = F.relu(layer)
        
        layer = self.fc5(layer)
        layer = self.bn5(layer)
        layer = F.relu(layer)
        
        layer = self.fc6(layer)
        layer = self.bn6(layer)
        layer = F.relu(layer)

        
        action_value       = self.action_value(action)
        state_action_value = F.relu(T.add(layer, action_value))
        state_action_value = self.q(state_action_value)
        #print("state_action_value : ", state_action_value )
        return state_action_value

In [15]:
# Actor Network
class ActorNetwork(nn.Module):
    def __init__(self, alpha, n_state, fc_unit, n_actions, name):
        
        super(ActorNetwork, self).__init__()
        #print("Actor Network")
        self.alpha     = alpha
        self.n_actions = n_actions
        self.n_state   = n_state
        self.fc_unit   = fc_unit
        #self.drop      = nn.Dropout(p=drop_prob)
        self.name      = name
        
        self.fc1 = nn.Linear(*self.n_state, self.fc_unit)
        self.fc2 = nn.Linear(self.fc_unit, self.fc_unit)
        self.fc3 = nn.Linear(self.fc_unit, self.fc_unit) #hidden 3
        self.fc4 = nn.Linear(self.fc_unit, self.fc_unit) #hidden 4
        self.fc5 = nn.Linear(self.fc_unit, self.fc_unit) #hidden 5      
        self.fc6 = nn.Linear(self.fc_unit, self.fc_unit) #hidden 6
        
        #Layer Normalization
        #self.bn1 = nn.LayerNorm(self.fc_unit)
        #print("bc1: ", self.bn1)
        #self.bn2 = nn.LayerNorm(self.fc_unit)
        #print("bc2: ", self.bn2)
        #self.bn3 = nn.LayerNorm(self.fc_unit)
        #self.bn4 = nn.LayerNorm(self.fc_unit)
        #self.bn5 = nn.LayerNorm(self.fc_unit)
        #self.bn6 = nn.LayerNorm(self.fc_unit)
        
        #Batch Normalization
        self.bn1 = nn.BatchNorm1d(self.fc_unit)
        self.bn2 = nn.BatchNorm1d(self.fc_unit)
        self.bn3 = nn.BatchNorm1d(self.fc_unit)
        self.bn4 = nn.BatchNorm1d(self.fc_unit)
        self.bn5 = nn.BatchNorm1d(self.fc_unit)
        self.bn6 = nn.BatchNorm1d(self.fc_unit)
        
        self.mu  = nn.Linear(self.fc_unit, self.n_actions)
        #print("mu", self.mu)
        
        f1 = 1./np.sqrt(self.fc1.weight.data.size()[0])
        self.fc1.weight.data.uniform_(-f1, f1)
        self.fc1.bias.data.uniform_(-f1, f1)

        f2 = 1./np.sqrt(self.fc2.weight.data.size()[0])
        self.fc2.weight.data.uniform_(-f2, f2)
        self.fc2.bias.data.uniform_(-f2, f2)
        
        f3 = 1./np.sqrt(self.fc3.weight.data.size()[0])
        self.fc3.weight.data.uniform_(-f3, f3)
        self.fc3.bias.data.uniform_(-f3, f3)
        
        f4 = 1./np.sqrt(self.fc4.weight.data.size()[0])
        self.fc4.weight.data.uniform_(-f4, f4)
        self.fc4.bias.data.uniform_(-f4, f4)
        
        f5 = 1./np.sqrt(self.fc5.weight.data.size()[0])
        self.fc5.weight.data.uniform_(-f5, f5)
        self.fc5.bias.data.uniform_(-f5, f5)
        
        f6 = 1./np.sqrt(self.fc6.weight.data.size()[0])
        self.fc6.weight.data.uniform_(-f6, f6)
        self.fc6.bias.data.uniform_(-f6, f6)

        f7 = 0.003
        self.mu.weight.data.uniform_(-f7, f7)
        self.mu.bias.data.uniform_(-f7, f7)

        self.optimizer = optim.Adam(self.parameters(), lr=self.alpha)
        self.device = T.device('cpu') 
        self.to(self.device)

    def forward(self, state):
        #print("Actor forward state: ", state)
        
        layer = self.fc1(state)
        layer = F.relu(self.bn1(layer))
        
        layer = self.fc2(layer)
        layer = F.relu(self.bn2(layer))
        
        layer = self.fc3(layer)
        layer = F.relu(self.bn3(layer))
        
        layer = self.fc4(layer)
        layer = F.relu(self.bn4(layer))
        
        layer = self.fc5(layer)
        layer = F.relu(self.bn5(layer))

        layer = self.fc6(layer)
        layer = F.relu(self.bn6(layer))
        
        #leaky Relu
        #leaky_act = nn.LeakyReLU(0.1)
        #L_layer = leaky_act(self.mu(layer))
        #print("Leaky Relu: ", L_layer)
        
        #relu
        #R_layer = F.relu(self.mu(layer))
        #print("Relu: ", R_layer)
        
        #linear
        layer = T.sigmoid(self.mu(layer))
        #print("sigmoid: ", layer)
        return layer 

In [16]:
## Primary Network ##

class PR_DDQN(nn.Module):
    # nn.Module for constructing more hidden layers.
    def __init__(self, lr, pr_n_actions, pr_n_state, fc_unit, drop_prob): #name, chkpt_dir
        
        #calls __inti__ method of nn.module class
        super(PR_DDQN, self).__init__()
        print("PR_DDQN")
        self.pr_n_state   = pr_n_state
        self.fc_unit      = fc_unit
        self.pr_n_actions = pr_n_actions
        self.drop         = nn.Dropout(p=drop_prob)
        
        # Creating Fully Connected Layers between Input and Hidden layer of 128 neurons
        self.fc1 = nn.Linear(*self.pr_n_state,self.fc_unit) #input - hidden 1
        self.fc2 = nn.Linear(self.fc_unit,self.fc_unit) #hidden 2 
        #self.fc3 = nn.Linear(self.fc_unit,self.fc_unit) #hidden 3
        #self.fc4 = nn.Linear(self.fc_unit,self.fc_unit) #hidden 4
        #self.fc5 = nn.Linear(self.fc_unit,self.fc_unit) #hidden 5      
        #self.fc6 = nn.Linear(self.fc_unit,self.fc_unit) #hidden 6
        #self.fc7 = nn.Linear(self.fc_unit,self.fc_unit) #hidden 7      
        #self.fc8 = nn.Linear(self.fc_unit,self.fc_unit) #hidden 8 
        #self.fc9 = nn.Linear(self.fc_unit,self.fc_unit) #hidden 9      
        #self.fc10 = nn.Linear(self.fc_unit,self.fc_unit) #hidden 10
        # Creating Fully Connected Layers between Hidden layer of 128 neurons and Outout layer.
        self.fc_op = nn.Linear(self.fc_unit, self.pr_n_actions) #output
        # Using Adam's Optimiser
        self.optimizer = optim.Adam(self.parameters(), lr=lr)  
        # Using Mean Square Error for calculating the Loss
        self.loss = nn.MSELoss() 
        #checking of the GPU is availible or not.
        self.device = T.device('cpu') 
        # sending the entire network to the device
        self.to(self.device) 
    
    # Feedforward
    def forward(self, state): #activation
        #print("forward state",state)
        layer = F.relu(self.drop(self.fc1(state))) # 1st hidden layer neuron activation
        #print("forward layer1:",layer1)
        layer = F.relu(self.drop(self.fc2(layer)))
        #layer = F.relu(self.drop(self.fc3(layer)))
        #layer = F.relu(self.drop(self.fc4(layer)))
        #layer = F.relu(self.drop(self.fc5(layer)))
        #layer = F.relu(self.drop(self.fc6(layer)))
        #layer = F.relu(self.drop(self.fc7(layer)))
        #layer = F.relu(self.drop(self.fc8(layer)))
        #layer = F.relu(self.drop(self.fc9(layer)))
        #layer = F.relu(self.drop(self.fc10(layer)))
        actions = self.fc_op(layer)# no activation function currently  #output later resul
        #print("forward actions", actions)
        #print("")
        return actions

In [17]:
## Primary Network ##

class LinearDDQN(nn.Module):
    # nn.Module for constructing more hidden layers.
    def __init__(self, lr, n_actions, n_state, fc_unit, drop_prob): #name, chkpt_dir
        
        #calls __inti__ method of nn.module class
        super(LinearDDQN, self).__init__()
        self.n_state   = n_state
        self.fc_unit   = fc_unit
        self.n_actions = n_actions
        self.drop      = nn.Dropout(p=drop_prob)
        # Creating Fully Connected Layers between Input and Hidden layer of 128 neurons
        self.fc1 = nn.Linear(*self.n_state,self.fc_unit) #input - hidden 1
        self.fc2 = nn.Linear(self.fc_unit,self.fc_unit) #hidden 2 
        self.fc3 = nn.Linear(self.fc_unit,self.fc_unit) #hidden 3
        self.fc4 = nn.Linear(self.fc_unit,self.fc_unit) #hidden 4
        self.fc5 = nn.Linear(self.fc_unit,self.fc_unit) #hidden 5      
        self.fc6 = nn.Linear(self.fc_unit,self.fc_unit) #hidden 6
        #self.fc7 = nn.Linear(self.fc_unit,self.fc_unit) #hidden 7      
        #self.fc8 = nn.Linear(self.fc_unit,self.fc_unit) #hidden 8 
        #self.fc9 = nn.Linear(self.fc_unit,self.fc_unit) #hidden 9      
        #self.fc10 = nn.Linear(self.fc_unit,self.fc_unit) #hidden 10
        # Creating Fully Connected Layers between Hidden layer of 128 neurons and Outout layer.
        self.fc_op = nn.Linear(self.fc_unit, self.n_actions) #output
        # Using Adam's Optimiser
        self.optimizer = optim.Adam(self.parameters(), lr=lr) 
        # Using Mean Square Error for calculating the Loss
        self.loss = nn.MSELoss() 
        #checking of the GPU is availible or not.
        #print(self.device)
        self.device = T.device('cpu') 
        #print(self.device)  
        # sending the entire network to the device
        self.to(self.device) 
    
    # Feedforward
    def forward(self, state): #activation
        #print("forward state",state)
        layer = F.relu(self.drop(self.fc1(state))) # 1st hidden layer neuron activation
        #print("forward layer1:",layer1)
        layer = F.relu(self.drop(self.fc2(layer)))   
        layer = F.relu(self.drop(self.fc3(layer)))
        layer = F.relu(self.drop(self.fc4(layer)))
        layer = F.relu(self.drop(self.fc5(layer)))
        layer = F.relu(self.drop(self.fc6(layer)))
        #layer = F.relu(self.drop(self.fc7(layer)))
        #layer = F.relu(self.drop(self.fc8(layer)))
        #layer = F.relu(self.drop(self.fc9(layer)))
        #layer = F.relu(self.drop(self.fc10(layer)))
        actions = self.fc_op(layer) # no activation function currently  #output later resul
        #print("forward actions", actions)
        #print("")
        return actions

### Noise


In [18]:
class OUActionNoise():
    def __init__(self, mu, sigma=0.15, theta=0.2, dt=1e-2, x0=None):
        self.theta = theta
        self.mu = mu
        self.sigma = sigma
        self.dt = dt
        self.x0 = x0
        self.reset()
        #print("==> NOISE")

    def __call__(self):
        x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + \
                self.sigma * np.sqrt(self.dt) * np.random.normal(size=self.mu.shape)
        self.x_prev = x
        return x

    def reset(self):
        self.x_prev = self.x0 if self.x0 is not None else np.zeros_like(self.mu)

## Agent

In [19]:
class DDPG_Agent():
    def __init__(self, sch_n_state, sch_n_actions, alpha, beta, tau,
                 sfc_cnt, mem_size, batch_size, gamma, sch_model,
                 sch_state_memory, sch_action_memory, sch_reward_memory,
                 sch_next_state_memory, sch_terminal_memory, Neurons_per_HiddenLayer, 
                 algo = None, env_name = None,
                 chkpt_dir ='tmp/dqn'):
        
        #added on 02112021
        self.sch_model = sch_model
        print("DDPG agent")
        self.sch_n_state   = sch_n_state #Input Neurons
        self.sch_n_actions = sch_n_actions
        self.alpha        = alpha #alpha
        self.beta         = beta 
        self.sfc_cnt      = sfc_cnt
        self.gamma        = gamma  # Discounting Factor
        self.mem_size     = mem_size # maximum memory size
        self.batch_size   = batch_size
        self.algo         = algo
        self.env_name     = env_name
        self.chkpt_dir    = chkpt_dir 
        self.mem_cntr     = 0 #vnf_cnt
        self.iter_cnt     = 0 #vnf_cnt
        #print("Memory count",self.mem_cntr)
        #print("Iteration count",self.iter_cnt)
        self.tau          = tau   
        #print("tau: ", tau)
                 
        # Initializing the different types of memory
        self.sch_state_memory      = sch_state_memory
        self.sch_action_memory     = sch_action_memory
        self.sch_reward_memory     = sch_reward_memory
        self.sch_next_state_memory = sch_next_state_memory
        self.sch_terminal_memory   = sch_terminal_memory

        #Noise
        self.noise = OUActionNoise(mu=np.zeros(self.sch_n_actions))
        #print("self.noise: ", self.noise)
        
        #Actor-Critic 
        self.actor         = ActorNetwork(alpha     = self.alpha, 
                                          n_state   = self.sch_n_state, 
                                          fc_unit   = Neurons_per_HiddenLayer,
                                          n_actions = self.sch_n_actions, 
                                          name      = 'actor')
        
        self.critic        = CriticNetwork(beta      = self.beta, 
                                           n_state   = self.sch_n_state,  
                                           fc_unit   = Neurons_per_HiddenLayer,
                                           n_actions = self.sch_n_actions, 
                                           name      = 'critic')

        self.target_actor  = ActorNetwork(alpha     = self.alpha,
                                          n_state   = self.sch_n_state, 
                                          fc_unit   = Neurons_per_HiddenLayer,
                                          n_actions = self.sch_n_actions,
                                          name      = 'target_actor')

        self.target_critic = CriticNetwork(beta      = self.beta,  
                                           n_state   = self.sch_n_state,
                                           fc_unit   = Neurons_per_HiddenLayer,
                                           n_actions = self.sch_n_actions,
                                           name      = 'target_critic')
        
        self.update_network_parameters(tau=1)
     
    
    # Storing the transition in the agent's memory
    def store_transition(self, sch_state, sch_action, sch_reward, sch_next_state, sch_done, sfc_cnt):
        #print("==> DDPG storing")
        self.sfc_cnt = sfc_cnt
        #print("sfc_cnt: ", sfc_cnt)
        #print("pr_action: ", pr_action)
        self.mem_cntr = sfc_cnt # when to update the weights of the target network with the weights of the eval network
        #finding for the Position of unoccupied memory, 
        #if the position value is greater than the memory size, then the overwrittien on the memories will occur
        index = self.mem_cntr % self.mem_size  #% = position of the unoccupied memory
        
        self.sch_state_memory[index]  = sch_state
        self.sch_action_memory[index] = sch_action
        self.sch_reward_memory[index] = sch_reward
        self.sch_next_state_memory[index] = sch_next_state
        self.sch_terminal_memory[index] = sch_done
        #self.mem_cntr += 1
        
        #print("==> Index", index, sch_state, sch_action)
        #print("self.sch_state_memory:", self.sch_state_memory)
        #print("self.sch_action_memory:", self.sch_action_memory)
        #print("self.sch_reward_memory:", self.sch_reward_memory)
        #print("self.sch_terminal_memory:", self.sch_terminal_memory)

    def choose_action(self, sch_observation, sch_n_state): 
        #print("==> DDPG action selection")
        #print(sch_observation, sch_n_state)
        self.actor.eval()
        state = T.tensor([sch_observation], dtype = T.float).to(self.actor.device)
        #print("Choose_action : state: ", state)
        mu = self.actor.forward(state).to(self.actor.device)
        #print("mu: ", mu)
        mu_prime = mu + T.tensor(self.noise(), dtype = T.float).to(self.actor.device)
        #print("mu_prime: ",mu_prime)
        self.actor.train()
        #print("pr_level: ", mu_prime.cpu().detach().numpy()[0])
        sch_level = abs(mu_prime.cpu().detach().numpy()[0,0])
        #print("sch_level: ", sch_level)
        #print("")
        return sch_level
    

    def learn(self):
        #print("==> DDPG learning")
        
        self.iter_cnt = self.sfc_cnt
        
        #print("self.iter_cnt: ", self.iter_cnt, " self.mem_cntr: ", self.mem_cntr )
        if self.mem_cntr < self.batch_size: 
            return

        max_mem           = min(self.mem_cntr, self.mem_size) 
        batch             = np.random.choice(max_mem, self.batch_size)              
        batch_index       = np.arange(self.batch_size, dtype=np.int32)
        #print("batch: ", batch)
        
        sch_state_batch       = T.tensor(self.sch_state_memory[batch]).to(self.actor.device)
        sch_action_batch      = T.tensor(self.sch_action_memory[batch]).to(self.actor.device)
        sch_reward_batch      = T.tensor(self.sch_reward_memory[batch]).to(self.actor.device)    
        sch_next_states_batch = T.tensor(self.sch_next_state_memory[batch]).to(self.actor.device)
        sch_dones_batch       = T.tensor(self.sch_terminal_memory[batch]).to(self.actor.device)
        
        #print("==> sch_state_batch: ", sch_state_batch)
        #print("==> sch_action_batch: ", sch_action_batch)
        #print("==> sch_next_states_batch:", sch_next_states_batch)
        
        target_actions = self.target_actor.forward(sch_next_states_batch)
        #print("target_actions:", target_actions)
        
        critic_value_  = self.target_critic.forward(sch_next_states_batch, target_actions)
        critic_value   = self.critic.forward(sch_state_batch, sch_action_batch)
        
        critic_value_[sch_dones_batch] = 0.0
        critic_value_ = critic_value_.view(-1)
        
        target = sch_reward_batch + self.gamma*critic_value_
        target = target.view(self.batch_size, 1)
        
        self.critic.optimizer.zero_grad()
        critic_loss = F.mse_loss(target, critic_value)
        critic_loss.backward()
        self.critic.optimizer.step()
        
        self.actor.optimizer.zero_grad()
        actor_loss = -self.critic.forward(sch_state_batch, self.actor.forward(sch_state_batch))
        actor_loss = T.mean(actor_loss)
        actor_loss.backward()
        self.actor.optimizer.step()
        self.update_network_parameters()
        
    
    def update_network_parameters(self, tau = None):
        #print("inside the network update")
        if tau is None: 
            tau = self.tau
        #print("tau", tau)

        actor_params         = self.actor.named_parameters()
        critic_params        = self.critic.named_parameters()
        target_actor_params  = self.target_actor.named_parameters()
        target_critic_params = self.target_critic.named_parameters()
        
        critic_state_dict        = dict(critic_params)
        actor_state_dict         = dict(actor_params)
        target_critic_state_dict = dict(target_critic_params)
        target_actor_state_dict  = dict(target_actor_params)
        #print(actor_state_dict, critic_state_dict )
        
        
        for name in critic_state_dict:
            critic_state_dict[name] = tau*critic_state_dict[name].clone() + \
                                (1-tau)*target_critic_state_dict[name].clone()

        for name in actor_state_dict:
             actor_state_dict[name] = tau*actor_state_dict[name].clone() + \
                                 (1-tau)*target_actor_state_dict[name].clone()
                

        self.target_critic.load_state_dict(critic_state_dict, strict=False)
        self.target_actor.load_state_dict(actor_state_dict, strict=False)        
        
        

In [20]:
## u ## updated on 2nd oct 2020
## Creating an agent and construting all th required elements for implementing it.
class LinearDDQNAgent():
    def __init__(self, n_state, n_actions, epsilon, lr,
                 vnf_cnt, mem_size, batch_size, state_memory, action_memory, reward_memory,
                 next_state_memory, terminal_memory,Neurons_per_HiddenLayer, drop_prob,
                 gamma, eps_dec, eps_min,replace, model,
                 algo = None, env_name = None,
                 chkpt_dir ='tmp/dqn'):
                
        print("DDQN Linear agent")
        
        #added on 02112021
        self.model = model
        #print(self.model)
        
        self.n_state  = n_state #Input Neurons
        self.n_actions = n_actions #Output Neurons = Num of HVS ina network
        self.epsilon = epsilon
        self.lr = lr #Learning Rate
        self.vnf_cnt = vnf_cnt
        self.gamma = gamma  # Discounting Factor
        self.eps_dec = eps_dec # Epsilon rate decay
        self.eps_min = eps_min 
        self.mem_size = mem_size # maximum memory size
        self.batch_size = batch_size
        self.replace_target_cnt = replace
        self.algo =algo
        self.env_name = env_name
        self.chkpt_dir = chkpt_dir 
        self.mem_cntr = 0 #vnf_cnt
        self.iter_cnt = 0 #vnf_cnt
        #print("Memory count",self.mem_cntr)
        #print("Iteration count",self.iter_cnt)
        
        # Primary network
        self.Q_eval = LinearDDQN(lr, n_actions=n_actions, n_state=n_state, 
                                fc_unit = Neurons_per_HiddenLayer, drop_prob =drop_prob)
     
        #print("self Q:",self.Q_eval)
        
        #Target Network (backpropagation or gradient decend is not performed on this network)
        self.Q_target = LinearDDQN(lr, n_actions=n_actions, n_state=n_state, 
                                fc_unit = Neurons_per_HiddenLayer, drop_prob=drop_prob)
            
        #print("self Q target:",self.Q_target)
        
        # Initializing the different types of memory
        self.state_memory = state_memory
        #print("state_memory", state_memory)
        #print("self.state_memory", self.state_memory)
        self.action_memory = action_memory
        self.reward_memory = reward_memory
        self.next_state_memory = next_state_memory
        #print("self.next_state_memory", self.next_state_memory)
        self.terminal_memory = terminal_memory

     
    # Storing the transition in the agent's memory
    def store_transition(self, state, action, reward, next_state, done, vnf_cnt):
        self.vnf_cnt = vnf_cnt
        self.mem_cntr = vnf_cnt # when to update the weights of the target network with the weights of the eval network
        #finding for the Position of unoccupied memory, 
        #if the position value is greater than the memory size, then the overwrittien on the memories will occur
        index = self.mem_cntr % self.mem_size  #% = position of the unoccupied memory
        
        #print("self.mem_cntr",self.mem_cntr)
        #print("self.mem_size",self.mem_size)
        #print("Index", index)
        
        self.state_memory[index]  = state
        #print("State", state)
        #print("State value inside the replay buffer for given index", self.state_memory[index])
        #print("State value inside the replay buffer", self.state_memory)
        self.action_memory[index] = action
        #print("action value inside the replay buffer", self.action_memory[index])
        self.reward_memory[index] = reward
        #print("reward value inside the replay buffer", self.reward_memory[index])
        self.next_state_memory[index] = next_state
        #print("next_state value inside the replay buffer", self.next_state_memory[index])
        self.terminal_memory[index] = done
        #print("terminal_memory value inside the replay buffer", self.terminal_memory[index])
        
        #once all is done, then the position is incremented
        self.mem_cntr += 1
    
    ## Greedy method for action selection
    def choose_action(self, observation, norm_vals_sorted_action, vals_sorted_action, action_space): # for the current state = observation
        self.action_space = action_space 
        self.norm_vals_sorted_action = norm_vals_sorted_action
        #print("Norm action space",self.norm_vals_sorted_action)
        self.vals_sorted_action = vals_sorted_action
        #exploration_rate_threshold = random.uniform(0,1)
        #print("observation",observation)
        limit = np.random.random()
        #print("limit", limit)
        if limit > self.epsilon:
            
            # Expliotation
            state   = T.tensor([observation], dtype=T.float).to(self.Q_eval.device)# make sure that the state is in tensor
            #print("state", state)
            actions = self.Q_eval.forward(state) # retriving the actions produced by the DQN
            
            #print("Expliotation: Actions",actions)
            action  = T.argmax(actions).item() # achieving the max action element for that particular state
            #print("action: ", action)
        else:
            #exploration
            #print("Norm action space",self.norm_vals_sorted_action)
            #action  = np.random.choice(self.action_space, p = self.norm_vals_sorted_action) #if the exploration rate is less than epsilon value, then random choice of action is done.
            
            #print("norm", self.norm_vals_sorted_action)
            action  = rand.choices(self.action_space) 
            #action  = rand.choices(self.action_space, weights = self.vals_sorted_action) #if the exploration rate is less than epsilon value, then random choice of action is done.
            #print(self.vals_sorted_action)
            #print("trail action",action)
            action = action[0]
            #print("Exploration: Action", action)
            
        #print("selected action:",action)
        return action
    
     
    ## How Agent will learn from the experiences. The learning function will take current state, action, reward and next state.
    
    def learn(self):
        
        #print("==> Inside learn")
        # When to start the learning? 
        
        # Waiting until the agent fills a batch size of memories and then returns if it has not
        #print("self.mem_cntr:",self.mem_cntr)
        #print("self.batch_size:",self.batch_size)
        #print("++> self.mem_cntr",self.mem_cntr)
        self.iter_cnt = self.vnf_cnt
        
        if self.mem_cntr < self.batch_size: # checking the batch size
            return
        
        # Zero the gradient on the optimizer.        
        self.Q_eval.optimizer.zero_grad()
        #print(self.Q_eval.optimizer.zero_grad)
        
        # Updating the Target network after every C iterations(transistions)
        if self.replace_target_cnt is not None and self.iter_cnt % self.replace_target_cnt == 0:
            self.Q_target.load_state_dict(self.Q_eval.state_dict())
            #print("replacing ")
        
        # figuring out the how much memory is filled up in the buffer.
        # checking 
        max_mem = min(self.mem_cntr, self.mem_size) # figuring out the last pposition of the stored memory.
        #print("max_mem",max_mem)
        
        #uniformly sampling the trasitions from the buffer
        batch = np.random.choice(max_mem, self.batch_size, replace =False) 
        #print("batch",batch)
        # replace is false so that there is no repeat in the transitions
        
        
        batch_index = np.arange(self.batch_size, dtype=np.int32)
        #print("batch_index",batch_index)
        
        state_batch  = T.tensor(self.state_memory[batch]).to(self.Q_eval.device)
        #print("state_batch",state_batch)
        action_batch = self.action_memory[batch]
        #print("action_batch",action_batch)
        reward_batch = T.tensor(self.reward_memory[batch]).to(self.Q_eval.device)    
        #print("reward_batch",reward_batch)
        next_states_batch = T.tensor(self.next_state_memory[batch]).to(self.Q_eval.device)
        #print("next_states_batch",next_states_batch)
        dones_batch      = T.tensor(self.terminal_memory[batch]).to(self.Q_eval.device)
        #print("dones_batch", dones_batch)

        
        #editing 24th feb 2021
        # Calculating the Q estimate
        # This is the value what the network estimated for that state
        q_pred = self.Q_eval.forward(state_batch)[batch_index, action_batch]
        #print("Predcit",q_pred)
        
        q_next = self.Q_target.forward(next_states_batch)
        
        # Target Q value
        q_eval = self.Q_eval.forward(next_states_batch)
        #print("Q next",q_eval)
        
        max_actions = T.argmax(q_eval, dim=1)
        
        q_next[dones_batch]= 0.0
        
        q_target = reward_batch + self.gamma*q_next[batch_index,max_actions]
        #print("Target", q_target)
        
        # calcualting the loss between the what actually the network should have predicted and what it has done
        # Loss is the MSE
        loss = self.Q_eval.loss(q_target, q_pred).to(self.Q_eval.device)
        #print(loss)
        #self.service_loss.append(loss)
        #print(self.service_loss)
        
        
        ## Defining the Exploration rate decay
        self.epsilon = self.epsilon - self.eps_dec \
                        if self.epsilon > self.eps_min else self.eps_min
        #print("Epsilon Dec",self.epsilon)
        
        
        #Backpropagation
        #print("Backpropagation")
        loss.backward()
        self.Q_eval.optimizer.step()  # step our optimizer 
        #print(self.Q_eval.optimizer.step)
        self.iter_cnt += 1
        #print('==> self.iter_cnt',self.iter_cnt)
        #print(self.Q_eval.optimizer.step)


In [21]:
#adding on 02112021

class LinearDQNAgent():
    def __init__(self, n_state, n_actions, epsilon, lr,
                 vnf_cnt, Neurons_per_HiddenLayer, drop_prob,
                 gamma, eps_dec, eps_min, model,
                 algo = None, env_name = None,
                 chkpt_dir ='tmp/dqn'):
        
        print("Linear DQN agent")
        self.model = model
        #print(self.model)
        #print(lr)
        
        self.n_state  = n_state #Input Neurons
        self.n_actions = n_actions #Output Neurons = Num of HVS ina network
        self.epsilon = epsilon
        self.lr = lr #Learning Rate
        self.vnf_cnt = vnf_cnt
        self.gamma = gamma  # Discounting Factor
        self.eps_dec = eps_dec # Epsilon rate decay
        self.eps_min = eps_min 
        self.algo =algo
        self.env_name = env_name
        self.chkpt_dir = chkpt_dir
        self.iter_cnt = 0 #vnf_cnt
        
        #Primary network
        #print("before Q_eval")
        self.Q_eval = LinearDDQN(lr, n_actions=n_actions, n_state=n_state, fc_unit = Neurons_per_HiddenLayer,
                                    drop_prob =drop_prob)
        #print("self.Q_eval: ",self.Q_eval)
        
    def store_transition(self, state, action, reward, next_state, done, vnf_cnt):
        self.state = state
        #print("self.state", self.state)
        self.action = action
        self.reward = reward
        self.next_state = next_state
        self.done = False #done 
        self.vnf_cnt = vnf_cnt
        
        
        ## Greedy method for action selection
    def choose_action(self, observation, norm_vals_sorted_action, vals_sorted_action, action_space): # for the current state = observation
        self.action_space = action_space 
        self.norm_vals_sorted_action = norm_vals_sorted_action
        #print("Norm action space",self.norm_vals_sorted_action)
        self.vals_sorted_action = vals_sorted_action
        #exploration_rate_threshold = random.uniform(0,1)
        #print("observation",observation)
        limit = np.random.random()
        #print("limit", limit)
        if limit > self.epsilon:
            
            # Expliotation
            state   = T.tensor([observation], dtype=T.float).to(self.Q_eval.device)# make sure that the state is in tensor
            #print("state", state)
            actions = self.Q_eval.forward(state) # retriving the actions produced by the DQN
            
            #print("Expliotation: Actions",actions)
            action  = T.argmax(actions).item() # achieving the max action element for that particular state
            #print("action: ", action)
        else:
            #exploration
            #print("Norm action space",self.norm_vals_sorted_action)
            #action  = np.random.choice(self.action_space, p = self.norm_vals_sorted_action) #if the exploration rate is less than epsilon value, then random choice of action is done.
            #print("norm", self.norm_vals_sorted_action)
            action  = rand.choices(self.action_space) 
            #action  = rand.choices(self.action_space, weights = self.vals_sorted_action) #if the exploration rate is less than epsilon value, then random choice of action is done.
            #print(self.vals_sorted_action)
            #print("trail action",action)
            action = action[0]
            #print("Exploration: Action", action)
            
        #print("selected action:",action)
        return action
    
    def learn(self):
        # Zero the gradient on the optimizer.        
        self.Q_eval.optimizer.zero_grad()
        #self.done = False
        #print("DQN")
        #print("==> self.state", self.state)
        #Converting to tensor form
        states = T.tensor(self.state, dtype=T.float).to(self.Q_eval.device) 
        #print("Learn State",states)
        actions = T.tensor(self.action).to(self.Q_eval.device) 
        #print("Learn Action",actions)
        rewards = T.tensor([self.reward]).to(self.Q_eval.device) 
        #print("Learn Reward ",rewards)
        next_states = T.tensor(self.next_state, dtype=T.float).to(self.Q_eval.device) 
        #print("Learn Next state",next_states)
        dones =  T.tensor(self.done).to(self.Q_eval.device)

        #print("DQN")
        #DQN estimation
        q_pred = self.Q_eval.forward(states)[actions]
        #print("Predcit", q_pred)

        #changing here on 26th oct 2021
        q_next = self.Q_eval.forward(next_states).max()
        #print("q_next", q_next)
        #q_next[dones]= 0.0
        q_target = rewards + self.gamma*q_next
        #print("==> Target", q_target)
        
        
        # calcualting the loss between the what actually the network should have predicted and what it has done
        # Loss is the MSE
        loss = self.Q_eval.loss(q_target, q_pred).to(self.Q_eval.device)
        #print(loss)
        #self.service_loss.append(loss)
        #print(self.service_loss)
        
        ## Defining the Exploration rate decay
        self.epsilon = self.epsilon - self.eps_dec \
                        if self.epsilon > self.eps_min else self.eps_min
        #print("Epsilon Dec",self.epsilon)
        
        
        #Backpropagation
        #print("Backpropagation")
        loss.backward()
        self.Q_eval.optimizer.step()  # step our optimizer 
        #print(self.Q_eval.optimizer.step)
        self.iter_cnt += 1
        #print('==> self.iter_cnt',self.iter_cnt)
        #print(self.Q_eval.optimizer.step)


In [22]:
#adding on 01032022
class HeuAgent():
    def __init__(self, n_state, n_actions):
        
        print("Heuristic model")
        #print("State", n_state)
        #print("Action", n_actions)
        
        self.n_state = n_state
        self.n_actions = n_actions
    
    def choose_action(self, observation, norm_vals_sorted_action, vals_sorted_action, action_space):
        #print("++++++ Inside Right Choose Action class++++++")
        self.observation = observation
        #print("self.observation",self.observation)
        
        self.norm_vals_sorted_action = norm_vals_sorted_action
        #print("self.norm_vals_sorted_action", self.norm_vals_sorted_action)
        
        self.vals_sorted_action = vals_sorted_action
        #print("self.vals_sorted_action", self.vals_sorted_action)
        
        self.action_space = action_space
        #print("self.action_space", self.action_space)
        
        action = np.random.choice(self.action_space, p = self.norm_vals_sorted_action) #if the exploration rate is less than epsilon value, then random choice of action is done.
        #print("Normalized: Action", action)
        
        #action = rand.choices(self.action_space, weights = self.vals_sorted_action)
        #action = action[0]
        #print("Sorted value: Action", action)
         
        # without any infromation about the resource avaialability    
        #action = random.choice(self.action_space)
        #action = action[0]
        return action
    
     
    def store_transition(self, state, action, reward, next_state, done, vnf_cnt):
        pass
    
    def learn(self):
        self.epsilon = 1
        pass
        
        

## Experience Replay

Concept:
THe transition, i.e., the for an observed state an action value is estimated by the network. According to the state-action pair the environment provide the rewards as a feedback to the agent along with the next state. These values state, action, reward and next state are stored as memory. From this memory the transitions are given to the traget network for estimating the loss function. That is analysisng the performce of the policy network (Primamry network). Thus I am contructing the classes called ReplayBuffer or ReplayMemory to store the transitions. 

I have constructed two types of constructor for experiment. 

In class ReplayBuffer, stored the values of space seperately. That is the state, actions, rewards, and nxt state are stored in a seperate array and using the index values these are formed as Transitions.

### PER: Sumtree
The ‘sum-tree’ data structure used here is very similar in spirit to the array representation of a binary heap. However, instead of the usual heap property, the value of a parent node is the sum of its children. Leaf nodes store the transition priorities and the internal nodes are intermediate sums, with the parent node containing the sum over all priorities, p_total. This provides a efficient way of calculating the cumulative sum of priorities, allowing O(log N) updates and sampling.

Additional useful links:
Good tutorial about SumTree data structure:  https://adventuresinmachinelearning.com/sumtree-introduction-python/
How to represent full binary tree as array: https://stackoverflow.com/questions/8256222/binary-tree-represented-using-array

## Main

Creating a dummy catalog, stating this info has been recoved or is available to the VNF descriptor. Thus the descriptor will know the extact name or in this case ID of the micro vnfs. 

Since the allocation of the ids are performed randomly. For example: for SFC:2, the vnf0 is decommposed into 6 microvnfs with various resouces demand such as 
microvnf    Resource      ID
0             1           23 <==
1             2           17
2             3           15
3             1           20
4             1            1
5             1           41

microvnf    Resource      ID
0             1           27
1             2           48
2             2           23 <==
3             2           35
4             2           11

same id cant have different resouce requirment, thus we have categorized the resouces according to the microvnf ID; Declared the dependency



In [23]:
class microVNFFG():
    def __init__(self,cpu_res, nodal_avail):
        
        self.cpu_res = cpu_res
        self.nodal_avail = nodal_avail
        
    ## Resource initialization for the micro vnfs
    def micro_resource_init(cpu_res, nodal_avail, granular_index):
        #print("granular_index", granular_index)
        #print("cpu_res",cpu_res,"nodal_avail",nodal_avail)
        granular_idx = granular_index 
        i = cpu_res
        exclude = [i]
        while i > 0:
            #print("while i ", i)
            if i <= 2:
                granular_index = i
                #print("i <= 2")
                #print("granular_index", granular_index)
            else:
                granular_index = round(cpu_res*nodal_avail)
                #print("else")
                #print("==.>granular_index", granular_index)
                if granular_index == 0:
                    if granular_idx == 1:
                        granular_index = granular_idx
                        #print("==granular_index", granular_index)
                        
                    else:
                        #print("empty microvnf init: break")
                        break
            #print("granular_index", granular_index)
            n = rand.randint(1,granular_index)
            #print(n)
            
            if n == 1:
                n = n
                #print("n")
            else:
                while n in exclude:
                    #print("stuck here")
                    n = rand.randint(1, i)           
            yield n
            i -= n
        
    
    # Generating the IDs for the decomposed microvnfs, according to the requested resource with the help of the catalog. 

    def microvnfid_init(micro_vnf, micro_vnf_req, unique_id, catalog):
        # constraint 1: setting the upper and lower limit 
        #print("catalog: ",catalog, "micro_vnf: ", micro_vnf, "micro_vnf_req: ", micro_vnf_req)
        max_id = catalog[micro_vnf_req] # setting the higher bound for the id generation
        #print("max_id: ", max_id)
        
        if micro_vnf_req == 1:
            min_id = 0 #setting the lower bound for the id generation
        else:
            min_id = catalog[micro_vnf_req-1]+1
        #print("min_id", min_id)
        id_gen = rand.randint(min_id,max_id)
        #print("id_gen: ",id_gen)

        # constaint 2: to not repeat any micro vnf id.  unique id represents the ids of a particular decomposed microvnf 
        while id_gen in unique_id:
            id_gen = rand.randint(min_id,max_id)
            #print("in while break here")
        return id_gen   
    
    def microvnf_descriptor(max_vnf_resource):
        catalog = {}
        count = max_vnf_resource
        for resource in range(1, max_vnf_resource):
            catalog[resource] = count
            count += 5
        return catalog
    

In [24]:
#print(list(microVNFFG.micro_resource_init(cpu_res = 5, nodal_avail = 0.8, granular_index =10)))
#cpu_res, nodal_avail = 20, 0.8
#granular_index = 4
#max_vnf_resource  = 10
#unique_id =[]
#qq = microVNFFG(cpu_res,nodal_avail)
#catalog = microVNFFG.microvnf_descriptor(max_vnf_resource)
#print("catalog",catalog)
#microVNFFG.microvnfid_init(micro_vnf = 1, micro_vnf_req = 2, unique_id = unique_id, catalog = catalog)

In [25]:
class microvnf_construction():
    def __init__(self,max_vnf_resource, episode, sfc, vnf, cpu_req, n, val, pr_level, sfc_rel, traffic_pred):
        
        self.max_vnf_resource = max_vnf_resource
        self.episode = episode
        self.sfc = sfc
        self.vnf = vnf
        self.cpu_req = cpu_req
        self.n = n
        self.val = val
        self.pr_level = pr_level
        self.sfc_rel = sfc_rel
        self.traffic_pred = traffic_pred 
        
        
    # microvnf construction
    def microvnf(self, microvnf_per_vnf, current_cpu_req, micro_vnf_res, decompose_candidate):
        #print("micro vnf const site", self.val)
        seed_array = [1] #[1,3,4]
        rand_seed = rand.choice(seed_array)
        #print(rand_seed)
        eps_value = 0.1
        #print("microvnf_per_vnf inside the def",microvnf_per_vnf)
        #print("current_cpu_req",current_cpu_req)
        
        p = (1 + eps_value)*(math.log(microvnf_per_vnf)/microvnf_per_vnf)

        er_graph_seed_microvnf = nx.erdos_renyi_graph(microvnf_per_vnf,p,seed=rand_seed, 
                                                      directed =True) 
        #generating the graph
        #nx.draw(er_graph_seed_microvnf, with_labels=True) 
        #plt.show()
        
        #print("==>",microvnf_per_vnf,current_cpu_req, micro_vnf_res, decompose_candidate)

        unique_id = []
        for micro_vnf in er_graph_seed_microvnf.nodes():
            #print("Micro VNF: ", micro_vnf)
            id_gen = microVNFFG.microvnfid_init(micro_vnf,
                                                micro_vnf_res[micro_vnf], 
                                                unique_id, 
                                                microVNFFG.microvnf_descriptor(self.max_vnf_resource))
            unique_id.append(id_gen)

            if micro_vnf == 0:
                microVNF_position = 'head'
            elif micro_vnf == (microvnf_per_vnf - 1):
                microVNF_position = 'tail'
            else:
                microVNF_position = 'middle'
            

            er_graph_seed_microvnf.add_node(micro_vnf,
                                               episode = self.episode,
                                               sfc = self.sfc,
                                               vnf = self.vnf,
                                               vnf_req = current_cpu_req,
                                               pr_level = self.pr_level,
                                               traffic_pred = self.traffic_pred, 
                                               #sfc_rel = self.sfc_rel,
                                               numMicroVNF = microvnf_per_vnf,
                                               microVNF = micro_vnf,
                                               microVNF_ID = id_gen,
                                               micro_cpu_req = micro_vnf_res[micro_vnf],
                                               microVNF_pos = microVNF_position,
                                               decompose_candidate = decompose_candidate
                                              )
        #print("Unique Id:", unique_id)
        self.microvnf_node_resources= list(er_graph_seed_microvnf.nodes(data=True)) 
        #print(self.microvnf_node_resources)


        # Micro virtual link
        microlink_count = 0
        for micro_link in er_graph_seed_microvnf.edges():
            #INITIALIZATION RESOURCE
            # Allocating the bandwidth resouce.
            #print(micro_link)
            er_graph_seed_microvnf.add_edge(micro_link[0],micro_link[1],
                                            src_vnf  = micro_link[0],
                                            dest_vnf = micro_link[1],
                                            episode = self.episode,
                                            sfc = self.sfc,
                                            vnf = self.vnf,
                                            numVNF = self.n,
                                            microlink_num = microlink_count,
                                            bw  = rand.randrange(1, 2),#Mbps
                                            latency = rand.randrange(1,2) #ms
                                            )
            microlink_count += 1
            #print("Total Micro Link", microlink_count)
        # Combining the nodes and edges per VNF-FG
        self.microvnf_link_resources = list(er_graph_seed_microvnf.edges(data=True))
        #print(self.microvnf_link_resources)

        return self.microvnf_node_resources, self.microvnf_link_resources, unique_id
    
    # micro vnf
    def microvnf_agent(self, episode, sfc, vnf, decomp, n_actions, lr, epsilon, mem_size, batch_size, vnf_cnt, gamma, 
                       eps_dec, eps_min, traffic_pred,traffic_load,
                       replace, state_memory, action_memory,reward_memory, next_state_memory, terminal_memory, 
                       Neurons_per_HiddenLayer, drop_prob, chkpt_dir, algo, env_name, count_VNF, done, 
                       amount_local_reward, reward_per_current_vnfstates, microvnf_accpeted_action, accpeted_action,
                       updated_path_perVNF, reward_per_current_states, nodal_avail, sum_resouces, state_space_size,
                       threshold_delay, rewardlink_per_current_states, vnf_link_resources, temp_accepted_action, 
                       t_microvnf_accpeted_action, prob_val, temp_val, original_actionspace,  prev_maction, 
                       min_pr,max_pr, mvf_t_max, mvf_t_min, mvf_old_t_current, new_reward_per_current_states):
        
        #print("inside mvnf agent: accpeted_action", accpeted_action)
        #print("reward_per_current_states: ", reward_per_current_states)
        #print("inside mvnf agent: micro accpeted_action", microvnf_accpeted_action)
        count_microvnf, count_updated_path_pervnf = 0, 0
        my_dict = {}
        micro_sorted_action = {} 
        micro_keys_sorted_action = []
        reward_mvnfs = []
        dict_node_avail = {}

        for node, available_res in env.network_nodes:
            if available_res['emb_vnf'] !=0:
                #print("inside the env.network")
                #print(">> Avaliable node",node)
                my_dict[node] = available_res['emb_vnf']
                micro_sorted_action = dict(sorted(my_dict.items(), key=lambda item: item[1], reverse = True))
        
        if micro_sorted_action == None or micro_sorted_action == {}:
            #print("xxxxxx empty sorted action xxxxxx")
            micro_keys_sorted_action, micro_action_space, micro_up_actionspace = [], [], []
            micro_vals_sorted_action, micro_norm_vals_sorted_action =[], []
            micro_sorted_action = None
            
        else: 
            micro_keys_sorted_action = micro_sorted_action.keys()
            micro_action_space = list(map(int,micro_keys_sorted_action))
            #print("micro_action_space",micro_action_space)   
            micro_up_actionspace = micro_action_space 
            micro_vals_sorted_action = micro_sorted_action.values()
            micro_norm_vals_sorted_action = [float(i)/sum(micro_vals_sorted_action) for i in micro_vals_sorted_action]

        for micro_vnf, microvnf_res in decomp[0]: 
            #print("==> micro_vnf", micro_vnf)
            #print("==> new_reward_per_current_states", new_reward_per_current_states)
            #print("reward_mvnfs", reward_mvnfs)
            mvnf_start = time.perf_counter_ns()
            
            if micro_sorted_action is None:
                #print("xxxxxx break xxxxxx")
                break
            else:
                #print("===== micro_vnf_res", microvnf_res, microvnf_res['microVNF_ID'])
                microstate_space_size = microvnf_res['numMicroVNF']
                micro_vnf_res         = microvnf_res['micro_cpu_req']
                #traffic_pred          = microvnf_res['classifier_class']
                          
                #print("micro_vnf", micro_vnf, " micro_vnf_res",micro_vnf_res)
                #print("pr_level: ", pr_level)
                micro_observation     = [episode, sfc, vnf, vnf_res_req, pr_level, micro_vnf, micro_vnf_res, traffic_pred]
                state_dim             = [len(micro_observation)]
                #print("micro state_dim: ", state_dim)
                #print("micro_vnf", micro_vnf)
                status_existing_action = False
                mVNF_ID = microvnf_res['microVNF_ID']
                micro_reward_action_not = 0
                
                if temp_accepted_action:
                    #print("=====> before:temp_accepted_action", temp_accepted_action)
                    existing_action = None
                    for key, val in temp_accepted_action.items():
                        #print("key", key, " val", val)
                        for mvnfid in val:
                            #print("mvnfid", mvnfid)
                            #print("mVNF_ID", mVNF_ID)
                            if mVNF_ID is mvnfid:
                                #print("micro_vnf", micro_vnf)
                                #print("=====> temp_accepted_action", temp_accepted_action)
                                status_existing_action = True
                                existing_action = key
                                #print("mVNFID:", mvnfid, "new action: ", existing_action)
                            else: 
                                pass
                
                action_space = micro_up_actionspace
                vals_sorted_action = list(micro_vals_sorted_action)
                #print("")
                #print("before action agent is active: Mvnf action: ",prev_maction)
                #print("original_actionspace", original_actionspace)
                #print("Inside the mvnf: prob_val", prob_val)
                #print("Inside the mvnf: temp_val", temp_val)
                
                #print("micro_norm_vals_sorted_action", micro_norm_vals_sorted_action)
                #print("vals_sorted_action", vals_sorted_action)
                if status_existing_action is False:
                    action = agent.choose_action(micro_observation, 
                                                 prob_val,
                                                 temp_val, 
                                                 original_actionspace)
                
                    #print("Mvnf action: ",action)
                    prev_maction = action

                   # print("micro_up_actionspace",micro_up_actionspace)
                    if not action in micro_up_actionspace:
                        #print("NOT in list")
                        micro_reward_action_not = -1000
                    else:
                        micro_reward_action_not = 0

                    #print("micro_sorted_action: ",micro_sorted_action)
                    if str(action) in micro_sorted_action.keys():
                        current_res = micro_sorted_action[str(action)]
                        #print("current_res", current_res)
                        updated_res = current_res - micro_vnf_res
                        #print("updated_res", updated_res)

                        if updated_res > 0:
                            updated_res = updated_res
                        else:
                            updated_res = 0

                        micro_sorted_action.update({str(action): updated_res})
                        #print("updated micro_sorted_action", micro_sorted_action)
                    else:
                        current_res = 0
                        #print("current_res", current_res)
                        #print("updated micro_sorted_action", micro_sorted_action)
                        pass

                else: 
                    action = existing_action
                
                mvnf_end = time.perf_counter_ns()
                #print("mvnf_end: ", mvnf_end)
                mvnf_exutime = mvnf_end - mvnf_start
                #print("mvnf_runtime", mvnf_exutime)
                
                #print("Mvnf action: ",action)
                max_val = max(micro_sorted_action.values())
                vnf_cnt += 1

                #print("Curent micro action: ", "Micro vnf: ", micro_vnf, "Micro req res: ",micro_vnf_res,  "Action: ",action, )
                #print("micro_sorted_action", micro_sorted_action,"str(action)", str(action))

                state  = micro_vnf
                #constraints
                micro_subjto = microvnf_constraints(action, micro_vnf_res, amount_local_reward, microvnf_accpeted_action)

                if status_existing_action is False:
                    
                    #Constraint 1: checking for the available resouce for VNF: Resource checking
                    local_rewardmicroVNF = micro_subjto.constraint_1()
                    #print("local_rewardmicroVNF", local_rewardmicroVNF)
                    #print("reward_per_current_vnfstates", reward_per_current_vnfstates)
                    
                else: 
                    t_action = [None, None]
                    local_rewardmicroVNF = amount_local_reward
                    #print("====> action", action)
                    t_action[0] = 'existing'
                    t_action[1] = action
                    #print("================== t_action: ", t_action)
                    microvnf_accpeted_action.append(t_action)
                
                reward_per_current_vnfstates.append(local_rewardmicroVNF)  
                #print("after: microvnf_accpeted_action", microvnf_accpeted_action)
                #print("==== _action", action, "--", "local_rewardmicroVNF", local_rewardmicroVNF)
                
                
                #Modifying the Reward function added on 13th April 2022
                #print("reward_mvnfs", reward_mvnfs)
                #Part 1: Reward based on the quality of Nodes selected
                for node, available_res in env.network_nodes:   
                    dict_node_avail[node] = (available_res['emb_vnf'] / tot_vnf_per_core)    
                reward_Node_based = local_rewardmicroVNF*dict_node_avail[str(action)]
                #print("reward_Node_based", reward_Node_based)
                reward_mvnfs.append(reward_Node_based)
                
                #Part 2: Reward based on the Priority service
                reward_pr    = local_rewardmicroVNF*pr_level
                reward_mvnfs.append(reward_pr)
                
                # Part 3: Reward based on the sfc_rel
                reward_rel = local_rewardmicroVNF*sfc_rel
                reward_vnfs.append(reward_rel)
                
                # Part 4: Reward based on the traffic_pred
                if traffic_load == None:
                    traffic_load = 0
                else:
                    traffic_load = traffic_load

                if local_rewardmicroVNF == 1000:
                    reward_traffic_pred = local_rewardmicroVNF*(traffic_pred*traffic_load)
                    #print("reward_traffic_pred:", reward_traffic_pred, traffic_pred, local_rewardVNF)
                else: 
                    if traffic_pred == -1:
                        reward_traffic_pred = -(local_rewardmicroVNF*(traffic_pred*traffic_load))
                    else:
                        reward_traffic_pred = (local_rewardmicroVNF*(traffic_pred*traffic_load))
                #print("reward_traffic_pred:", reward_traffic_pred, traffic_pred, local_rewardVNF)
                reward_vnfs.append(reward_traffic_pred)
                
                #Part 4: Reward based on scheduling delay
                mvf_t_current = mvnf_exutime #rand.randint(1, 50)  #vnf_runtime #
                #print("vnf current time", t_current)
               
                
                #updating the t max
                if mvf_t_max is None or mvf_t_current >= mvf_t_max: mvf_t_max = mvf_t_current
                
                #updating the t min
                if (mvf_t_current < mvf_t_max and mvf_t_min == 0) or (mvf_t_current <= mvf_t_min): mvf_t_min = mvf_t_current
                elif (mvf_t_current >= mvf_old_t_current and mvf_t_min == 0):mvf_t_min = mvf_old_t_current
                norm_time = 1 - Normalization(mvf_t_current, mvf_t_max, mvf_t_min)
                #print("t_current: ", mvf_t_current,"-- t_max: ", mvf_t_max, "-- t_min: ",mvf_t_min, "-- norm_time: ", norm_time)
                time_reward = local_rewardmicroVNF*norm_time
                reward_mvnfs.append(time_reward)
                mvf_old_t_current = mvf_t_current
                #print("reward_mvnfs", reward_mvnfs)
                
                # Total Rewards:
                total_reward_mvnf = round(sum(reward_mvnfs), 4)
                #print("total_reward_mvnf: ", total_reward_mvnf)
                #new_reward_per_current_states.append(total_reward_vnf)
                #print("new_reward_per_current_states: ", new_reward_per_current_states)
                #print("vnf", vnf, "-->", new_reward_per_current_states[vnf], "reward: ", total_reward_mvnf)
                new_reward_per_current_states[vnf] = total_reward_mvnf
                #print("vnf", vnf, "-->", new_reward_per_current_states[vnf], "reward: ", total_reward_mvnf)

                # 07032022
                if local_rewardmicroVNF == amount_local_reward:
                    pref_node_val = 10 #amount_local_reward #amount_local_reward #1
                else:
                    pref_node_val = 0
                #print("ACTION: ", action)
                #print("prev_maction", prev_maction)
                
                temp_val[action] = temp_val[action] + pref_node_val
                prob_val = [float(i)/sum(temp_val) for i in temp_val]
                #print("===> Inside the mvnf: prob_val", prob_val)
                #print("===> Inside the mvnf: temp_val", temp_val)
                #print("---")

                #Constraint 2 and 3: : checking for the available resouce for Links and estab. the path
                micro_subjto.constraint_2(microstate_space_size, decomp[1])
                rewardlink_per_current_vnfstates = micro_subjto.local_reward_delay
                microlink_space_size = len(rewardlink_per_current_vnfstates)
                #print("rewardlink_per_current_vnfstates", rewardlink_per_current_vnfstates)
                #print("updated_path_perVNF", updated_path_perVNF)

                # Reward Function
                global_reward =  0              
                ## Global reward for VNF placement
                #print("microstate_space_size", microstate_space_size)
                #print("reward_per_current_vnfstates", reward_per_current_vnfstates)
                #print("microlink_space_size", microlink_space_size)
                #print("rewardlink_per_current_vnfstates", rewardlink_per_current_vnfstates)

                if (sum(reward_per_current_vnfstates) == amount_local_reward*microstate_space_size) and \
                    (sum(rewardlink_per_current_vnfstates) == amount_local_reward*microlink_space_size):
                    
                    #print("==> REWARD <==")
                    #done = True
                    reward_all = 1000*10
                    rewars_delay = sum(rewardDelay_per_current_states)
                    global_reward = reward_all + rewars_delay
                    accpeted_action.append(microvnf_accpeted_action)
                    score = 1
                    #print("inside mvnf REWARD: accpeted_action", accpeted_action)
                    #embedding edit this one: change the flow instead of embedding checking should be done here and embedding will be 
                    #performed during the main vnf coding side

                    microvnf_node_resources = decomp[0]
                    #env.embedding_microvnf(microvnf_accpeted_action,
                    #                       microvnf_node_resources, 
                    #                       count_microvnf)
                    #count_microvnf = env.count_microvnf
                    #print("count_microvnf", count_microvnf)
                    count_microvnf = microstate_space_size
                    #print("count_microvnf",count_microvnf)
                    
                    
                    microvnf_link_resources = decomp[1]
                    #env.embedding_microvl(updated_path_perVNF, 
                    #                      microvnf_link_resources, 
                    #                      count_updated_path_pervnf)
                    #count_updated_path_pervnf = env.count_updated_path_pervnf
                    #print("count_updated_path_pervnf", count_updated_path_pervnf)
                    count_updated_path_pervnf = microlink_space_size
                    #print("count_updated_path_pervnf",count_updated_path_pervnf)

                micro_local_reward = local_rewardmicroVNF #immediate rewards
                micro_rewards = total_reward_mvnf + global_reward + micro_reward_action_not 
                #print("MICRO REWARDS", micro_rewards)

                #Next state
                next_microstate = micro_vnf+1

                if next_microstate >= microstate_space_size:
                    next_microstate = 0
                    next_observation = [episode, sfc, vnf+1, vnf_res_req, pr_level, \
                                        next_microstate, micro_vnf_res, traffic_pred]
                else: 
                    next_microstate = next_microstate
                    next_observation = [episode, sfc, vnf, vnf_res_req, pr_level, \
                                        next_microstate, micro_vnf_res, traffic_pred]

                state = micro_observation
                next_state = next_observation

                # storing the transistions
                #print("state", state)
                agent.store_transition(state, action, micro_rewards, next_state, done, vnf_cnt)

                #learning:  providing state, action, reward and next state to the NN for learning.
                agent.learn()
                epsilon = agent.epsilon

                #print("count vnf",count_VNF)

                if microstate_space_size == count_microvnf and microlink_space_size == count_updated_path_pervnf:
                    #print("======>>Micro vnf embedded")
                    #print("count vnf",count_VNF)
                    #print("reward_per_current_states 1st", reward_per_current_states)
                    done_microvnf = True 
                    reward_microvnf = 1000
                    local_rewardVNF = 1000
                    #count_VNF =len(reward_per_current_states)
                    reward_per_current_states.append(reward_microvnf)   
                    #print("reward_per_current_states", reward_per_current_states)
                    count_VNF += 1
                    #print("count_VNF", count_VNF)

                    #if the last vnf is decomposed then:
                    check_pt = microvnf_node_resources[0][1]['decompose_candidate']
                    #print("check point", check_pt)
                    #print("count_VNF", count_VNF)
                    if state_space_size == count_VNF and  check_pt !='Penalty Decompose':
                        #print("======>>>> Last vnf")
                        done = True
                        #print("decomp", microvnf_node_resources[0][1]['decompose_candidate'])

                        subjected_to = vnf_constraints(amount_local_reward, vnf_res_req,
                                                       nodal_avail, max_vnf_resource, 
                                                       episode, sfc, vnf, vnf_per_service,
                                                       sum_resouces, accpeted_action)

                        vnf_link_resources = vnf_link_resources
                        env_networkgraph   = env.network_graph
                        env_networkedges   = env.network_edges
                        subjected_to.constraint_3(state_space_size, vnf_link_resources,
                                                 env_networkgraph, env_networkedges,
                                                 threshold_delay)
                        rewardlink_per_current_states = subjected_to.local_reward_link

        #print(" inside mvnf: accpeted_action", accpeted_action)
        return count_VNF,reward_per_current_states, rewardlink_per_current_states, accpeted_action, updated_path_perVNF, vnf_cnt, microvnf_accpeted_action, mvf_t_max, mvf_t_min, mvf_old_t_current, new_reward_per_current_states


In [26]:
#decomp  = [[(0, {'episode': 0, 'sfc': 0, 'vnf': 1, 'vnf_req': 8, 'numMicroVNF': 3, 'microVNF': 0, 'microVNF_ID': 56, 'micro_cpu_req': 6, 'microVNF_pos': 'head', 'decompose_candidate': 'VNF Decompose'}), 
#(1, {'episode': 0, 'sfc': 0, 'vnf': 1, 'vnf_req': 8, 'numMicroVNF': 3, 'microVNF': 1, 'microVNF_ID': 4, 'micro_cpu_req': 1, 'microVNF_pos': 'middle', 'decompose_candidate': 'VNF Decompose'}), (2, {'episode': 0, 'sfc': 0, 'vnf': 1, 'vnf_req': 8, 'numMicroVNF': 3, 'microVNF': 2, 'microVNF_ID': 5, 'micro_cpu_req': 1, 'microVNF_pos': 'tail', 'decompose_candidate': 'VNF Decompose'})], 
#[(0, 1, {'src_vnf': 0, 'dest_vnf': 1, 'episode': 0, 'sfc': 0, 'numVNF': 3, 'microlink_num': 0, 'bw': 6, 'latency': 1}), 
#(1, 2, {'src_vnf': 1, 'dest_vnf': 2, 'episode': 0, 'sfc': 0, 'numVNF': 3, 'microlink_num': 1, 'bw': 7, 'latency': 1})]]

In [27]:
#microvnf_const = microvnf_construction(max_vnf_resource = 10, episode = 0, sfc = 0, vnf = 1, cpu_req = 8, n = 3, val=0)
#microvnf_const.microvnf_agent(episode =0, sfc=0, vnf=1, decomp =decomp, num_micro_vnf = 3)

In [28]:
class decomp_identifier():
    
    def __init__(self,cpu_req, nodal_avail, max_vnf_resource, episode, sfc, vnf, vnf_per_service):
        
        
        self.cpu_req = cpu_req
        self.nodal_avail = nodal_avail
        #self.micro_vnf_res =list(microVNFFG.micro_resource_init(self.cpu_req, self.nodal_avail)) 
        #self.num_micro_vnf = len(self.micro_vnf_res) #cal the nun of micro vnfs
        self.max_vnf_resource = max_vnf_resource
        self.episode = episode
        self.sfc = sfc
        self.vnf = vnf
        self.vnf_per_service = vnf_per_service

        
    def microvnf_res_init(self, cpu_req, nodal_avail, granular_index):
        self.microvnf_const = microvnf_construction(max_vnf_resource = self.max_vnf_resource, 
                                                    episode = self.episode, 
                                                    sfc = self.sfc, 
                                                    vnf = self.vnf, 
                                                    cpu_req = self.cpu_req, 
                                                    n = self.vnf_per_service,
                                                    val = 100, 
                                                    pr_level = pr_level,
                                                    sfc_rel = sfc_rel, 
                                                    traffic_pred = traffic_pred)
        #print("self.microvnf_const: ",self.microvnf_const)
        #print("resource identifier: microvnf_res_init")
        self.micro_vnf_res =list(microVNFFG.micro_resource_init(self.cpu_req, self.nodal_avail, self.granular_index)) 
        #print("self.micro_vnf_res", self.micro_vnf_res)
        self.num_micro_vnf = len(self.micro_vnf_res) #cal the nun of micro vnfs
        #print("self.num_micro_vnf", self.num_micro_vnf)
        
        if self.num_micro_vnf == 0 or self.num_micro_vnf == []:
            self.num_micro_vnf = []
            self.microvnf_status  = False
        else:
            self.num_micro_vnf = self.num_micro_vnf
            self.microvnf_status  = True
        
        return self.num_micro_vnf, self.micro_vnf_res, self.microvnf_status
    
    
    def potential_vnf(self,threshold_decompose, done_potential_vnf):
        #print("VNF: ", self.vnf)
        self.done_potential_vnf = done_potential_vnf
        self.granular_index = round(self.cpu_req*self.nodal_avail)
        self.microvnf_info = self.microvnf_res_init(self.cpu_req, self.nodal_avail, self.granular_index)
        
        
        if self.num_micro_vnf:# and not self.num_micro_vnf:
            #print("VNF Decomposition")
            
            self.decompose_candidate = "VNF Decompose"
            #print("SFC: ", self.sfc, "VNF: ", self.vnf, "self.decompose_candidate: ", self.decompose_candidate)
            self.done_potential_vnf = True
            self.decomp = self.microvnf_const.microvnf(self.microvnf_info[0], 
                                                       self.cpu_req,
                                                       self.microvnf_info[1], 
                                                       self.decompose_candidate) #construct the micro-VNF-FG  
            
            #print("self.decomp", self.decomp)
            self.unique_id = self.decomp[2]
            
        else:
            #print("in potential else")
            self.decomp = None
            self.decompose_candidate = None
            self.num_micro_vnf = None
            self.micro_vnf_res = None
            self.unique_id = None
            
        return self.num_micro_vnf, self.micro_vnf_res, self.decompose_candidate, self.unique_id
    
    def nodal_availability(self,done_nodal):
        self.done_nodal = done_nodal
        self.granular_index = self.cpu_req*self.nodal_avail
        self.microvnf_info = self.microvnf_res_init(self.cpu_req, self.nodal_avail, self.granular_index)
        #print("microvnf_info", self.microvnf_info)
        
        if self.num_micro_vnf and self.microvnf_info[2] == True:
            self.done_nodal = True
            self.decompose_candidate = "Nodal Decompose"
            #print("SFC: ", self.sfc, "VNF: ", self.vnf, "self.decompose_candidate: ", self.decompose_candidate)
            #print("num_micro_vnf", self.num_micro_vnf)
            self.decomp = self.microvnf_const.microvnf(self.microvnf_info[0], 
                                                       self.cpu_req,
                                                       self.microvnf_info[1], 
                                                       self.decompose_candidate)
            self.unique_id = self.decomp[2]
            #print("==> self.done_nodal",  self.done_nodal)
        else:
            #print("in nodal else")
            self.decomp = None
            self.decompose_candidate = None
            self.num_micro_vnf = None
            self.micro_vnf_res = None
            self.unique_id = None
            #print(self.decomp, self.decompose_candidate)
            
        return self.num_micro_vnf, self.micro_vnf_res, self.decompose_candidate, self.unique_id
    
    
    def unsuccessful_vnf(self,sum_resouces, granular_index, done_unsuccessful_vnf):
        self.granular_index = granular_index
        self.done_unsuccessful_vnf = done_unsuccessful_vnf
        self.microvnf_info = self.microvnf_res_init(self.cpu_req, self.nodal_avail, self.granular_index)
        #print("self.microvnf_info ", self.microvnf_info )
        #print("self.num_micro_vnf", self.num_micro_vnf)
        #self.num_micro_vnf = []
        
        if self.num_micro_vnf and sum_resouces >= self.cpu_req: 
            self.done_unsuccessful_vnf = True
            #print("self.done_unsuccessful_vnf", self.done_unsuccessful_vnf)
            self.decompose_candidate = "Penalty Decompose"
            #print("SFC: ", self.sfc, "VNF: ", self.vnf, "self.decompose_candidate: ", self.decompose_candidate)
            self.decomp = self.microvnf_const.microvnf(self.microvnf_info[0], 
                                                       self.cpu_req, 
                                                       self.microvnf_info[1], 
                                                       self.decompose_candidate) 
            self.unique_id = self.decomp[2]
            #print("decomp", self.decomp)
            
        else:
            #print("in penalty else")
            self.decomp = None
            self.decompose_candidate = None
            self.num_micro_vnf = None
            self.micro_vnf_res = None
            self.unique_id = None
            
        return self.num_micro_vnf, self.micro_vnf_res, self.decompose_candidate, self.unique_id, self.done_unsuccessful_vnf

In [29]:
#episode = 1
#sfc = 3

#vnf = 2
#decompose_candidate = (2, [1, 1], 'Nodal Decompose', [2, 1]) 
#microvnf_done_status = True
#qq= Trail(num_episode = 2, num_sfc =5, vNetwork_service_er =[] , vNetwork_service_details_er =[], max_vnf_resource =10,
#                 vnf_per_service =3, vnf_node_resources =[], vnf_link_resources =[])
#qq.update_vnf(episode, sfc, vnf, decompose_candidate, microvnf_done_status)

In [30]:
class substrate_network():
    def __init__(self):
        
        self.topology_name = 'Netrail'#'BtEurope'
        self.network_graph = nx.read_graphml('../Topology_Dataset/Netrail.graphml')
        self.fig = plt.figure() 
        #print (nx.info(self.network_graph))
        #nx.draw(self.network_graph,with_labels=True)
        #plt.show()
        #fig.savefig('Generated_Plots/core/for_100ms/BtEurope.png')
        
        self.network_len_nodes = len(self.network_graph.nodes)
        self.range_nodes = str(self.network_graph.nodes)
        self.tot_links= self.network_graph.edges()
        self.len_tot_links = len(self.tot_links)
        
        
    def substrate_node_init(self, removed_nodes, tot_vnf_per_core): 
        #print("initialization of network resouce")
        for node in self.network_graph.nodes():
            if int(node) in removed_nodes:              
                self.network_graph.add_node(node, emb_vnf = 0) #(no of core * no of VNFper core)
            else:
                self.network_graph.add_node(node, emb_vnf = tot_vnf_per_core) #(no of core * no of VNFper core)
        #print(list(self.network_graph.nodes(data=True)))
        self.network_nodes = list(self.network_graph.nodes(data=True))
        return self.network_nodes
    
    
    def substrate_link_init(self):
        count = 0
        for link in self.network_graph.edges():
            self.network_graph.add_edge(link[0],link[1],
                                        id = count,
                                        bw = int(rand.uniform(1000, 10000)), # random.choice(link_capacity),
                                        weight = 1,
                                        initial = int(link[0]),
                                        final = int(link[1]),
                                        emb = None,
                                        latency = round(rand.uniform(0,10), 2)#ms
                                       )
            count +=1
        #print(list(self.network_graph.edges(data = True)))
        self.network_edges = list(self.network_graph.edges(data=True))
        return self.network_edges
    
    
    def embedding_vnf(self, actions_per_iteration, vnf_node_resources, count_vnf, global_decomp, emb_mVNFID, 
                      switch_reconst):
        self.actions_per_iteration = actions_per_iteration
        self.vnf_node_resources = vnf_node_resources
        self.count_vnf = count_vnf
        self.global_decomp = global_decomp
        #print("self.actions_per_iteration", self.actions_per_iteration)
        #print("inside emb")
        
        if not self.actions_per_iteration:
            pass 
        else: 
            self.count_vnf = 0
            count_action = 0
            #print("emb")
            #print("self.count_vnf", self.count_vnf)
            for key, val in self.vnf_node_resources:
                #count_action = self.count_vnf
                #print("self.actions_per_iteration",self.actions_per_iteration)
                #print("count_action:",count_action)
                if count_action >= len(self.actions_per_iteration):
                    break
                else:
                    count_decomp = 0
                    for action in self.actions_per_iteration:
                        emb_vnf = False
                        #print("action", action)
                        #print("self.count_vnf", self.count_vnf)
                        #print("count_action", count_action)
                        
                        if type(action) is list:
                            #print(action)
                            microvnf_accpeted_action = action
                            microvnf_node_resources =  self.global_decomp[count_decomp][0]
                            count_microvnf = 0
                            count_microvnf = self.embedding_microvnf(microvnf_accpeted_action,
                                                                     microvnf_node_resources, 
                                                                     count_microvnf, 
                                                                     emb_mVNFID, 
                                                                     switch_reconst)
                            
                            count_action +=1 
                            #print("count_microvnf:", count_microvnf[0])
                            #print("count_emb_mvnf", count_microvnf[1])
                            count_decomp += 1
                            
                            if len(action) == count_microvnf[1]:
                                emb_vnf = True

                        else:
                            for node, available_res in self.network_nodes:
                                if available_res['id'] == self.actions_per_iteration[self.count_vnf]:
                                    count_action +=1 
                                    #print("count_action:",count_action)
                                    CPU_old = available_res['emb_vnf']
                                    #print("CPU_old: ", CPU_old)
                                    
                                    requested_vnf_resource = val['cpu_req']
                                    #print("requested_vnf_resource:", requested_vnf_resource)
                                    
                                    CPU_remian = CPU_old - requested_vnf_resource
                                    #print("CPU_remian: ", CPU_remian)
                                    
                                    if CPU_old >= requested_vnf_resource:
                                        emb_vnf = True
                                        #print("here")
                                        #print("emf vnf: ", self.actions_per_iteration[self.count_vnf])
                                        #print('CPU_old:', CPU_old,' requested_vnf_resource:', requested_vnf_resource,' CPU_remian', CPU_remian)
                                        if CPU_remian <= 0:
                                            #embedding
                                            self.network_graph.add_node(node, emb_vnf = 0)
                                        else: 
                                            self.network_graph.add_node(node, emb_vnf = CPU_remian)
                                        

                                    else:
                                        #print("in else when no avail res")
                                        break
                                else: 
                                    pass
                        
                        if emb_vnf is True:
                            self.count_vnf += 1
                        if count_action >= len(self.actions_per_iteration):
                            break
            #print("self.count_vnf", self.count_vnf)
            #print("count_action", count_action)
            return self.count_vnf

        
    def embedding_vl(self, updated_path_perSFC, vnf_link_resources, count_updated_path_perSFC, global_decomp, 
                     global_updated_path_perVNF):
        self.count_updated_path_perSFC = count_updated_path_perSFC
        self.global_decomp = global_decomp
        self.global_updated_path_perVNF = global_updated_path_perVNF

        
        if self.global_decomp:
            count_decomp = 0
            for key in self.global_updated_path_perVNF:
                updated_path_perVNF = key
                microvnf_link_resources = self.global_decomp[count_decomp][1]
                count_updated_path_pervnf = 0
                count_updated_path_pervnf = self.embedding_microvl(updated_path_perVNF,
                                                                   microvnf_link_resources,
                                                                   count_updated_path_pervnf)
                count_decomp += 1

        len_updated_path = len(updated_path_perSFC)
        if not updated_path_perSFC:
            self.count_updated_path_perSFC = 0
            pass

        elif len_updated_path != link_space_size:
            self.count_updated_path_perSFC = 0
            pass

        else:
            count_sfc_link = 0
            self.count_updated_path_perSFC = 0
            len_updated_path = len(updated_path_perSFC)

            if self.count_updated_path_perSFC == len_updated_path:
                pass

            else:
                for requested_bw in vnf_link_resources:
                    #print("Link",count_sfc_link, requested_bw)
                    link_perVNF = updated_path_perSFC[self.count_updated_path_perSFC]
                    #print("link_perVNF: ",link_perVNF, " count_updated_path_perSFC: ",self.count_updated_path_perSFC)
                    source_node =[]
                    target_node =[]
                    count_dis = 0

                    if not link_perVNF:
                        pass
                    else:
                        len_dis = len(link_perVNF)

                    for key in link_perVNF:
                        if (count_dis == len_dis-1):
                            #break
                            pass
                        else: 
                            source_node = link_perVNF[count_dis]
                            target_node = link_perVNF[count_dis+1]

                            #Embbeding coding 
                            for available_bw in self.network_edges:                        
                            #Checking for the direct links 
                                if (available_bw[2]['initial'] == int(source_node) and available_bw[2]['final'] == int(target_node)) or (available_bw[2]['initial'] == int(target_node) and available_bw[2]['final'] == int(source_node)): 

                                    sfc_avail_bw = available_bw[2]['bw']
                                    vnf_req_bw = requested_bw[2]['bw']
                                    direct_remain_bw = available_bw[2]['bw'] - requested_bw[2]['bw']

                                    if sfc_avail_bw >= vnf_req_bw:
                                        #print("emb link", link_perVNF)
                                        self.network_graph.add_edge(available_bw[0],available_bw[1],
                                                               bw = direct_remain_bw,
                                                               emb= 1)
                                else:
                                    #print("not emb link")
                                    pass
                        count_dis +=1
                    self.count_updated_path_perSFC +=1  
                    count_sfc_link +=1
        
        return self.count_updated_path_perSFC

    def embedding_microvnf(self, microvnf_accpeted_action, microvnf_node_resources, count_microvnf, emb_mVNFID, switch_reconst):
        self.microvnf_accpeted_action = microvnf_accpeted_action
        self.microvnf_node_resources = microvnf_node_resources
        self.count_microvnf = count_microvnf
        self.emb_mVNFID = emb_mVNFID
        #print("emb_mVNFID", emb_mVNFID)
        #print("inside emb")
        if not self.microvnf_accpeted_action:
            pass 
        else: 
            #print("self.microvnf_accpeted_action", self.microvnf_accpeted_action)
            self.count_microvnf = 0
            self.count_emb_mvnf = 0
            #print("emb", self.count_microvnf)
            for key, val in self.microvnf_node_resources:
                count_action = self.count_microvnf
                for action in self.microvnf_accpeted_action:
                    microemb_vnf = False
                    if self.count_microvnf == count_action:
                        for node, available_res in self.network_nodes:
                            #print("self.count_microvnf", self.count_microvnf)
                            if available_res['id'] == microvnf_accpeted_action[self.count_microvnf]:
                                CPU_old = available_res['emb_vnf']
                                requested_vnf_resource = val['micro_cpu_req']
                                #print("Avail Resource", CPU_old, "Reqested Resource",requested_vnf_resource)
                                #print("emb microvnf",microvnf_accpeted_action[self.count_microvnf],"microVNF_ID: ",val['microVNF_ID'])
                                
                                if switch_reconst is True:
                                    #print("switch_reconst", switch_reconst)
                                    #print(self.emb_mVNFID)
                                    if val['microVNF_ID'] in self.emb_mVNFID:
                                        CPU_remian = CPU_old
                                        self.count_emb_mvnf +=1
                                        #print(" ==> alreay present: val['microVNF_ID']", val['microVNF_ID'])
                                    else:  
                                        if CPU_old >= requested_vnf_resource:
                                            CPU_remian = CPU_old - requested_vnf_resource
                                            self.emb_mVNFID.append(val['microVNF_ID']) 
                                            self.count_emb_mvnf +=1
                                            #print("val['microVNF_ID']", val['microVNF_ID'])
                                        else:
                                            #print("breAK")
                                            break
                                else:

                                    CPU_remian = CPU_old - requested_vnf_resource
                                #print("==> count_emb_mvnf", self.count_emb_mvnf)
                                #print("CPU remain: ", CPU_remian)
                                if CPU_remian <= 0:
                                    #embedding
                                    self.network_graph.add_node(node, emb_vnf = 0)
                                else: 
                                    self.network_graph.add_node(node, emb_vnf = CPU_remian)
                                count_action +=1 
                                #print(count_action)

                            else:
                                #print("not emb microvnf")
                                pass
                    else:
                        pass

                self.count_microvnf +=1
            #print("inside count_action:",count_action)
            #print("inside self.count_microvnf", self.count_microvnf)
            return self.count_microvnf, self.count_emb_mvnf
        

    def embedding_microvl(self, updated_path_perVNF, microvnf_link_resources, count_updated_path_pervnf):
        self.count_updated_path_pervnf = count_updated_path_pervnf
        #print("count_updated_path_pervnf: ", count_updated_path_pervnf)
        #print("updated_path_perVNF", updated_path_perVNF)
        #print("microvnf_link_resources", microvnf_link_resources)
        
        
        len_updated_path = len(updated_path_perVNF)
        #print("len_updated_path", len_updated_path)
        
        if not updated_path_perVNF:
            self.count_updated_path_pervnf = 0
            pass
        elif len_updated_path != len(microvnf_link_resources):
            self.count_updated_path_pervnf = 0
            pass
        else:
            count_sfc_link = 0
            self.count_updated_path_pervnf = 0
            len_updated_path = len(updated_path_perVNF)

            if self.count_updated_path_pervnf == len_updated_path:
                #print("break")
                pass
                
            else:
                for requested_bw in microvnf_link_resources:
                    #print("Link",count_sfc_link, requested_bw)
                    link_perVNF = updated_path_perVNF[self.count_updated_path_pervnf]
                    #print("link_perVNF",link_perVNF, self.count_updated_path_pervnf)
                    source_node =[]
                    target_node =[]
                    count_dis = 0

                    if not link_perVNF:
                        pass
                    else:
                        len_dis = len(link_perVNF)
                        #print("len_dis", len_dis)

                    for key in link_perVNF:
                        if (count_dis == len_dis-1):
                            #break
                            pass
                        else: 
                            source_node = link_perVNF[count_dis]
                            target_node = link_perVNF[count_dis+1]
                            
                            #print("source_node", source_node)
                            #print("target_node", target_node)
                            
                            #Embbeding coding 
                            for available_bw in self.network_edges:                        
                            #Checking for the direct links 
                                if (available_bw[2]['initial'] == int(source_node) and available_bw[2]['final'] == int(target_node)) or (available_bw[2]['initial'] == int(target_node) and available_bw[2]['final'] == int(source_node)): 
                                    sfc_avail_bw = available_bw[2]['bw']
                                    vnf_req_bw = requested_bw[2]['bw']
                                    direct_remain_bw = available_bw[2]['bw'] - requested_bw[2]['bw']
                                    #print("==> micro link emb")
                                    if sfc_avail_bw >= vnf_req_bw:
                                        self.network_graph.add_edge(available_bw[0],available_bw[1],
                                                               bw = direct_remain_bw,
                                                               emb= 1)
                                else:
                                    #print("not emb link")
                                    pass
                        count_dis +=1
                        #print("count_dis", count_dis)
                    self.count_updated_path_pervnf +=1               
                    count_sfc_link +=1
        #print("count_updated_path_pervnf: ", self.count_updated_path_pervnf)
                    
        return self.count_updated_path_pervnf


In [31]:
class vnf_constraints():
    def __init__ (self, amount_local_reward, vnf_res_req, nodal_avail, max_vnf_resource, episode, sfc, vnf, 
                  vnf_per_service, sum_resouces, accpeted_action, switch_decomp):
        
        self.amount_local_reward = amount_local_reward
        self.vnf_res_req = vnf_res_req
        self.nodal_avail = nodal_avail
        self.max_vnf_resource = max_vnf_resource
        self.episode = episode
        self.sfc = sfc
        self.vnf = vnf
        self.vnf_per_service = vnf_per_service
        self.sum_resouces = sum_resouces
        self.accpeted_action = accpeted_action
        self.decomp = []
        #
        
    def constraint_1(self, action):
        self.action = action
        #
        decompose_candidate = []
        #print("constraint_1")
        for key, available_res in env.network_nodes:
            #print("key",key)
            if available_res['id'] == self.action:
                #print("self.vnf_res_req", self.vnf_res_req)
                if available_res['emb_vnf'] >= self.vnf_res_req: 
                    #print("available_res['emb_vnf']", available_res['emb_vnf'])
                    local_rewardVNF = self.amount_local_reward
                    #print("Reward", local_rewardVNF)
                    self.accpeted_action.append(self.action)
                    #print("const 1: self.accpeted_action", self.accpeted_action)
                else:
                    if switch_decomp is False: 
                        self.decomp = None
                        local_rewardVNF = -(self.amount_local_reward)
                        #print("insufficent resouce in vnf", local_rewardVNF)
                    else:
                        #print("")
                        #print("decomposition inside constraint_1")
                        #print("action", self.action, available_res['emb_vnf'])
                        #print("insufficent resouce in vnf")
                        topology_available_resource = available_res['emb_vnf']
                        local_rewardVNF = -(self.amount_local_reward)
                        done_unsuccessful_vnf = False
                        granular_index = self.vnf_res_req * self.nodal_avail

                        if granular_index <= 1:
                            granular_index = 1
                        else: 
                            granular_index = granular_index

                        #print("granular_index", granular_index)

                        decompose_criteria = decomp_identifier(self.vnf_res_req, self.nodal_avail,self.max_vnf_resource, 
                                                             self.episode, self.sfc, self.vnf, self.vnf_per_service) 

                        decompose_criteria.unsuccessful_vnf(self.sum_resouces, granular_index, done_unsuccessful_vnf) 
                        decompose_candidate = [decompose_criteria.num_micro_vnf, 
                                               decompose_criteria.micro_vnf_res, 
                                               decompose_criteria.decompose_candidate
                                              ]
                        #print(decompose_candidate)
                        if decompose_criteria.decompose_candidate is None:
                            pass #print(done_unsuccessful_vnf)
                        else: 
                            done_unsuccessful_vnf = True
                            #print(done_unsuccessful_vnf)
                        #print('done_unsuccessful_vnf', done_unsuccessful_vnf)
                        #print(decompose_candidate.done_unsuccessful_vnf)
                        global_services.update_vnf(self.episode, 
                                                   self.sfc, 
                                                   self.vnf, 
                                                   decompose_candidate, 
                                                   done_unsuccessful_vnf)

                        self.decomp = decompose_criteria.decomp
                    
                    #print("vnf_const.er_graph_seed.add_node", vnf_const.er_graph_seed.add_node)
                    #print("local_rewardVNF: ",local_rewardVNF)
        #print('done_unsuccessful_vnf: ', done_unsuccessful_vnf)
        return local_rewardVNF
    
    def constraint_2(self,up_actionspace, sorted_action):
        #print("constraint_2")
        self.up_actionspace = up_actionspace
        self.sorted_action = sorted_action
        #print("self.up_actionspace", self.up_actionspace)
        #print("self.sorted_action", self.sorted_action)
        #print("self.accpeted_action", self.accpeted_action)
        
        for a_key in self.accpeted_action:
            if a_key in self.up_actionspace:
                self.up_actionspace.remove(a_key)
                del self.sorted_action[str(a_key)]
                #print("a_key", a_key)
        return #self.up_actionspace, self.sorted_action 
    

    def constraint_3(self, state_space_size, vnf_link_resources, env_networkgraph, env_networkedges, 
                     threshold_delay, updated_path_perSFC):
        #print("constraint_3")
        self.state_space_size   = state_space_size
        self.vnf_link_resources = vnf_link_resources
        self.env_networkgraph   = env_networkgraph    
        self.env_networkedges   = env_networkedges
        self.threshold_delay    = threshold_delay
        self.local_reward_link  = []
        self.local_reward_delay = 0
        self.decomp_nodes       = []
        self.updated_path_perSFC = updated_path_perSFC

        if len(self.accpeted_action) == self.state_space_size:
            
            count_sfc_link = 1
            for requested_bw in self.vnf_link_resources:
                #print("link", count_sfc_link, requested_bw)
                
                #Finding the Path
                Start_node = self.accpeted_action[requested_bw[0]]
                End_node   = self.accpeted_action[requested_bw[1]]
                
                #print("==")
                if type(Start_node) == list:
                    #print("Start Node", Start_node)
                    t_cnt = 0
                    for t_key in Start_node:
                        if type(t_key) is list:
                            #print("t_key",t_key)
                            Start_node[t_cnt] = t_key[1]
                            #print("Start_node[t_cnt]", Start_node[t_cnt])
                        t_cnt += 1 
                    self.decomp_nodes = Start_node
                    len_startnode = len(Start_node)
                    Start_node = self.decomp_nodes[len_startnode-1]
                #print("Start Node", Start_node)
                
                if type(End_node) == list:
                    #print("End_node", End_node)
                    t_cnt = 0
                    for t_key in End_node:
                        if type(t_key) is list:
                            End_node[t_cnt] = t_key[1]
                        t_cnt += 1
                    self.decomp_nodes = End_node
                    End_node = self.decomp_nodes[0]
                #print("End_node", End_node)

                    
                #print("inside const 3: accpeted_action", self.accpeted_action, count_sfc_link)    
                if Start_node == End_node:
                    self.local_reward_delay = 1000
                    intralink = ['intra link']
                    self.updated_path_perSFC.append(intralink)
                    count_sfc_link +=1
                else:
                    
                    #print(Start_node,End_node)
                    #print("..")
                    path =  nx.all_simple_paths(self.env_networkgraph,str(Start_node),str(End_node))
                    count_path = 0
                    
                    #self.local_reward_link  = [] 
                    tot_score, delay_per_link =[], []
                    len_delay_per_link,len_each_path =[], []
                    score = 0

                    for each_path in path:
                        #print("each_path", each_path)
                        delay_per_link =[]
                        updated_path_perVNF = each_path

                        if score == 0:
                            len_dis = len(each_path)
                            #print("len_dis",len_dis)
                            count_dis = 0
                            source_node, target_node =[], []

                            for key in each_path:
                                #print("key in each path", key)
                                if count_dis == len_dis-1:
                                    #print("break because it has reached the end of the path")
                                    break
                                else:
                                    source_node = each_path[count_dis]
                                    target_node = each_path[count_dis+1]
                                    #print("source and target: ", source_node, target_node)

                                for available_bw in env_networkedges:                        
                                    if (available_bw[2]['initial'] == int(source_node) and available_bw[2]['final'] == int(target_node)) or (available_bw[2]['initial'] == int(target_node) and available_bw[2]['final'] == int(source_node)): 
                                        #Checking for the available resources
                                        sfc_avail_bw    = available_bw[2]['bw']
                                        vnf_req_bw      = requested_bw[2]['bw']
                                        sfc_avail_delay = available_bw[2]['latency']
                                        vnf_req_delay   = requested_bw[2]['latency']

                                        if (sfc_avail_bw >= vnf_req_bw):# and (sfc_avail_delay >= vnf_req_delay):
                                            local_reward_link = self.amount_local_reward
                                            score = 1
                                            delay_per_link.append(sfc_avail_delay) 
                                            #print(" link Reward ", source_node,target_node) 
                                        else: 
                                            local_reward_link = -10000
                                            pen_score = 2
                                            score = pen_score
                                            #print("link penalty", source_node,target_node) 
                                            pass

                                count_dis += 1

                            tot_delay_perPath  = sum(delay_per_link)
                            len_delay_per_link = len(delay_per_link)
                            len_each_path      = len(each_path)-1
                            #print("count_sfc_link",count_sfc_link)
                            #print("len_delay_per_link", len_delay_per_link, delay_per_link)
                            #print("len_each_path", len_each_path, each_path)
                            #print("self.threshold_delay", self.threshold_delay)
                            #print("tot_delay_perPath", tot_delay_perPath)


                            if len_delay_per_link == len_each_path: #end of the path
                                
                                if tot_delay_perPath <= self.threshold_delay:
                                    self.updated_path_perSFC.append(each_path)
                                    self.local_reward_delay = self.amount_local_reward
                                    #print("==> Reward in latency", each_path)
                                    #print(" link Reward ",self.local_reward_delay)
                                else: 
                                    score = 0
                                    #print("link not found vnf")
                                    self.local_reward_delay = -10000
                            else:
                                score = 0
                                #print("link not found vnf", each_path, tot_delay_perPath)
                                self.local_reward_delay = -10000

                        else:
                            pass
                            #break

                        count_path +=1
                    count_sfc_link += 1 
                self.local_reward_link.append(self.local_reward_delay)
                #print("local delay reward link: ",self.local_reward_delay)
                #print("count_sfc_link",count_sfc_link)
                #print("updated_path_perSFC", self.updated_path_perSFC)
            #print("end of cont 3: accpeted_action", self.accpeted_action)
            #print("temp_accpeted_action", temp_accpeted_action)
        #print("+++++++++++++ trail_ac", trail_ac)
        return 
    


In [32]:
class microvnf_constraints():
    def __init__ (self, action, micro_vnf_res, amount_local_reward, microvnf_accpeted_action):
        #print("import all the reqired inputs")
        self.action = action
        self.micro_vnf_res = micro_vnf_res
        self.amount_local_reward = amount_local_reward
        self.microvnf_accpeted_action = microvnf_accpeted_action
        #print("microvnf_accpeted_action", self.microvnf_accpeted_action)
        
    def constraint_1(self):
        #print("Constraint 1")
        for key, available_res in env.network_nodes:
            if available_res['id'] == self.action:
                if available_res['emb_vnf'] >= self.micro_vnf_res: 
                    local_rewardmicroVNF = self.amount_local_reward
                    #print("Reward", local_rewardmicroVNF)
                    self.microvnf_accpeted_action.append(self.action)
                else:
                    #print("insufficent resouce in micro")
                    #print(self.micro_vnf_res)
                    local_rewardmicroVNF = -1000
        
        return local_rewardmicroVNF
    
    
    def constraint_2(self, microstate_space_size, microvnf_link_resources):

        self.microstate_space_size = microstate_space_size
        #print(self.microstate_space_size)
        self.microvnf_link_resources = microvnf_link_resources
        
        self.threshold_delay    = 50 #30
        self.local_reward_link  = []
        self.local_reward_delay = []
        local_reward_delay = 0
        
        #print("microvnf_accpeted_action", microvnf_accpeted_action)
        if len(self.microvnf_accpeted_action) == self.microstate_space_size:
            count_microsfc_link = 1
            #print("Constraint 2")
            #print(self.microvnf_accpeted_action)
            for head_link, end_link, requested_bw in self.microvnf_link_resources:
                
                
                #Finding the Path
                Start_node = self.microvnf_accpeted_action[head_link]
                End_node   = self.microvnf_accpeted_action[end_link]
                #print(self.microvnf_link_resources)
                #print(type(Start_node), type(End_node))
                
                if type(Start_node) == list:
                    Start_node = Start_node[1]
                    
                if type(End_node) is list:
                    End_node = End_node[1]
                    
                if Start_node == End_node:
                    local_reward_delay = 1000
                    intralink = ['intra link']
                    updated_path_perVNF.append(intralink)
                    #sprint("updated_path_perVNF", updated_path_perVNF)
                    #print("local_reward_delay", local_reward_delay)
                    count_microsfc_link +=1
                else: 
                
                    path = nx.all_simple_paths(env.network_graph,
                                               str(Start_node),
                                               str(End_node))
                    #print("Path", path)
                    
                    #shortest_path = nx.all_shortest_paths(env.network_graph, 
                    #                                      str(Start_node), 
                    #                                     str(End_node),
                    #                                      weight=None, 
                    #                                      method='dijkstra')
                    
                    count_path = 0
                    tot_score, delay_per_link =[], []
                    len_delay_per_link,len_each_path =[], []
                    score = 0

                    for each_path in path:
                        #print("each path", each_path)
                        delay_per_link =[]
                        updated_path_permVNF = each_path

                        if score == 0:
                            len_dis = len(each_path)
                            count_dis = 0
                            source_node, target_node =[], []

                            for key in each_path:
                                if count_dis == len_dis-1:
                                    break
                                else:
                                    source_node = each_path[count_dis]
                                    target_node = each_path[count_dis+1]
                                    #print(source_node, target_node)

                                for initial_link, final_link, available_bw in env.network_edges:   
                                    if (initial_link == source_node and final_link == target_node) or (initial_link == target_node and final_link == source_node): 
                                        #Checking for the available resources
                                        sfc_avail_bw    = available_bw['bw']
                                        vnf_req_bw      = requested_bw['bw']
                                        sfc_avail_delay = available_bw['latency']
                                        vnf_req_delay   = requested_bw['latency']


                                        if (sfc_avail_bw >= vnf_req_bw): 
                                            local_reward_link = self.amount_local_reward
                                            score = 1
                                            delay_per_link.append(sfc_avail_delay) 
                                            #print(" link Reward ",local_reward_link)
                                        else: 
                                            local_reward_link = -10000
                                            pen_score = 2
                                            score = pen_score
                                            #print("link penalty") 
                                            pass

                                count_dis += 1

                            tot_delay_perPath  = sum(delay_per_link)
                            #print(each_path, tot_delay_perPath)
                            len_delay_per_link = len(delay_per_link)
                            len_each_path      = len(each_path)-1

                            self.local_reward_link.append(local_reward_link)
                            #print("local reward link: ",self.local_reward_link)

                            if len_delay_per_link == len_each_path: #end of the path
                                if tot_delay_perPath <= self.threshold_delay:
                                    updated_path_perVNF.append(each_path)
                                    #print("Each path", each_path, tot_delay_perPath)
                                    local_reward_delay = self.amount_local_reward
                                    #print("Reward in latency", score)
                                    #print(" link Reward ",local_reward_delay)
                                    pass
                                else: 
                                    score = 0
                                    #print("link not found microvnf")
                                    local_reward_delay = -10000
                            else:
                                score = 0
                                local_reward_delay = -10000
                        else:
                            break


                        count_path +=1
                    count_microsfc_link += 1   
                
                self.local_reward_delay.append(local_reward_delay)
                #print(self.local_reward_delay)

                
        #print("local reward delay: ", self.local_reward_delay)
        return 
                                    


In [33]:
class Reward_Function():
    
    def __init__ (self):
        self.localreward = 1000
        
    def reward_vnf(self, global_reward, rewardDelay_per_current_states, rewardlink_per_current_states, 
                   actions_per_iteration,
                   accpeted_action, vnf_node_resources, count_vnf, updated_path_perSFC, vnf_link_resources, 
                   count_updated_path_perSFC, count_accpeted, global_decomp, emb_mVNFID, switch_reconst, 
                   global_updated_path_perVNF):
        
        self.global_reward = global_reward
        self.rewardDelay_per_current_states = rewardDelay_per_current_states
        self.rewardlink_per_current_states = rewardlink_per_current_states
        self.actions_per_iteration = actions_per_iteration
        self.accpeted_action = accpeted_action
        self.vnf_node_resources = vnf_node_resources
        self.count_vnf = count_vnf
        self.updated_path_perSFC = updated_path_perSFC
        self.vnf_link_resources = vnf_link_resources
        self.count_updated_path_perSFC = count_updated_path_perSFC
        self.count_accpeted = count_accpeted
        self.global_decomp = global_decomp
        self.emb_mVNFID = emb_mVNFID
        self.global_updated_path_perVNF = global_updated_path_perVNF
        #print("inside reward funtion:  accpeted_action",accpeted_action)

        done = True
        reward_all = 1000*10
        rewars_delay = sum(self.rewardDelay_per_current_states)
        reward_link = sum(self.rewardlink_per_current_states)
        self.global_reward = reward_all + rewars_delay + reward_link 
        self.actions_per_iteration = self.accpeted_action
        score = 1
        self.count_accpeted +=1
        #print("emb", self.actions_per_iteration)
        #---Embedding Starts here---#
        env.embedding_vnf(self.actions_per_iteration, self.vnf_node_resources, self.count_vnf, self.global_decomp, 
                          self.emb_mVNFID, switch_reconst)
        self.count_vnf = env.count_vnf

        ##---- Link Placement ----##
        #print("self.updated_path_perSFC", self.updated_path_perSFC)
        #print("self.vnf_link_resources", self.vnf_link_resources)
        #print("self.count_updated_path_perSFC", self.count_updated_path_perSFC)
    
        env.embedding_vl(self.updated_path_perSFC, self.vnf_link_resources, self.count_updated_path_perSFC, self.global_decomp, 
                        self.global_updated_path_perVNF)   
        self.count_updated_path_perSFC = env.count_updated_path_perSFC
        
        return 


In [34]:
from numpy import vstack
from numpy import argmax
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import MSELoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

# model definition
class MLP(Module):
    print("inside the MLP model")
    # define model elements
    def __init__(self, tr_n_state, tr_n_actions, hidden_neuron):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(tr_n_state[0], hidden_neuron)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        # second hidden layer
        self.hidden2 = Linear(hidden_neuron, hidden_neuron)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        # third hidden layer and output
        self.hidden3 = Linear(hidden_neuron, tr_n_actions)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Sigmoid()
 
    # forward propagate input
    def forward(self, X):
        #print("x forward: ", X)
        #X = T.tensor([X], dtype = T.float)
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        X = self.act3(X)
        return X
    
    
    def traffic_prepare_train_data(tr_input_train_mem, tr_target_train_mem, tr_input_train_data, tr_target_train_data,
                                X, Y, traf_mem_cnt, traf_mem_size, traf_batch_size):
        data = [X, Y]
        #print(data)
        #storing the dataset
        index = traf_mem_cnt % traf_mem_size
        tr_input_train_mem[index] = X
        tr_target_train_mem[index] = Y[0]
        traf_mem_cnt += 1
        #print(tr_mem_cnt, tr_batch_size)

        if traf_mem_cnt < traf_batch_size:       
            batch = np.random.choice(traf_mem_cnt, traf_mem_cnt, replace =False)
            tr_input_train_data = T.tensor(tr_input_train_mem[batch], dtype=T.float)
            tr_target_train_data = T.tensor(tr_target_train_mem[batch], dtype=T.float)
        else:
            max_mem = min(traf_mem_cnt, traf_mem_size)
            batch = np.random.choice(max_mem, traf_batch_size, replace =False) 
            #print("batch: ", batch)
            tr_input_train_data = T.tensor(tr_input_train_mem[batch], dtype=T.float)
            tr_target_train_data = T.tensor(tr_target_train_mem[batch], dtype=T.float)

        return tr_input_train_data, tr_target_train_data, traf_mem_cnt

    # train the model
    def train_model(input_train_data, target_train_data, pr_MLP_model):
        # define the optimization
        criterion = MSELoss()
        optimizer = optim.Adam(pr_MLP_model.parameters(), lr=0.001) #, momentum=0.9)
        optimizer.zero_grad()
        yhat = pr_MLP_model(input_train_data)
        loss = criterion(yhat, target_train_data)
        loss.backward()
        optimizer.step()
        #print(yhat)
        return yhat[0], loss


    # evaluate the model
    def evaluate_model(input_test_data, target_test_data, pr_MLP_model):
        predictions, actuals = list(), list()

        # evaluate the model on the test set
        yhat = pr_MLP_model(input_test_data)

        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = target_test_data.detach().numpy()

        actual = actual.reshape((len(actual), 1))
        yhat = yhat.reshape((len(yhat), 1))
        #print(yhat, actual)

        # store
        predictions.append(yhat)
        actuals.append(actual)

        predictions, actuals = vstack(predictions), vstack(actuals)
        # calculate accuracy
        mse = mean_squared_error(actuals, predictions)
        return mse, yhat


    # make a class prediction for one row of data
    def predict(X, pr_MLP_model):
        # convert row to data
        X = T.tensor([X], dtype = T.float)  #T.tensor([X], dtype = T.float)
        # make prediction
        yhat = pr_MLP_model(X)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        #print("predict yhat: ", yhat)
        return yhat


inside the MLP model


In [35]:
def prepare_train_data(input_train_mem, target_train_mem, input_train_data, target_train_data,
                       X, Y, tr_mem_cnt, tr_mem_size, tr_batch_size):
    data = [X, Y]
    #print(data)
    #storing the dataset
    index = tr_mem_cnt % tr_mem_size
    input_train_mem[index] = X
    target_train_mem[index] = Y[0]
    tr_mem_cnt += 1
    #print(tr_mem_cnt, tr_batch_size)
    
    if tr_mem_cnt < tr_batch_size:       
        batch = np.random.choice(tr_mem_cnt, tr_mem_cnt, replace =False)
        input_train_data = T.tensor(input_train_mem[batch], dtype=T.float)
        target_train_data = T.tensor(target_train_mem[batch], dtype=T.float)
    else:
        max_mem = min(tr_mem_cnt, tr_mem_size)
        batch = np.random.choice(max_mem, tr_batch_size, replace =False) 
        #print("batch: ", batch)
        input_train_data = T.tensor(input_train_mem[batch], dtype=T.float)
        target_train_data = T.tensor(target_train_mem[batch], dtype=T.float)

    return input_train_data, target_train_data, tr_mem_cnt


def prepare_test_data(input_test_mem, target_test_mem, input_test_data, target_test_data, 
                      test_mem_cnt, test_mem_size, test_batch_size, X, Y):
    data = [X, Y]
    index = test_mem_cnt % test_mem_size
    input_test_mem[index] = X
    target_test_mem[index] = Y[0]

    test_mem_cnt += 1

    if test_mem_cnt < test_batch_size:    
        batch = np.random.choice(test_mem_cnt, test_mem_cnt, replace =False)
        input_test_data  = T.tensor(input_test_mem[batch], dtype=T.float)
        target_test_data = T.tensor(target_test_mem[batch], dtype=T.float)
    else:
        max_mem = min(test_mem_cnt, test_mem_size)
        batch = np.random.choice(max_mem, test_batch_size, replace =False) 
        input_test_data  = T.tensor(input_test_mem[batch], dtype=T.float)
        target_test_data = T.tensor(target_test_mem[batch], dtype=T.float)

    return input_test_data, target_test_data, test_mem_cnt   



def wfq_sch(arr_sfc_de, h_pr_queue, m_pr_queue, l_pr_queue):
    wt_H,wt_M, wt_L = 20,10,5
    temp_sorted = []
    cnt = 0
    cnt_wt_H, cnt_wt_M, cnt_wt_L = 0, 0, 0
    status = False

    
    while status != True:
        
        if cnt == (len(arr_sfc_de)):
            #print("here", cnt)
            status = True
        else: 
            # high pr queue
            if not h_pr_queue:
                pass
            else: 
                temp_cnt_wt_H = 0
                if len(h_pr_queue) >= wt_H:
                        cond1 = wt_H
                else:
                    cond1 = len(h_pr_queue)
                while temp_cnt_wt_H <= (cond1-1):
                    var = h_pr_queue[0]
                    temp_sorted.append(var)
                    h_pr_queue.remove(var)
                    cnt_wt_H +=1
                    temp_cnt_wt_H += 1
                    cnt += 1

            # medium pr queue
            if not m_pr_queue:
                pass
            else: 
                temp_cnt_wt_M = 0
                if len(m_pr_queue) >= wt_M:
                        cond1 = wt_M
                else:
                    cond1 = len(m_pr_queue)

                while temp_cnt_wt_M <= (cond1-1):
                    var = m_pr_queue[0]
                    #print(var)
                    temp_sorted.append(var)
                    m_pr_queue.remove(var)
                    cnt_wt_M +=1
                    temp_cnt_wt_M += 1
                    cnt += 1

            # low pr queue
            if not l_pr_queue:
                pass
            else: 
                temp_cnt_wt_L = 0
                if len(l_pr_queue) >= wt_L:
                        cond1 = wt_L
                else:
                    cond1 = len(l_pr_queue)
                while temp_cnt_wt_L <= (cond1-1):
                    var = l_pr_queue[0]
                    temp_sorted.append(var)
                    l_pr_queue.remove(var)
                    cnt_wt_L +=1
                    temp_cnt_wt_L += 1
                    cnt += 1
    
    return temp_sorted
 

In [36]:
#h_pr_queue =  [1, 3, 5]
#m_pr_queue =  [0, 2, 6, 4]
#l_pr_queue =  [] #[1, 3, 5]
#print(len(arr_sfc_de))

#ww = wfq_sch(arr_sfc_de, h_pr_queue, m_pr_queue, l_pr_queue)


## Adaptive scheduling

In [37]:
class arrival_sfc():
    def  __init__ (self, num_sfc, arr_sfc_dist):
        
        self.num_sfc = num_sfc
        self.arr_sfc_dist = arr_sfc_dist
    
    def sfc_poisson_dist(self):

        #generate Poisson distribution with sample size 10000
        for i in range(self.num_sfc): 
            sfc_poisson = poisson.rvs(mu=self.num_sfc, size=1)
            print("sfc_poisson: ", sfc_poisson)
            temp_arr_sfc_dist  = sum(self.arr_sfc_dist) + int(sfc_poisson)
            print("->",temp_arr_sfc_dist)
            done_dist = False
            if temp_arr_sfc_dist == self.num_sfc: 
                done_dist = True
                self.arr_sfc_dist.append(int(sfc_poisson))
                break

            elif temp_arr_sfc_dist < self.num_sfc:
                done_dist = False
                self.arr_sfc_dist.append(int(sfc_poisson))
            else:
                done_dist = False
                pass
        print("arr_sfc_dist: ", arr_sfc_dist)
        return self.arr_sfc_dist
    
    
    def sfc_details(self,per_episode,arr_sfc_dist, sfc_arrival_detail,sfc_arrival_num):
        
        self.per_episode = per_episode
        self.arr_sfc_dist = arr_sfc_dist
        self.sfc_arrival_detail = sfc_arrival_detail
        self.sfc_arrival_num = sfc_arrival_num
        cnt = 0
        for i in self.arr_sfc_dist:
            temp, temp_detail = [], []
            for ii in range(i):
                temp.append(cnt)
                temp_detail.append(self.per_episode[cnt])
                cnt += 1
            self.sfc_arrival_num.append(temp)
            self.sfc_arrival_detail.append(temp_detail)      
        #print("=>", self.sfc_arrival_num) #self.sfc_arrival_detail, self.sfc_arrival_num)
        return   
    
    

In [38]:
arr_sfc_dist = []
class_arr_sfc_dist = arrival_sfc(num_sfc, arr_sfc_dist)
arr_sfc_dist = list(class_arr_sfc_dist.sfc_poisson_dist())
sfc_arrival_detail, sfc_arrival_num = [], []
#arr_sfc_detail = class_arr_sfc_dist.sfc_details(per_episode, arr_sfc_dist, sfc_arrival_detail, sfc_arrival_num)

#print(sfc_arrival_detail)

sfc_poisson:  [56]
-> 56
sfc_poisson:  [49]
-> 49
sfc_poisson:  [44]
-> 93
sfc_poisson:  [55]
-> 104
sfc_poisson:  [45]
-> 94
sfc_poisson:  [54]
-> 103
sfc_poisson:  [41]
-> 90
sfc_poisson:  [56]
-> 105
sfc_poisson:  [43]
-> 92
sfc_poisson:  [56]
-> 105
sfc_poisson:  [56]
-> 105
sfc_poisson:  [54]
-> 103
sfc_poisson:  [60]
-> 109
sfc_poisson:  [62]
-> 111
sfc_poisson:  [45]
-> 94
sfc_poisson:  [51]
-> 100
sfc_poisson:  [46]
-> 95
sfc_poisson:  [48]
-> 97
sfc_poisson:  [44]
-> 93
sfc_poisson:  [47]
-> 96
sfc_poisson:  [61]
-> 110
sfc_poisson:  [49]
-> 98
sfc_poisson:  [60]
-> 109
sfc_poisson:  [43]
-> 92
sfc_poisson:  [61]
-> 110
sfc_poisson:  [57]
-> 106
sfc_poisson:  [52]
-> 101
sfc_poisson:  [45]
-> 94
sfc_poisson:  [60]
-> 109
sfc_poisson:  [47]
-> 96
sfc_poisson:  [44]
-> 93
sfc_poisson:  [54]
-> 103
sfc_poisson:  [60]
-> 109
sfc_poisson:  [57]
-> 106
sfc_poisson:  [61]
-> 110
sfc_poisson:  [60]
-> 109
sfc_poisson:  [46]
-> 95
sfc_poisson:  [52]
-> 101
sfc_poisson:  [45]
-> 94
sfc_

In [39]:
class traffic_clustering():
    def  __init__ (self, traffic_num_sfc_requested, epi_temp_sfc_arrived, deployed_sfc_size, sfc_deployment_time, 
                           arr_traffic_packet_per_sfc, traffic_transition_dict, traffic_batch_size, episode):
        
        self.traffic_num_sfc_requested = traffic_num_sfc_requested
        self.epi_temp_sfc_arrived = epi_temp_sfc_arrived
        self.deployed_sfc_size = deployed_sfc_size
        self.sfc_deployment_time = sfc_deployment_time
        self.arr_traffic_packet_per_sfc = arr_traffic_packet_per_sfc
        self.traffic_transition_dict = traffic_transition_dict
        self.traffic_batch_size = traffic_batch_size
        self.episode = episode
        #print("--:Traffic Tansistions:--", self.traffic_transition_dict)

    def input_traffic_transistion_storage(self, x_traffic_mem, traffic_mem_size, input_train_traffic_mem_cnt):
        self.x_traffic_mem = x_traffic_mem 
        self.input_train_traffic_mem_cnt = input_train_traffic_mem_cnt
        self.traffic_mem_size = traffic_mem_size
        
        for i in self.epi_temp_sfc_arrived: 
            #storing the dataset
            #print(i)
            index = self.input_train_traffic_mem_cnt % self.traffic_mem_size
            #print(index)
            self.x_traffic_mem[index] = self.traffic_transition_dict[i]
            self.input_train_traffic_mem_cnt += 1
            
            if self.input_train_traffic_mem_cnt < self.deployed_sfc_size:
                batch = np.random.choice(self.input_train_traffic_mem_cnt, self.input_train_traffic_mem_cnt, replace =False)
                input_train_data = T.tensor(x_traffic_mem[batch], dtype=T.float)
            else:
                max_mem = min(self.input_train_traffic_mem_cnt, self.traffic_mem_size)
                batch = np.random.choice(max_mem, self.deployed_sfc_size, replace =False) 
                input_train_data = T.tensor(x_traffic_mem[batch], dtype=T.float)
                
        #print(input_train_data.tolist())                
        return self.input_train_traffic_mem_cnt, input_train_data
        
    
    def kmean(self, n_clusters, traffic_input_dataset, kmeans_runtime):
        kmean_start = time.perf_counter()
        self.n_clusters = n_clusters
        self.traffic_input_dataset = traffic_input_dataset.tolist()
        kmeans = KMeans(self.n_clusters)
        kmeans.fit(self.traffic_input_dataset)
        kmean_labels = kmeans.labels_
        kmean_end = time.perf_counter()
        kmean_temp_Rtime = kmean_end - kmean_start
        kmeans_runtime.append(kmean_temp_Rtime)
        return kmean_labels.tolist()
    
    def hierarchical(self, n_clusters, traffic_input_dataset, Aggo_runtime):
        Aggo_start = time.perf_counter()
        self.n_clusters = n_clusters
        self.traffic_input_dataset = traffic_input_dataset.tolist()
        hierarchical_cluster = AgglomerativeClustering(n_clusters=self.n_clusters, affinity='euclidean', linkage='ward')
        hierarchical_labels = hierarchical_cluster.fit_predict(self.traffic_input_dataset)
        Aggo_end = time.perf_counter()
        Aggo_temp_Rtime = Aggo_end - Aggo_start
        Aggo_runtime.append(Aggo_temp_Rtime)
        return hierarchical_labels.tolist()
    
    def brc(self, n_clusters, traffic_input_dataset, birch_runtime):
        birch_start = time.perf_counter()
        self.n_clusters = n_clusters
        self.traffic_input_dataset = traffic_input_dataset.tolist()
        brc = Birch(n_clusters = self.n_clusters)
        brc_labels = brc.fit_predict(self.traffic_input_dataset)
        birch_end = time.perf_counter()
        birch_temp_Rtime = birch_end - birch_start
        birch_runtime.append(birch_temp_Rtime)
        return brc_labels.tolist()
    
    #def spectral(self, n_clusters, traffic_input_dataset):
    #    self.n_clusters = n_clusters
    #    self.traffic_input_dataset = traffic_input_dataset.tolist()
    #    spectral = SpectralClustering(n_clusters=self.n_clusters, eigen_solver = 'amg') #{‘arpack’, ‘lobpcg’, ‘amg’})
    #    spectral_labels = spectral.fit_predict(self.traffic_input_dataset)
    #    return spectral_labels.tolist()
    
    def GM(self, n_clusters, traffic_input_dataset, GM_runtime):
        GM_start = time.perf_counter()
        self.n_clusters = n_clusters
        self.traffic_input_dataset = traffic_input_dataset.tolist()
        GM = GaussianMixture(n_components=self.n_clusters)
        GM.fit(self.traffic_input_dataset)
        GM_labels = GM.predict(self.traffic_input_dataset)
        GM_end = time.perf_counter()
        GM_temp_Rtime = GM_end - GM_start
        GM_runtime.append(GM_temp_Rtime)
        return GM_labels.tolist()
    
    def clusteringplot(self, epi_clustering_status, traffic_input_dataset, cluster_label_val, epi_temp_sfc_arrived):
        self.epi_clustering_status = epi_clustering_status #labels
        self.traffic_input_dataset = traffic_input_dataset.tolist() #x and y axis
        self.cluster_label_val = cluster_label_val
        self.epi_temp_sfc_arrived = epi_temp_sfc_arrived
        
        clustering_x_axis_sfc, clustering_y_axis_priority, clustering_y_axis_packets, clustering_y_axis_load = [], [], [], []
        
        cnt_ = 0
        for i in self.traffic_input_dataset:
            clustering_x_axis_sfc.append(self.epi_temp_sfc_arrived[cnt_])
            clustering_y_axis_priority.append(i[0]*100)
            clustering_y_axis_packets.append(i[2])
            clustering_y_axis_load.append(i[3]*10)
            cnt_ += 1

        for i in epi_clustering_status:
            #print(i)
            temp_HD_sfc = []
            cluster_title = epi_clustering_status[i][0]
            labels = epi_clustering_status[i][1]
            #print(labels)
            
            for j in range(len(labels)):
                if labels[j] == 1:
                    temp_HD_sfc.append(clustering_x_axis_sfc[j])
                    
            self.cluster_label_val[i] = [cluster_title, temp_HD_sfc]
    
            fig, ax = plt.subplots(1)
            ax.scatter(clustering_x_axis_sfc, clustering_y_axis_load, s= clustering_y_axis_packets,c=labels,alpha = 0.7)
            ax.legend()
            ax.set_xlabel("Services")
            ax.set_ylabel("Traffic Load")
            #ax.set_title(cluster_title)
            
            #ax[1].scatter(clustering_x_axis_sfc, clustering_y_axis_packets, s= clustering_y_axis_priority, c=labels)
            #ax[1].set_xlabel("Services")
            #ax[1].set_ylabel("Traffic Load")
            #ax[1].set_title(cluster_title)
            
            #plt.show()
            title = "Clustering Experiment"
            dt = datetime.now()
            # getting the timestamp
            ts = datetime.timestamp(dt)
            # convert to datetime
            date_time = datetime.fromtimestamp(ts)
            # convert timestamp to string in dd-mm-yyyy HH:MM:SS
            str_date_time = date_time.strftime("%d-%m-%Y-%H-%M-%S")
            #print(str_date_time)
            
            figure_title = str(title)+"_"+str(cluster_title)+"_"+str(len(labels))+"_"+str(self.episode)+"_"+date_time.strftime("%d-%m-%Y-%H-%M-%S")
            ##saving the figure
            fig.savefig('Generated_plots/cluster/epi_2000_100sfc_agglo_DT/Plot_'+str(figure_title)+'.png')
            fig.savefig('Generated_plots/cluster/epi_2000_100sfc_agglo_DT/Plot_'+str(figure_title)+'.pdf', bbox_inches='tight')
            
            #saving clustering data
            cluster_data = ["Time: "+str_date_time+ "Title: " + str(title) + "Cluster model: " + str(cluster_title) +\
                            "clustering_x_axis_sfc: " + str(clustering_x_axis_sfc) +\
                            "clustering_y_axis_packets: " + str(clustering_y_axis_packets) +\
                            "clustering_y_axis_load: " + str(clustering_y_axis_load) +\
                            "clustering_y_axis_priority: " + str(clustering_y_axis_priority) +\
                            "labels: " + str(labels) +\
                            "cluster_label_val: " + str(self.cluster_label_val)]
            
            fname = str(title)+"_"+str(cluster_title)+"_"+str(len(labels))+"_"+str(self.episode)+"_"+str_date_time
            print(fname)
            f = open('Generated_plots/cluster/epi_2000_100sfc_agglo_DT/'+fname,"a")
            f.write(str(cluster_data))
            f.close()
            print("XXXXXXXXXXXXXXXXX")
            print("")

        
        return self.cluster_label_val

        

In [40]:
## Classifer
class traffic_classifer():
    def  __init__ (self, traffic_input_dataset, traffic_output_dataset, episode, cluster_testing):
        
        if traffic_input_dataset == []:
            self.traffic_input_dataset  = traffic_input_dataset
        else:
            self.traffic_input_dataset  = traffic_input_dataset.tolist()
        
        self.traffic_output_dataset = traffic_output_dataset
        self.episode = episode
        self.cluster_testing = cluster_testing
        #print("self.traffic_input_dataset:", self.traffic_input_dataset)
        #print("self.traffic_output_dataset: ", self.traffic_output_dataset)
        #print("Traffic Classifier")

    def classifier_dataset(self, train_traffic_classifier_input_dataset, train_traffic_classifier_output_dataset,
                                 test_traffic_classifier_input_dataset,  test_traffic_classifier_output_dataset):
        birch_cluster
        self.train_traffic_classifier_input_dataset  = train_traffic_classifier_input_dataset
        self.train_traffic_classifier_output_dataset = train_traffic_classifier_output_dataset
        self.test_traffic_classifier_input_dataset   = test_traffic_classifier_input_dataset
        self.test_traffic_classifier_output_dataset  = test_traffic_classifier_output_dataset
        #print("Classifier")

        if self.cluster_testing == True:
            training_index = int((len(self.traffic_input_dataset)*60)/100)
            #input datateset
            for i in range(len(self.traffic_input_dataset)):
                if i <= (training_index-1):
                    self.train_traffic_classifier_input_dataset.append(self.traffic_input_dataset[i])
                    self.train_traffic_classifier_output_dataset.append(self.traffic_output_dataset[i])
                else:
                    self.test_traffic_classifier_input_dataset.append(self.traffic_input_dataset[i])
                    self.test_traffic_classifier_output_dataset.append(self.traffic_output_dataset[i])                 
        else:
            for i in range(len(self.traffic_input_dataset)):
                self.train_traffic_classifier_input_dataset.append(self.traffic_input_dataset[i])
                self.train_traffic_classifier_output_dataset.append(self.traffic_output_dataset[i])

        return self.train_traffic_classifier_input_dataset, self.train_traffic_classifier_output_dataset, self.test_traffic_classifier_input_dataset, self.test_traffic_classifier_output_dataset
        
    def classifier_models(self, classifier_models, classifier_datasets, predict_dataset, classifier_training_status, Switch_classifier_info):
        self.classifier_models = classifier_models
        self.train_traffic_classifier_input_dataset =  classifier_datasets[0]
        self.train_traffic_classifier_output_dataset = classifier_datasets[1]
        self.test_traffic_classifier_input_dataset = classifier_datasets[2]
        self.test_traffic_classifier_output_dataset = classifier_datasets[3]
        self.predict_dataset = predict_dataset
        self.classifier_training_status = classifier_training_status
        #print("train input: classifier_datasets", self.train_traffic_classifier_input_dataset)
        #print("train output: classifier_datasets", self.train_traffic_classifier_output_dataset)
        #print("test input: classifier_datasets", self.test_traffic_classifier_input_dataset)
        #print("test output: classifier_datasets", self.test_traffic_classifier_output_dataset)
        
        
        if self.episode == 0 and self.classifier_training_status == True:
            if Switch_classifier_info == 'Decision Trees': 
                self.classifier_models['Decision Trees'] = DecisionTreeClassifier()
            elif Switch_classifier_info == 'Logistic Regression': 
                self.classifier_models['Logistic Regression'] = LogisticRegression()
            elif Switch_classifier_info == 'Support Vector Machines': 
                self.classifier_models['Support Vector Machines'] = LinearSVC()
            elif Switch_classifier_info == 'Random Forest':            
                self.classifier_models['Random Forest'] = RandomForestClassifier()
            elif Switch_classifier_info == 'Naive Bayes': 
                self.classifier_models['Naive Bayes'] = GaussianNB()
            elif Switch_classifier_info == 'K-Nearest Neighbor': 
                self.classifier_models['K-Nearest Neighbor'] = KNeighborsClassifier()
            else: 
                print("no classifer mdel found")
                
        accuracy, precision, recall = {}, {}, {}
        Runtime_classifier = {}
        
        for key in self.classifier_models.keys():
            #print("key:", key)
            if self.classifier_training_status == True:
                # Fit the classifier
                R_classifier_start = time.perf_counter()
                count_zero = self.train_traffic_classifier_output_dataset.count(0)
                count_one  = self.train_traffic_classifier_output_dataset.count(1)
                if count_zero == 0 or count_one == 0:
                    print("no learning as all the output sets are of same class")
                    pass
                else:
                    self.classifier_models[key].fit(self.train_traffic_classifier_input_dataset, 
                                                    self.train_traffic_classifier_output_dataset)
                    R_classifier_end   = time.perf_counter()
                    temp_class_runtime = (R_classifier_end - R_classifier_start)
                    Runtime_classifier[key] = temp_class_runtime
                    #print("Runtime_classifier: ",Runtime_classifier)
                    #print("self.classifier_models: ", self.classifier_models)

                    # Make predictions
                    #print("self.test_traffic_classifier_input_dataset: ",self.test_traffic_classifier_input_dataset)
                    predictions = self.classifier_models[key].predict(self.test_traffic_classifier_input_dataset)

                    # Calculate metrics
                    accuracy[key] = accuracy_score(predictions, self.test_traffic_classifier_output_dataset)
                    precision[key] = precision_score(predictions, self.test_traffic_classifier_output_dataset)
                    recall[key] = recall_score(predictions, self.test_traffic_classifier_output_dataset)
                    #online_predictions = self.classifier_models[key].predict([self.predict_dataset]) 
                    #print("online_predictions: ", online_predictions)
                
            else:
                #trail prediction
                #print("Key: ", key, "PREDICTION")
                predictions = self.classifier_models[key].predict([self.predict_dataset])
                #predictions = self.classifier_models[key].predict([self.predict_dataset])  
                #online_predictions = self.classifier_models[key].predict([self.predict_dataset])  
                
        return accuracy, precision, recall, Runtime_classifier, predictions
    
    
    #def pred_classifier(self, predict_dataset, classifier_models):
    #    self.predict_dataset = predict_dataset
    #    self.classifier_models = classifier_models
    #    
    #    for key in self.classifier_models.keys():
           # online_predictions = self.classifier_models[key].predict([self.predict_dataset])  
    #    return online_predictions
    
    
    
    def classifier_plot(self, classifier_measurements,num_sfc, baseline):
            #overall_epi_pr_emb
            col4 = 'lightcyan' 
            col5 =  '#CDB5CD' 
            col6 =  '#548B54' #'#7CCD7C'#'#4EEE94'
            col7 =  '#FFE7BA'
            col8 =  '#8B3A62' #'hotpink4'
            col9 =  '#C67171' #'lightcoral'#'salmon'#'tomato'#'#8F8F8F'#'powderblue'

            
            xaxis_clustering_models = []
            self.classifier_measurements = classifier_measurements
            log_acc, svm_acc, dt_acc, rf_acc, nb_acc, knn_acc = [],[],[],[],[],[]
            log_pre, svm_pre, dt_pre, rf_pre, nb_pre, knn_pre = [],[],[],[],[],[]
            log_recall, svm_recall, dt_recall, rf_recall, nb_recall, knn_recall = [],[],[],[],[],[]
            log_runtime, svm_runtime, dt_runtime, rf_runtime, nb_runtime, knn_runtime = [],[],[],[],[],[]
            print("baseline: ", baseline)
            for key in self.classifier_measurements:
                cluster_model = key
                print("cluster_model: ", cluster_model)
                xaxis_clustering_models.append(key)
                models_acc = self.classifier_measurements[key][0] 
                models_precision = self.classifier_measurements[key][1] 
                models_recall = self.classifier_measurements[key][2] 
                models_runtime = self.classifier_measurements[key][3] 
                
                log_acc.append(models_acc['Logistic Regression'])
                svm_acc.append(models_acc['Support Vector Machines'])
                dt_acc.append(models_acc['Decision Trees'])
                rf_acc.append(models_acc['Random Forest']) 
                nb_acc.append(models_acc['Naive Bayes'])
                knn_acc.append(models_acc['K-Nearest Neighbor'])
                
                log_pre.append(models_precision['Logistic Regression'])
                svm_pre.append(models_precision['Support Vector Machines'])
                dt_pre.append(models_precision['Decision Trees'])
                rf_pre.append(models_precision['Random Forest']) 
                nb_pre.append(models_precision['Naive Bayes'])
                knn_pre.append(models_precision['K-Nearest Neighbor'])
                
                log_recall.append(models_recall['Logistic Regression'])
                svm_recall.append(models_recall['Support Vector Machines'])
                dt_recall.append(models_recall['Decision Trees'])
                rf_recall.append(models_recall['Random Forest']) 
                nb_recall.append(models_recall['Naive Bayes'])
                knn_recall.append(models_recall['K-Nearest Neighbor'])
                
                log_runtime.append(models_runtime['Logistic Regression'])
                svm_runtime.append(models_runtime['Support Vector Machines'])
                dt_runtime.append(models_runtime['Decision Trees'])
                rf_runtime.append(models_runtime['Random Forest']) 
                nb_runtime.append(models_runtime['Naive Bayes'])
                knn_runtime.append(models_runtime['K-Nearest Neighbor'])

            xaxis_classifier_models = []
            for key in models_acc:
                xaxis_classifier_models.append(key)
            #print("xaxis_classifier_models" , xaxis_classifier_models)
            print("xaxis_clustering_models: ", xaxis_clustering_models)
        
            title  = 'Performace of Clustering and Classifier Models'
            xlabel = 'Clustering Models'
            ylabel = 'Accuracy'
            #print(title)
            
            # set width of bar
            barWidth = 0.15
            fig1 = plt.figure(figsize=(15,4))
            
            # set height of bar
            x_axis = xaxis_clustering_models #xaxis_classifier_models
            y_axis_baseline = []
            for i in range(len(x_axis)):
                y_axis_baseline.append(baseline)
            
            
            ## Set position of bar on X axis
            br1 = np.arange(len(xaxis_clustering_models))
            br2 = [x + barWidth for x in br1]
            br3 = [x + barWidth for x in br2]
            br4 = [x + barWidth for x in br3]
            br5 = [x + barWidth for x in br4]
            br6 = [x + barWidth for x in br5]
            
            # Make the plot
            plt.plot(br1, y_axis_baseline, color = 'cyan')
            plt.bar(br1, log_acc, color = col4, width = barWidth,
                    edgecolor ='#2E2E2E', label ='LR',  hatch='/////')
            
            plt.plot(br2, y_axis_baseline, color = 'cyan')
            plt.bar(br2, svm_acc, color =col5, width = barWidth,
                    edgecolor ='#2E2E2E', label ='SVM', hatch='----')
            
            plt.plot(br3, y_axis_baseline, color = 'cyan')
            plt.bar(br3, dt_acc,  color =col6, width = barWidth,
                    edgecolor ='#2E2E2E', label ='DT', hatch='xxxx')
            
            plt.plot(br4, y_axis_baseline, color = 'cyan')
            plt.bar(br4, rf_acc, color =col7, width = barWidth,
                    edgecolor ='#2E2E2E', label ='RF', hatch='\\\\')
            
            plt.plot(br5, y_axis_baseline, color = 'cyan')
            plt.bar(br5, nb_acc, color =col8, width = barWidth,
                    edgecolor ='#2E2E2E',label ='NB', hatch='....')
            
            plt.plot(br6, y_axis_baseline, color = 'cyan')
            plt.bar(br6, knn_acc, color =col9, width = barWidth,
                    edgecolor ='#2E2E2E', label ='KNN', hatch='+++')
            
            # Adding Xticks
            temp =  ['K-means', 'Agglomerative', 'BIRCH', 'GM']
            plt.xlabel(xlabel)
            plt.ylabel(ylabel)
            plt.xticks([r + barWidth for r in range(len(temp))], ['K-means', 'Agglomerative', 'BIRCH', 'GM']) 
            #plt.legend()
            plt.grid()
            plt.show()
            
            
            ##saving the figure
            dt = datetime.now()
            # getting the timestamp
            ts = datetime.timestamp(dt)
            # convert to datetime
            date_time = datetime.fromtimestamp(ts)
            # convert timestamp to string in dd-mm-yyyy HH:MM:SS
            str_date_time = date_time.strftime("%d-%m-%Y-%H-%M-%S")
            #print(str_date_time)
            
            figure_title = str(title)+str(ylabel)+"_"+str(cluster_num_sfc)+"_"+str(self.episode)+"_"+str(str_date_time)
            fig1.savefig('Generated_plots/classifier/epi_2000_100sfc_agglo_DT/Plot_'+str(figure_title)+'.png')
            fig1.savefig('Generated_plots/classifier/epi_2000_100sfc_agglo_DT/Plot_'+str(figure_title)+'.pdf', bbox_inches='tight')
            
            
            fig2 = plt.figure(figsize=(15,4))
            ylabel = 'Precision'
            # set height of bar
            x_axis = xaxis_clustering_models #xaxis_classifier_models
            ## Set position of bar on X axis
            br1 = np.arange(len(xaxis_clustering_models))
            br2 = [x + barWidth for x in br1]
            br3 = [x + barWidth for x in br2]
            br4 = [x + barWidth for x in br3]
            br5 = [x + barWidth for x in br4]
            br6 = [x + barWidth for x in br5]
            
            # Make the plot
            
            plt.bar(br1, log_pre, color = col4, width = barWidth,
                    edgecolor ='#2E2E2E', label ='LR',  hatch='/////')
            plt.bar(br2, svm_pre, color =col5, width = barWidth,
                    edgecolor ='#2E2E2E', label ='SVM', hatch='----')
            plt.bar(br3, dt_pre,  color =col6, width = barWidth,
                    edgecolor ='#2E2E2E', label ='DT', hatch='xxxx')
            plt.bar(br4, rf_pre, color =col7, width = barWidth,
                    edgecolor ='#2E2E2E', label ='RF', hatch='\\\\')
            plt.bar(br5, nb_pre, color =col8, width = barWidth,
                    edgecolor ='#2E2E2E',label ='NB', hatch='....')
            plt.bar(br6, knn_pre, color =col9, width = barWidth,
                    edgecolor ='#2E2E2E', label ='KNN', hatch='+++')
            
            # Adding Xticks
            temp =  ['K-means', 'Agglomerative', 'BIRCH', 'GM']
            plt.xlabel(xlabel)
            plt.ylabel(ylabel)
            plt.xticks([r + barWidth for r in range(len(temp))], ['K-means', 'Agglomerative', 'BIRCH', 'GM']) 
            plt.legend()
            plt.grid()
            plt.show()
            
            ##saving the figure
            ##saving the figure
            dt = datetime.now()
            # getting the timestamp
            ts = datetime.timestamp(dt)
            # convert to datetime
            date_time = datetime.fromtimestamp(ts)
            # convert timestamp to string in dd-mm-yyyy HH:MM:SS
            str_date_time = date_time.strftime("%d-%m-%Y-%H-%M-%S")
            #print(str_date_time)
            
            figure_title = str(title)+str(ylabel)+"_"+str(cluster_num_sfc)+"_"+str(self.episode)+"_"+str(str_date_time)
            fig2.savefig('Generated_plots/classifier/epi_2000_100sfc_agglo_DT/Plot_'+str(figure_title)+'.png')
            fig2.savefig('Generated_plots/classifier/epi_2000_100sfc_agglo_DT/Plot_'+str(figure_title)+'.pdf', bbox_inches='tight')
            
            
            
            fig3 = plt.figure(figsize=(15,4))
            ylabel = 'Recall'
            # set height of bar
            x_axis = xaxis_clustering_models #xaxis_classifier_models
            
            ## Set position of bar on X axis
            br1 = np.arange(len(xaxis_clustering_models))
            br2 = [x + barWidth for x in br1]
            br3 = [x + barWidth for x in br2]
            br4 = [x + barWidth for x in br3]
            br5 = [x + barWidth for x in br4]
            br6 = [x + barWidth for x in br5]
            
            # Make the plot
            plt.bar(br1, log_recall, color = col4, width = barWidth,
                    edgecolor ='#2E2E2E', label ='LR',  hatch='/////')
            plt.bar(br2, svm_recall, color =col5, width = barWidth,
                    edgecolor ='#2E2E2E', label ='SVM', hatch='----')
            plt.bar(br3, dt_recall,  color =col6, width = barWidth,
                    edgecolor ='#2E2E2E', label ='DT', hatch='xxxx')
            plt.bar(br4, rf_recall, color =col7, width = barWidth,
                    edgecolor ='#2E2E2E', label ='RF', hatch='\\\\')
            plt.bar(br5, nb_recall, color =col8, width = barWidth,
                    edgecolor ='#2E2E2E',label ='NB', hatch='....')
            plt.bar(br6, knn_recall, color =col9, width = barWidth,
                    edgecolor ='#2E2E2E', label ='KNN', hatch='+++')
            
            # Adding Xticks
            temp =  ['K-means', 'Agglomerative', 'BIRCH', 'GM']
            plt.xlabel(xlabel)
            plt.ylabel(ylabel)
            plt.xticks([r + barWidth for r in range(len(temp))], ['K-means', 'Agglomerative', 'BIRCH', 'GM']) 
            plt.legend()
            plt.grid()
            plt.show()
            
            ##saving the figure
            dt = datetime.now()
            # getting the timestamp
            ts = datetime.timestamp(dt)
            # convert to datetime
            date_time = datetime.fromtimestamp(ts)
            # convert timestamp to string in dd-mm-yyyy HH:MM:SS
            str_date_time = date_time.strftime("%d-%m-%Y-%H-%M-%S")
            #print(str_date_time)
            
            figure_title = str(title)+str(ylabel)+"_"+str(cluster_num_sfc)+"_"+str(self.episode)+"_"+str(str_date_time)
            fig3.savefig('Generated_plots/classifier/epi_2000_100sfc_agglo_DT/Plot_'+str(figure_title)+'.png')
            fig3.savefig('Generated_plots/classifier/epi_2000_100sfc_agglo_DT/Plot_'+str(figure_title)+'.pdf', bbox_inches='tight')

            ##### Runtime
            fig4 = plt.figure(figsize=(15,4))
            ylabel = 'Runtime'
            # set height of bar
            x_axis = xaxis_clustering_models #xaxis_classifier_models
            barWidth = 0.15

            ## Set position of bar on X axis
            br1 = np.arange(len(xaxis_clustering_models))
            br2 = [x + barWidth for x in br1]
            br3 = [x + barWidth for x in br2]
            br4 = [x + barWidth for x in br3]
            br5 = [x + barWidth for x in br4]
            br6 = [x + barWidth for x in br5]
            
            # Make the plot
            
            plt.bar(br1, log_runtime, color = col4, width = barWidth,
                    edgecolor ='#2E2E2E', label ='LR',  hatch='/////')
            plt.bar(br2, svm_runtime, color =col5, width = barWidth,
                    edgecolor ='#2E2E2E', label ='SVM', hatch='----')
            plt.bar(br3, dt_runtime,  color =col6, width = barWidth,
                    edgecolor ='#2E2E2E', label ='DT', hatch='xxxx')
            plt.bar(br4, rf_runtime, color =col7, width = barWidth,
                    edgecolor ='#2E2E2E', label ='RF', hatch='\\\\')
            plt.bar(br5, nb_runtime, color =col8, width = barWidth,
                    edgecolor ='#2E2E2E',label ='NB', hatch='....')
            plt.bar(br6, knn_runtime, color =col9, width = barWidth,
                    edgecolor ='#2E2E2E', label ='KNN', hatch='+++')
            
            # Adding Xticks
            temp =  ['K-means', 'Agglomerative', 'BIRCH', 'GM']
            
            plt.xlabel(xlabel)
            plt.ylabel(ylabel)
            plt.xticks([r + barWidth for r in range(len(temp))], ['K-means', 'Agglomerative', 'BIRCH', 'GM']) 
            plt.legend()
            plt.grid()
            plt.show()
            
            ##saving the figure
            dt = datetime.now()
            # getting the timestamp
            ts = datetime.timestamp(dt)
            # convert to datetime
            date_time = datetime.fromtimestamp(ts)
            # convert timestamp to string in dd-mm-yyyy HH:MM:SS
            str_date_time = date_time.strftime("%d-%m-%Y-%H-%M-%S")
            #print(str_date_time)
            
            figure_title = str(title)+str(ylabel)+"_"+str(cluster_num_sfc)+"_"+str(self.episode)+"_"+str(str_date_time)
            fig4.savefig('Generated_plots/classifier/epi_0_100_100sfc/Plot_'+str(figure_title)+'.png')
            fig4.savefig('Generated_plots/classifier/epi_0_100_100sfc/Plot_'+str(figure_title)+'.pdf', bbox_inches='tight')
            
            
            #saving clustering data
            cluster_data = ["Title: " + str(title) +\
                            "Classifier_measurements " + str(self.classifier_measurements) +\
                            "xaxis_clustering_models: " + str(xaxis_clustering_models) +\
                            "xaxis_classifier_models: " + str(xaxis_classifier_models) +\
                            "num_sfc: " + str(cluster_num_sfc)+\
                           "baseline accuracy:"+str(baseline)]
            
            fname = str(title)+"_"+str(cluster_num_sfc)+"_"+str(self.episode)+"_"+str(str_date_time)
            print(fname)
            f = open('Generated_plots/classifier/epi_2000_100sfc_agglo_DT/'+fname,"a")
            f.write(str(cluster_data))
            f.close()
            print("XXXXXXXXXXXXXXXXX")
            

##  Priority Class

In [41]:
# Parameters

arr_core_cpu = [12] #[2, 4, 8, 12]
arr_num_vnf_per_core = [4] #, 32, 64] #

# Nodal Outage
arr_nodal_outage_percentage = [0] #[0, 10, 20, 30, 40, 50]

tot_avail_res_network = (arr_core_cpu[0]*arr_num_vnf_per_core[0])*7
print(tot_avail_res_network)
avail_res_per_node = (arr_core_cpu[0]*arr_num_vnf_per_core[0])
print(avail_res_per_node)

336
48


In [42]:
# Switch 
switch_priorty = True  #False #
switch_reconst = True # False # 
switch_decomp  = True #False # 
pr_model = 'ridge' #'MPL' #'MPL'#'DDPG' #'DDQN'
ar_sch_model = ['DDPG'] #['DDPG', 'WFQ', 'Pr_val_based', 'FIFO'] #'WFQ'#  'Pr_val_based' #'FIFO' #
switch_scheduling = True #False #
switch_sfc_expiring_check = False #True #False
switch_clustering = True
Switch_cluster_info = ['brich']#['kmean', 'Aggo', 'brich', 'GM']
Switch_classifier_info = 'Support Vector Machines' 
## 'Decision Trees', 'Logistic Regression', 'Support Vector Machines', 'Random Forest', 'Naive Bayes', 'K-Nearest Neighbor'
        
cluster_testing = True #False
stop_classifier_training = 1
classifier_training_status = True
classifier_prediction_status = True
length = 1
min_samples = 10
classifier_models ={}
classifier_training_interval = 20

## Clustering Training Model

In [43]:
# Create a different environment
cluster_num_episode, cluster_num_sfc = 1, 50

cluster_global_services =Trail(num_episode= cluster_num_episode, 
                                  num_sfc = cluster_num_sfc, 
                                  max_vnf_resource = max_vnf_resource, 
                                  vnf_per_service = vnf_per_service,
                                  vNetwork_service_er =[], 
                                  vNetwork_service_details_er=[],
                                  vnf_link_resources = [], 
                                  vnf_node_resources=[], 
                                  mean = mean, 
                                  var = var
                                 )

#print(cluster_global_services.vNetwork_service_details_er)


In [44]:
if switch_clustering is True:
    print("switch_clustering: ", switch_clustering)
    n_traffic_transition = [4]
    tr_n_state = [3]
    tr_n_actions = 1

    traffic_mem_size = 500000
    x_traffic_mem = np.zeros((traffic_mem_size, *n_traffic_transition), dtype = np.float32)
    y_traffic_mem = np.zeros((traffic_mem_size), dtype = np.float32)

    hidden_neuron = 20
    tr_input_train_data, tr_target_train_data = [], [] 
    tr_input_test_data,  tr_target_test_data  = [], []
    tr_MLP_model = MLP(tr_n_state = tr_n_state, 
                       tr_n_actions = tr_n_actions, 
                       hidden_neuron = hidden_neuron)

    tr_input_train_mem     = np.zeros((mem_size, *tr_n_state), dtype = np.float)
    tr_target_train_mem    = np.zeros(mem_size, dtype = np.float)
    tr_input_test_mem      = np.zeros((mem_size, *tr_n_state), dtype = np.float)
    tr_target_test_mem     = np.zeros(mem_size, dtype = np.float)
    tr_done_status = "Train"

else:
    pass

switch_clustering:  True


In [45]:
env = substrate_network()

if switch_clustering is True:
    #print("switch_clustering: ", switch_clustering)
    n_traffic_transition = [4]
    traffic_mem_size = 500000
    x_traffic_mem = np.zeros((traffic_mem_size, *n_traffic_transition), dtype = np.float32)
    y_traffic_mem = np.zeros((traffic_mem_size), dtype = np.float32)
else:
    pass


a = 0
input_train_traffic_mem_cnt, test_traffic_mem_cnt = 0, 0
for per_episode in cluster_global_services.vNetwork_service_details_er: 
    episode = a
    #print("")
    #print("Episode: ", episode)
    traffic_input_dataset = []
    epi_classification_grp_acc = {}
    traffic_sfc_arrival = []
    traffic_transition_dict, cluster_label_val = {}, {}
    arr_traffic_packet_per_epi = []
    epi_temp_sfc_arrived = []
    
    kmeans_runtime, Aggo_runtime, birch_runtime, GM_runtime = [],[],[],[]
    
    if switch_clustering is True:
        #traffic
        epi_traffic_packets_dict, epi_traffic_load_dict = {}, {}
        for i in range(cluster_num_sfc):
            epi_traffic_packets_dict[i] = 0
            epi_traffic_load_dict[i] = 0

        #clustering
        epi_clustering_status = {}
        for i in range(len(Switch_cluster_info)):
            #print(i, Switch_cluster_info[i])
            epi_clustering_status[i] = [str(Switch_cluster_info[i]), None]
            
    
    removed_nodes = [0]
    core_cpu = 12
    num_vnf_per_core = 4
    tot_vnf_per_core = core_cpu*num_vnf_per_core
    #----Resource Initialization for the network Network----#
    env.substrate_node_init(removed_nodes, tot_vnf_per_core)
    env.substrate_link_init()
    if switch_priorty is True and pr_model == 'DDPG':
        pr_agent.noise.reset()
    else:
        pass
    
    #---- End of Initialization of Resource ----#
    cnt_sfc= 0
    classifier_measurements= {}
    for per_sfc, per_link in per_episode:
        #print("")
        sfc = per_sfc[0][1]['sfc']
        min_samples = min_samples
        sfc_priority_level = per_sfc[0][1]['env_priority']
        sfc_int_flow = per_sfc[0][1]['int_flow_behaviour']
        arr_traffic_packet_per_sfc = []
        dict_traffic_packet_per_sfc ={}
        
        traffic_N =0 # None
        traffic_load = 0 # None
        temp_s = 0
        temp_pr_ft = [sfc_priority_level, sfc_int_flow, traffic_N, traffic_load]
        traffic_transition_dict[sfc] = temp_pr_ft
        #print("traffic_transition_dict: ", traffic_transition_dict, len(traffic_transition_dict))
        
        # Traffic : update 11jan2023
        test_list = np.arange(len(traffic_transition_dict))
        if sfc >= classifier_training_interval:
            k = rand.choice(test_list)
            while k <= classifier_training_interval-5:
                k = rand.choice(test_list)
        else: 
            k = rand.choice(test_list)
        arrving_sfc = rand.choices(test_list, k=k)
        #print("arrving_sfc: ", arrving_sfc)
        #epi_temp_sfc_arrived.append(arrving_sfc)
        epi_temp_sfc_arrived = arrving_sfc
       # print("epi_temp_sfc_arrived:", epi_temp_sfc_arrived)
       # 
        if switch_clustering is True:
            print("SFC: ", sfc)
            traffic_output_dataset = []
            
            #print(epi_temp_sfc_arrived)
            deployed_sfc_size  = len(epi_temp_sfc_arrived)
           # print("epi_temp_sfc_arrived : ", epi_temp_sfc_arrived)
            #print("deployed_sfc_size: ", deployed_sfc_size)
            sfc_deployment_time =1

            if deployed_sfc_size <= 1:
                #print("deployment is 0")
                pass
            else: 
                mu, std, size = 100, 300, deployed_sfc_size
                traffic_num_sfc_requested = abs(random.normal(mu, std, size))
                #print("===>", traffic_num_sfc_requested)

                traffic_cnt = 0
                for i in traffic_num_sfc_requested:
                    #print(i)
                    length = sfc_deployment_time
                    hurst= 0.7
                    num = abs(int(i))
                    if  num == 0:
                        traffic_packet_per_sfc = 0 #load
                        arr_traffic_packet_per_sfc.append(traffic_packet_per_sfc)
                        pass
                    else: 
                        f = FBM(n  = int(num), hurst=hurst, length=length, method='daviesharte')
                        fbm_sample = abs(f.fbm())
                        fgn_sample = f.fgn()
                        t_values   = f.times() #stationary traffic
                        traffic_packet_per_sfc = sum(t_values) #load
                        arr_traffic_packet_per_sfc.append(traffic_packet_per_sfc)

                    
                    deployed_sfc = epi_temp_sfc_arrived[traffic_cnt]
                    traffic_transition_dict[epi_temp_sfc_arrived[traffic_cnt]][2] = traffic_num_sfc_requested[traffic_cnt]
                    traffic_transition_dict[epi_temp_sfc_arrived[traffic_cnt]][3] = arr_traffic_packet_per_sfc[traffic_cnt]
                    epi_traffic_packets_dict[epi_temp_sfc_arrived[traffic_cnt]] = epi_traffic_packets_dict[epi_temp_sfc_arrived[traffic_cnt]] +  traffic_num_sfc_requested[traffic_cnt]
                    epi_traffic_load_dict[epi_temp_sfc_arrived[traffic_cnt]] = epi_traffic_load_dict[epi_temp_sfc_arrived[traffic_cnt]] + arr_traffic_packet_per_sfc[traffic_cnt]
                    
                    #print("epi_traffic_packets_dict:", epi_traffic_packets_dict)
                    #print("epi_traffic_packets_dict[epi_temp_sfc_arrived[traffic_cnt]]:", epi_traffic_packets_dict[epi_temp_sfc_arrived[traffic_cnt]])
                    
                    #print("deployed_sfc", deployed_sfc, traffic_num_sfc_requested[traffic_cnt], arr_traffic_packet_per_sfc[traffic_cnt])
                    
                    #updating in global level
                    cluster_global_services.traffic_update(episode, deployed_sfc, vnf_per_service, 
                                                   epi_traffic_packets_dict, epi_traffic_load_dict)
                    traffic_cnt += 1
                    
                    
                    arr_traffic_packet_per_epi.append(arr_traffic_packet_per_sfc) #traffic_clustering
                    
                    
                    class_traffic_clustering = traffic_clustering(traffic_num_sfc_requested,
                                                                  epi_temp_sfc_arrived, 
                                                                  deployed_sfc_size,
                                                                  sfc_deployment_time, 
                                                                  arr_traffic_packet_per_sfc,
                                                                  traffic_transition_dict,
                                                                  traffic_batch_size,
                                                                  episode)
                    input_traffic_storage = class_traffic_clustering.input_traffic_transistion_storage(x_traffic_mem, 
                                                                                           traffic_mem_size, 
                                                                                           input_train_traffic_mem_cnt)

                    input_train_traffic_mem_cnt = input_traffic_storage[0]
                    traffic_input_dataset = input_traffic_storage[1]
                    #print(traffic_input_dataset)
                    #kmeans_runtime, Aggo_runtime, birch_runtime, GM_runtime
                    #print("input_train_traffic_mem_cnt: ", input_train_traffic_mem_cnt)
                    
                    if input_train_traffic_mem_cnt > 10:
                        #pass
                        #kmean_cluster = class_traffic_clustering.kmean(n_clusters, traffic_input_dataset, kmeans_runtime)
                        #print("kmean_cluster: ", kmean_cluster)
                        #epi_clustering_status[0][1] = kmean_cluster

                        #hierarchical_cluster = class_traffic_clustering.hierarchical(n_clusters, traffic_input_dataset, 
                        #                                                             Aggo_runtime)
                        #print("hierarchical_cluster: ", hierarchical_cluster)
                        
                        
                        birch_cluster = class_traffic_clustering.brc(n_clusters, traffic_input_dataset, birch_runtime)
                        epi_clustering_status[0][1] = birch_cluster

                        #birch_cluster = class_traffic_clustering.brc(n_clusters, traffic_input_dataset, birch_runtime)
                        #print("birch_cluster: ", birch_cluster)
                        #epi_clustering_status[2][1] = birch_cluster

                        #spectral_cluster = class_traffic_clustering.spectral(n_clusters, traffic_input_dataset)
                        ##print("spectral_cluster: ", spectral_cluster)
                        #epi_clustering_status[3][1] = spectral_cluster 

                        #GM_cluster = class_traffic_clustering.GM(n_clusters, traffic_input_dataset, GM_runtime)
                        #print("GM_cluster: ", GM_cluster)
                        #epi_clustering_status[3][1] = GM_cluster

                    else:
                        pass
                    
            #print("epi_clustering_status: ", epi_clustering_status)    
            if sfc > classifier_training_interval and sfc%classifier_training_interval == 0:#sfc > classifier_training_interval: # input_train_traffic_mem_cnt > 10:
                for key, val in epi_clustering_status.items():
                    #print("key: ", val[0])
                    clusterModel = val[0]
                    train_traffic_classifier_input_dataset, train_traffic_classifier_output_dataset = [], []
                    test_traffic_classifier_input_dataset, test_traffic_classifier_output_dataset = [], []
                    traffic_output_dataset = val[1]
                    
                    class_traffic_classifer = traffic_classifer(traffic_input_dataset, 
                                                            traffic_output_dataset, 
                                                            episode, cluster_testing)
                    
                    
                    classifier_datasets = class_traffic_classifer.classifier_dataset(train_traffic_classifier_input_dataset, 
                                                                                 train_traffic_classifier_output_dataset,
                                                                                 test_traffic_classifier_input_dataset,  
                                                                                 test_traffic_classifier_output_dataset)
                    
                    temp_val1 = classifier_datasets[1].count(1)/len(classifier_datasets[1])
                    temp_val2 = classifier_datasets[1].count(0)/len(classifier_datasets[1])
                    baseline = (temp_val1* temp_val1) + (temp_val2*temp_val2)
                    
                    predict_dataset = [] #[sfc, sfc_priority_level, sfc_int_flow, pred_num, pred_traffic_packet_per_sfc] #temp_pr_ft #
                    #print("classifier_datasets: ", classifier_datasets)
                    #print("0 classifier_datasets: ", classifier_datasets[0])
                    #print("1 classifier_datasets: ", classifier_datasets[1])
                    #print("2 classifier_datasets: ", classifier_datasets[2])
                    #print("3 classifier_datasets: ", classifier_datasets[3])
                    Classifier_training = class_traffic_classifer.classifier_models(classifier_models, 
                                                                                    classifier_datasets,
                                                                                    predict_dataset,
                                                                                    classifier_training_status,
                                                                                    Switch_classifier_info)

                    #print("Classifier_training: ", Classifier_training)
                    classifier_measurements[clusterModel] = Classifier_training
                    #print("classifier_measurements: ", classifier_measurements)
                    
                    
            else:
                pass
    
        cnt_sfc += 1
    #print("epi_clustering_status: ", epi_clustering_status)
    #print("len traffic_input_dataset: ", len(traffic_input_dataset))
    if a%cluster_num_episode==0:
        pass
        '''
        clusterplot_val = class_traffic_clustering.clusteringplot(epi_clustering_status, 
                                                                  traffic_input_dataset, 
                                                                  cluster_label_val,
                                                                  epi_temp_sfc_arrived)
        
        #print("classifier_measurements: ", classifier_measurements)
    
        classifers_plots = class_traffic_classifer.classifier_plot(classifier_measurements, cluster_num_sfc, baseline)

        title = 'Runtime for cluster model'
        #kmeans_runtime, Aggo_runtime, birch_runtime, GM_runtime
        Y_axis_Rcluster = [sum(kmeans_runtime), sum(Aggo_runtime), sum(birch_runtime), sum(GM_runtime)]
        #print("X_axis_Rcluster: ", X_axis_Rcluster)
        X_axis_Rcluster = ['K-means', 'Agglomerative', 'BIRCH', 'GM']
        fig = plt.figure()
        plt.barh(X_axis_Rcluster, Y_axis_Rcluster, color = '#27408B', edgecolor ='#2E2E2E',hatch = '///') 
        plt.ylabel("Cluster Modelling")
        plt.xlabel("Runtime (ns)")
        plt.grid()
        plt.show()

        ##saving the figure
        dt = datetime.now()
        # getting the timestamp
        ts = datetime.timestamp(dt)
        # convert to datetime
        date_time = datetime.fromtimestamp(ts)
        # convert timestamp to string in dd-mm-yyyy HH:MM:SS
        str_date_time = date_time.strftime("%d-%m-%Y-%H-%M-%S")
        #print(str_date_time)
        
        figure_title = str(title)+"_"+str(cluster_num_sfc)+"_"+str(episode)+str(str_date_time)
        fig.savefig('Generated_plots/cluster/epi_2000_100sfc_agglo_DT/Plot_'+str(figure_title)+'.png')
        fig.savefig('Generated_plots/cluster/epi_2000_100sfc_agglo_DT/Plot_'+str(figure_title)+'.pdf', bbox_inches='tight')


        #saving clustering data
        cluster_data = ["Title: " + str(figure_title ) +\
                        "X_axis_Rcluster: " + str(X_axis_Rcluster) +\
                        "Y_axis_Rcluster: " + str(Y_axis_Rcluster)]

        fname = figure_title
        print(fname)
        f = open('Generated_plots/cluster/epi_2000_100sfc_agglo_DT/'+fname,"a")
        f.write(str(cluster_data))
        f.close()
        print("XXXXXXXXXXXXXXXXX") 
        '''
    
    a += 1

SFC:  0
SFC:  1
SFC:  2
SFC:  3
SFC:  4
SFC:  5
SFC:  6
SFC:  7
SFC:  8
SFC:  9
SFC:  10
SFC:  11
SFC:  12
SFC:  13
SFC:  14
SFC:  15
SFC:  16
SFC:  17
SFC:  18
SFC:  19
SFC:  20
SFC:  21
SFC:  22
SFC:  23
SFC:  24
SFC:  25
SFC:  26
SFC:  27
SFC:  28
SFC:  29
SFC:  30
SFC:  31
SFC:  32
SFC:  33
SFC:  34
SFC:  35
SFC:  36
SFC:  37
SFC:  38
SFC:  39
SFC:  40
SFC:  41
SFC:  42
SFC:  43
SFC:  44
SFC:  45
SFC:  46
SFC:  47
SFC:  48
SFC:  49


<Figure size 432x288 with 0 Axes>

## Binary Classifer

In [46]:
##print(("epi_clustering_status: ", epi_clustering_status))
##print(traffic_input_dataset, len(traffic_input_dataset))

## Alggo + DT

In [49]:
#cluster_mod, classifier_mod  = 
#'brich', 'Support Vector Machines'

classifier_training_status = False
classifier_prediction_status = True

if __name__ == '__main__': 
    sch_model_overall_cnt_sfc_starved_before_serving, sch_model_overall_cnt_sfc_Prclass_starved_before_serving = {},{}
    overall_sfc_arrival_detail,  overall_sfc_arrival_num = [],[]
    overall_sfc_priority, copy_overall_epi_pr, overall_episode_envpriority  = [], [],[]
    copy_overall_episode_envpriority = []
    #sclassifier_models ={}
    
    #saving the arrival services
    saving_services = global_services.vNetwork_service_details_er
    #print("saving_services: ", saving_services)
    f_name = str(trial)+'_'+ str('_Netrail_')+\
            '_cpucore_' + str(arr_core_cpu[0])+'_num_vnf_per_core_'+\
            str(arr_num_vnf_per_core[0]) + str('_saving_services')
    print(f_name)
    #f = open('Data_storage/epi_2000_100sfc_agglo_DT/'+f_name,"a")
    #f.write(str(saving_services))
    #f.close()
    print("XXXXXXXXXXXXXXXXX")
        
    status_cnt_ridge = False
    for sch_model in ar_sch_model:
        #print("______________________")
        #print("sch_model: ", sch_model, status_cnt_ridge)
        sch_model_overall_cnt_sfc_starved_before_serving[sch_model] = None
        sch_model_overall_cnt_sfc_Prclass_starved_before_serving [sch_model] = None
        env = substrate_network()
        reward_func = Reward_Function()
        temp_vals_sorted_action, temp_norm_vals_sorted_action = [], []
        min_pr,max_pr = 0, 6
        t_max, t_min, old_t_current = None, 0, 0
        mvf_t_max, mvf_t_min, mvf_old_t_current = None, 0, 0
        switch_or_run_once = False
        overall_emb_sfc, overall_emb_sfc_h_rel = [], []
        overall_unemb_sfc = []
        overall_ar_sorted_sch_list =[]

        # NODAL OUTAGE
        for nodal_outage_percentage in arr_nodal_outage_percentage:
            nodal_outage = (env.network_len_nodes*nodal_outage_percentage)/100
            round_nodal_outage = round(nodal_outage)

            # CPU CORE
            for core_cpu in arr_core_cpu:

                #VNF PER CPU CORE
                for num_vnf_per_core in arr_num_vnf_per_core:
                    tot_vnf_per_core = core_cpu*num_vnf_per_core
                    tot_vnf_network = tot_vnf_per_core*env.network_len_nodes
                    pr_epsilon, epsilon = 1, 1
                    eps_history,network_overall_accepted_action = [],[]
                    network_overall_accepted, network_overall_rejected = [],[]
                    monolithic_deploy_success, monolithic_deploy_rejected = [], []
                    overall_res_exhausted = []
                    network_overall_count_mvnf, network_overall_sum_mvnf = [],[]
                    network_overall_tot_decompsfc, network_overall_count_succes_decompvnf,  = [], []
                    network_overall_count_unsucces_decompvnf, overall_remain_cpu, overall_remain_bw = [], [], []
                    overall_count_numVNF_perNode, overall_count_num_mVNF_perNode = [], []
                    network_overall_epsilon, network_overall_sum_rewards_perepi = [], []
                    overall_episode_priority,  overall_episode_priority_err= [], []
                    overall_episode_pr_rewardfunc = []
                    over_count_macro_pr, over_count_micro_pr = [], []
                    overall_epi_pr,overall_epi_pr_emb = [],[]
                    overall_epi_traffic_packets_dict, overall_epi_traffic_load_dict = {}, {}
                    start = time.time()
                    print("")
                    print("Start Time:",start)
                    #overall runtime these needs to be initiated
                    n_state = [8]
                    state_memory      = np.zeros((mem_size, *n_state), dtype = np.float32) # size of mem size X input shape
                    action_memory     = np.zeros(mem_size, dtype = np.int64)
                    reward_memory     = np.zeros(mem_size, dtype = np.float32)
                    next_state_memory = np.zeros((mem_size, *n_state), dtype = np.float32)
                    terminal_memory   = np.zeros(mem_size, dtype = np.bool)
                    vnf_cnt = 0
                    temp_vals_sorted_action, temp_norm_vals_sorted_action = [], [] 
                    if model == 'DDQN':
                        print("activate agent DDQN")
                        #agent
                        agent = LinearDDQNAgent(n_state = n_state, 
                                                n_actions = env.network_len_nodes, 
                                                lr = learning_rate, 
                                                epsilon = epsilon,
                                                mem_size = mem_size,
                                                batch_size= batch_size,
                                                vnf_cnt = vnf_cnt,
                                                gamma =gamma,
                                                eps_dec =eps_dec,
                                                eps_min =eps_min,
                                                replace = replace,
                                                model = model,
                                                state_memory = state_memory,
                                                action_memory = action_memory,
                                                reward_memory = reward_memory,
                                                next_state_memory = next_state_memory,
                                                terminal_memory = terminal_memory,
                                                Neurons_per_HiddenLayer = Neurons_per_HiddenLayer,
                                                drop_prob =drop_prob,
                                                chkpt_dir = 'models/',
                                                algo = 'DDQNAgent',
                                                env_name = 'BtEurope')
                    elif model == 'DQN':
                        agent = LinearDQNAgent(n_state = n_state, 
                                               n_actions = env.network_len_nodes, 
                                               epsilon = epsilon, 
                                               lr = learning_rate, 
                                               vnf_cnt = vnf_cnt, 
                                               Neurons_per_HiddenLayer = Neurons_per_HiddenLayer,
                                               drop_prob =drop_prob,
                                               gamma =gamma,
                                               eps_dec =eps_dec,
                                               eps_min =eps_min, 
                                               model = model,
                                               algo = None, 
                                               env_name = None,
                                               chkpt_dir ='tmp/dqn')
                    #added on 01032022 Heuristic model
                    else:
                        print("Activate Heuristic model")
                        agent = HeuAgent(n_state = n_state, 
                                         n_actions = env.network_len_nodes)  
                        
                    cnt_ridge = 0
                    # Priority
                    if switch_priorty is True:
                        pr_mem_size = 5000
                        pr_n_state = [3]  #  waiting time, delay, jitter, Packetloss,
                        pr_state_input = 'waiting time, delay, jitter, Packetloss'
                        # [episode, sfc, flow behaviour, delay, jitter, Packetloss, reliability(removed), wating time]
                        pr_n_actions = 1
                        pr_state_memory      = np.zeros((pr_mem_size, *pr_n_state), dtype = np.float32) # size of mem size X input shape
                        pr_reward_memory     = np.zeros(pr_mem_size, dtype = np.float32)
                        pr_next_state_memory = np.zeros((pr_mem_size, *pr_n_state), dtype = np.float32)
                        pr_terminal_memory   = np.zeros(pr_mem_size, dtype = np.bool)
                        sfc_cnt = 0
                        if pr_model == 'DDPG':
                            print("pr_model: ",pr_model)
                            pr_action_memory = np.zeros((pr_mem_size, pr_n_actions), dtype = np.float32)
                            pr_agent = DDPG_Agent(pr_n_state       = pr_n_state, 
                                                  pr_n_actions     = pr_n_actions,
                                                  alpha            = alpha, 
                                                  beta             = beta,
                                                  tau              = tau, 
                                                  sfc_cnt          = sfc_cnt,
                                                  mem_size         = mem_size,
                                                  batch_size       = batch_size,
                                                  gamma            = gamma,
                                                  pr_model         = pr_model,
                                                  pr_state_memory  = pr_state_memory,
                                                  pr_action_memory = pr_action_memory,
                                                  pr_reward_memory = pr_reward_memory,
                                                  pr_next_state_memory = pr_next_state_memory,
                                                  pr_terminal_memory = pr_terminal_memory,
                                                  Neurons_per_HiddenLayer = Neurons_per_HiddenLayer,
                                                  chkpt_dir = 'models/',
                                                  algo = 'DDPGAgent',
                                                  env_name = 'Netrail')
                        elif pr_model == 'DDQN':
                            print(pr_model)
                            pr_action_memory = np.zeros((pr_mem_size), dtype = np.float32)
                            pr_agent = PR_DDQNAgent(pr_n_state       = pr_n_state,
                                                 pr_n_actions     = pr_n_actions,
                                                 lr               = learning_rate, 
                                                 pr_epsilon       = pr_epsilon,
                                                 mem_size         = mem_size,
                                                 batch_size       = batch_size,
                                                 sfc_cnt          = sfc_cnt,
                                                 gamma            = gamma,
                                                 eps_dec          = eps_dec,
                                                 eps_min          = eps_min,
                                                 replace          = replace,
                                                 pr_model         = pr_model,
                                                 pr_state_memory  = pr_state_memory,
                                                 pr_action_memory = pr_action_memory,
                                                 pr_reward_memory = pr_reward_memory,
                                                 pr_next_state_memory = pr_next_state_memory,
                                                 pr_terminal_memory = pr_terminal_memory,
                                                 Neurons_per_HiddenLayer = Neurons_per_HiddenLayer,
                                                 drop_prob =drop_prob,
                                                 chkpt_dir = 'models/',
                                                 algo = 'PR_DDQNAgent',
                                                 env_name = 'Netrail')    
                        elif pr_model == 'ridge':
                            print(pr_model)
                            #Lasso model
                            alpha = 0.0001
                            input_train_data,target_train_data = [], [] 
                            input_test_data, target_test_data  = [], []
                            input_train_mem     = np.zeros((mem_size, *pr_n_state), dtype = np.float32)
                            target_train_mem    = np.zeros(mem_size, dtype = np.float32)
                            input_test_mem      = np.zeros((mem_size, *pr_n_state), dtype = np.float32)
                            target_test_mem     = np.zeros(mem_size, dtype = np.float32)
                            # define model
                            ridge_model = Ridge(alpha=alpha, normalize = True) #Lasso(alpha=alpha, normalize = True)
                            cv = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeat, random_state=1)
                            done_status = "Train"
                        else: 
                            #print("here")
                            print(pr_model)
                            hidden_neuron = 20
                            input_train_data,target_train_data = [], [] 
                            input_test_data, target_test_data  = [], []
                            pr_MLP_model = MLP(pr_n_state = pr_n_state, 
                                               pr_n_actions = pr_n_actions, 
                                               hidden_neuron = hidden_neuron)
                            input_train_mem     = np.zeros((mem_size, *pr_n_state), dtype = np.float)
                            target_train_mem    = np.zeros(mem_size, dtype = np.float)
                            input_test_mem      = np.zeros((mem_size, *pr_n_state), dtype = np.float)
                            target_test_mem     = np.zeros(mem_size, dtype = np.float)
                            done_status = "Train"
                            
                    
                    # Admission control: Scheduling 
                    if switch_scheduling is True:
                        sch_mem_size = 5000
                        sch_n_state = [4]  #  waiting time, delay, jitter, Packetloss,
                        sch_state_input = 'waiting time, reliability, priority'
                        # [episode, sfc, flow behaviour, delay, jitter, Packetloss, reliability(removed), wating time]
                        sch_n_actions = 1
                        sch_n_tempsfc = [10]
                        sch_state_memory      = np.zeros((sch_mem_size,*sch_n_state), dtype = np.float32) # size of mem size X input shape
                        #print(sch_state_memory)
                        sch_reward_memory     = np.zeros(sch_mem_size, dtype = np.float32)
                        sch_next_state_memory = np.zeros((sch_mem_size, *sch_n_state), dtype = np.float32)
                        sch_terminal_memory   = np.zeros(sch_mem_size, dtype = np.bool)
                        sfc_cnt = 0
                        if sch_model == 'DDPG':
                            #print("sch_model", sch_model)
                            sch_action_memory     = np.zeros((sch_mem_size, sch_n_actions), dtype = np.float32)
                            sch_agent = DDPG_Agent(sch_n_state      = sch_n_state, 
                                                   sch_n_actions    = sch_n_actions,
                                                   alpha            = alpha, 
                                                   beta             = beta,
                                                   tau              = tau, 
                                                   sfc_cnt          = sfc_cnt,
                                                   mem_size         = mem_size,
                                                   batch_size       = batch_size,
                                                   gamma            = gamma,
                                                   sch_model        = sch_model,
                                                   sch_state_memory  = sch_state_memory,
                                                   sch_action_memory = sch_action_memory,
                                                   sch_reward_memory = sch_reward_memory,
                                                   sch_next_state_memory = sch_next_state_memory,
                                                   sch_terminal_memory = sch_terminal_memory,
                                                   Neurons_per_HiddenLayer = Neurons_per_HiddenLayer,
                                                   chkpt_dir = 'models/',
                                                   algo = 'DDPGAgent',
                                                   env_name = 'Netrail')
                        else:
                            pass

                    a = 0
                    cnt_tot_sfc , tr_cnt_tot_sfc, eval_cnt_tot_sfc, pred_cnt_tot_sfc= 0, 0, 0, 0
                    count_macro_pr, count_micro_pr = 0, 0
                    tr_count_macro_pr, tr_count_micro_pr = 0, 0
                    eval_count_macro_pr, eval_count_micro_pr = 0, 0
                    pred_count_macro_pr, pred_count_micro_pr = 0, 0
                    tr_mem_cnt, test_mem_cnt = 0, 0
                    input_train_traffic_mem_cnt, test_traffic_mem_cnt = 0, 0
                    epi_tr_correct_perc,  epi_pred_correct_perc = [], []
                    overall_deployed_sfc_per_episode, overall_deployed_priority_per_episode = [], []
                    epi_cnt_High_pr, epi_cnt_Medium_pr, epi_cnt_Low_pr = [], [], []
                    overall_epi_cnt_High_pr_emb, overall_epi_cnt_Medium_pr_emb, overall_epi_cnt_Low_pr_emb = [], [], []
                    overall_epi_cnt_High_traffic0_emb, overall_epi_cnt_Medium_traffic0_emb, overall_epi_cnt_Low_traffic0_emb = [], [], []
                    overall_epi_cnt_High_traffic1_emb, overall_epi_cnt_Medium_traffic1_emb, overall_epi_cnt_Low_traffic1_emb = [], [], []
                    overall_epi_cnt_low_pr_traffic_1 = []
                    overall_cnt_sfc_starved_before_serving, overall_cnt_sfc_Prclass_starved_before_serving = [], []
                    epi_cnt_sfc_starved_before_serving, epi_cnt_sfc_Prclass_starved_before_serving = [], []
                    epi_expected_wt_B = {}
                    overall_epi_traffic_class_gen ={}
                    overall_epi_predicted_traffic_classes_dict = {}
                    overall_epi_sfc_pr_traffic_class_emb, overall_epi_sfc_pr_traffic_class_all = {}, {}
                        

                    for per_episode in global_services.vNetwork_service_details_er: 
                        episode = a
                        print("")
                        print("Episode: ", episode)
                        action, vnf_res_req = False, False
                        pref_node_val = False
                        cnt_macro_pr, epi_cnt_macro_pr = 0,0 
                        cnt_High_pr, cnt_Medium_pr, cnt_Low_pr = 0, 0, 0
                        cnt_High_pr_emb, cnt_Medium_pr_emb, cnt_Low_pr_emb = 0, 0, 0
                        cnt_High_traffic0_emb, cnt_Medium_traffic0_emb, cnt_Low_traffic0_emb = 0, 0, 0
                        cnt_High_traffic1_emb, cnt_Medium_traffic1_emb, cnt_Low_traffic1_emb = 0, 0, 0
                        epi_pr = {"H": None,"M": None, "L": None}
                        epi_pr_emb = {"H": None,"M": None, "L": None}
                        cnt_sfc_starved_before_serving = 0
                        cnt_rel = 0
                        epi_cnt_low_pr_traffic_1 = 0
                        epi_classification_grp_acc = {}
                        kmeans_runtime, Aggo_runtime, birch_runtime, GM_runtime = [],[],[],[]
                        traffic_input_dataset = []
                        sfc_starved_before_serving, sfc_Prclass_starved_before_serving = [], []
                        nodal_actionspace, removed_nodes = [], []
                        epi_emb_sfc, epi_emb_sfc_h_rel = [], []
                        traffic_sfc_arrival = []
                        traffic_transition_dict, cluster_label_val = {}, {}
                        arr_traffic_packet_per_epi = []
                        epi_unemb_sfc = []
                        epi_temp_sfc_arrived = []
                        epi_traffic_class_gen = {}
                        epi_predicted_traffic_classes_arr = []
                        epi_predicted_traffic_classes_dict = {}
                        epi_sfc_pr_traffic_class_emb, epi_sfc_pr_traffic_class_all = {}, {}
                        
                        episode_priority, episode_priority_err, episode_envpriority, episode_pr_rewardfunc = [], [], [], []
                        actionspace = [i for i in range(env.network_len_nodes)]
                        original_actionspace = actionspace
                        # to remove certain number of the nodes from the network when outage happens
                        for key in range(round_nodal_outage):
                            random_action = random.choice(actionspace)
                            actionspace.remove(random_action)
                            removed_nodes.append(random_action)                 
                        #----Resource Initialization for the network Network----#
                        env.substrate_node_init(removed_nodes, tot_vnf_per_core)
                        env.substrate_link_init()
                        if switch_priorty is True and pr_model == 'DDPG':
                            pr_agent.noise.reset()
                        else:
                            pass
                        #---- End of Initialization of Resource ----#
                        count_accpeted, count_rejected, count_success,count_failed  = 0,0,0,0
                        reward_per_episode, sum_reward_per_episode = [], []
                        deployed_sfc_per_episode, deployed_priority_per_episode = [], []
                        micro_vnf_collection = []
                        tot_decompsfc = 0
                        undecomp_accepted = 0
                        res_exhausted = 0
                        count_mvnf_perepi =[]
                        sum_count_mvnf_perepi =0
                        sum_rewards_perepi = 0
                        count_succes_decompvnf, count_unsucces_decompvnf = 0, 0
                        nodal_avail = 1 #at the start of the episode the network will be fully available.
                        rewards_perepi = []
                        std_value = 1
                        
                        # Priority!
                        epi_tr_count_macro_pr,   epi_tr_count_micro_pr   = 0, 0
                        epi_eval_count_macro_pr, epi_eval_count_micro_pr = 0, 0
                        epi_pred_count_macro_pr, epi_pred_count_micro_pr = 0, 0
                        
                        # arrival distriution of SFC using Poisson dis.
                        if len(overall_sfc_arrival_num) != len(global_services.vNetwork_service_details_er):
                            arr_sfc_dist = []
                            class_arr_sfc_dist = arrival_sfc(num_sfc, arr_sfc_dist)
                            arr_sfc_dist = list(class_arr_sfc_dist.sfc_poisson_dist())
                            #print("==> SFC dist: ",arr_sfc_dist)
                            sfc_arrival_detail, sfc_arrival_num = [], []
                            arr_sfc_detail = class_arr_sfc_dist.sfc_details(per_episode, 
                                                                            arr_sfc_dist, 
                                                                            sfc_arrival_detail, 
                                                                            sfc_arrival_num)
                            overall_sfc_arrival_detail.append(sfc_arrival_detail)
                            overall_sfc_arrival_num.append(sfc_arrival_num)
                            print("sfc_arrival_num: ", sfc_arrival_num)

                        else: 
                            pass
                        #print("Length of overall_sfc_arrival_num: ",len(overall_sfc_arrival_num))
                        #print("overall_sfc_arrival_num: ", overall_sfc_arrival_num)
                        sfc_arrival_detail = overall_sfc_arrival_detail[episode]
                        sfc_arrival_num = overall_sfc_arrival_num[episode]
                        sfc_priority, sfc_priority_err, sfc_envpriority, sfc_pr_rewardfunc = [], [], [], []
                        done_pr   = False
                        ar_sfc_current_waitingtime = {}
                        ar_sorted_sch_list =[]
                        cnt_poisson_dist = 0
                        for arr_sfc_de in sfc_arrival_detail:
                            ar_sfc_start = time.perf_counter() # step 1
                            cnt_sfc_arrival = 0
                            temp_my_dict = {}
                            ar_starving_sfc = []
                            #print("")
                            #print("arr_sfc_de:", arr_sfc_de)
                            #print("cnt_poisson_dist: ", cnt_poisson_dist, " arr num: ", sfc_arrival_num[cnt_poisson_dist])
                            if sch_model == 'WFQ':
                                h_pr_queue, m_pr_queue, l_pr_queue = [], [], []
                            if not sfc_arrival_num[cnt_poisson_dist]:
                                sorted_sch_list =[]
                            ar_sfc_end = time.perf_counter() #step 1
                            expected_wt_A = round(ar_sfc_end - ar_sfc_start,6)

                            for arv_per_sfc, arv_per_link in arr_sfc_de:
                                sfc = arv_per_sfc[0][1]['sfc']
                                #print("arv_per_sfc: ", arv_per_sfc)
                                print("SFC: ", sfc)
                                ar_link_bw = []
                                for link in arv_per_link:
                                    temp_bw = link[2]['bw']
                                    ar_link_bw.append(temp_bw)
                                ar_sfc_cpu = []
                                for temp_sfc in arv_per_sfc:
                                    temp_cpu = temp_sfc[1]['cpu_req']
                                    ar_sfc_cpu.append(temp_cpu)     
                                # Priority for Arriving SFC
                                env_priority = None
                                env_priority = arv_per_sfc[0][1]['env_priority']
                                env_priority = env_priority - env_priority % 0.01
                                done_sch = False
                                A_cost, B_cost, S_cost = 0,0,0
                                reward_pts = 1000
                                #print("SFC: ", sfc)
                                if status_cnt_ridge is False:
                                    print("pr_model: ", pr_model)
                                    if switch_priorty is True:
                                            pr_sfc_start = time.perf_counter()
                                            threshold_latency          = arv_per_sfc[0][1]['norm_delay'] 
                                            threshold_jitter           = arv_per_sfc[0][1]['norm_jitter']     
                                            threshold_packetloss       = arv_per_sfc[0][1]['norm_packetloss'] 
                                            threshold_theo_wating_time = arv_per_sfc[0][1]['norm_waiting_time'] 
                                            
                                            # creating priority observation
                                            pr_obs = [threshold_latency, threshold_jitter, threshold_packetloss]
                                            if pr_model == 'ridge':
                                                X = pr_obs
                                                Y = [env_priority]
                                                sfc_envpriority.append(env_priority)
                                                env_pr_level = truncate(Y[0],2)
                                                if env_pr_level == 1:
                                                    macro_env_pr_level = int(truncate(env_pr_level, 0))
                                                    micro_env_pr_level = round((truncate(env_pr_level, 1)- macro_env_pr_level),2)
                                                else:
                                                    macro_env_pr_level = int(truncate(env_pr_level*10, 0))
                                                    micro_env_pr_level = int(truncate(env_pr_level*100, 1))- macro_env_pr_level*10
                                                if done_status == "Train": 
                                                    tr_cnt_tot_sfc += 1
                                                    train_dataset = prepare_train_data(input_train_mem, target_train_mem, 
                                                                                       input_train_data, target_train_data,
                                                                                       X, Y, 
                                                                                       tr_mem_cnt, tr_mem_size, tr_batch_size)
                                                    input_train_data = train_dataset[0]
                                                    target_train_data = train_dataset[1]
                                                    tr_mem_cnt = train_dataset[2]
                                                    #print("train_dataset: ", train_dataset)
                                                    if tr_cnt_tot_sfc < tr_batch_size:
                                                        temp_priority_level = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
                                                        priority_level  = random.uniform(0,1) #np.random.choice(temp_priority_level)
                                                        #print("priority_level: ", priority_level, type(priority_level))
                                                    else:
                                                        # evaluate model
                                                        scores = cross_val_score(ridge_model, input_train_data, target_train_data, 
                                                                                 scoring='neg_mean_absolute_error', 
                                                                                 cv=cv, n_jobs= 1)
                                                        # force scores to be positive
                                                        scores = absolute(scores)
                                                        # fit model
                                                        ridge_model.fit(input_train_data, target_train_data)
                                                        # make a prediction
                                                        yhat = ridge_model.predict([X])
                                                        # summarize prediction
                                                        priority_level = yhat[0]
                                                        #print("priority_level: ", priority_level, type(priority_level))

                                                    #classifing the priority levels into three classess.
                                                    if 0.8<=priority_level and priority_level<=1.0:
                                                        priority_class = 'high'
                                                        cnt_High_pr +=1
                                                    elif 0.4<=priority_level and priority_level<=0.7:
                                                        priority_class = 'medium'
                                                        cnt_Medium_pr +=1
                                                    else: 
                                                        priority_class = 'low'   
                                                        cnt_Low_pr +=1
                                                    sfc_priority.append(priority_level)
                                                    global_services.priority_update(episode, sfc, 
                                                                                    vnf_per_service, 
                                                                                    priority_level, 
                                                                                    priority_class)
                                                    if priority_level == 1:
                                                        macro_priority_level = 1 #int(truncate(priority_level, 0))
                                                        micro_priority_level = 0 #round((truncate(priority_level, 1)- macro_priority_level),2)
                                                    else:
                                                        macro_priority_level = int(truncate(priority_level*10, 0))
                                                        micro_priority_level = int(truncate(priority_level*100, 1))- macro_priority_level*10
                                                    if macro_priority_level == macro_env_pr_level:
                                                        #first level clearance
                                                        tr_count_macro_pr += 1
                                                        epi_tr_count_macro_pr += 1
                                                        if micro_priority_level == micro_env_pr_level:
                                                            tr_count_micro_pr += 1
                                                            epi_tr_count_micro_pr += 1
                                                    tr_macro_acc = round(((tr_count_macro_pr/tr_cnt_tot_sfc)*100), 2)
                                                    tr_micro_acc = round(((tr_count_micro_pr/tr_cnt_tot_sfc)*100), 2)
                                                    if a == 0:
                                                        pass
                                                    else:
                                                        if (tr_macro_acc/100) >= acc_theshold_limit:
                                                            #done_status == "Train"
                                                            done_status = "Predict"
                                                            #print("Train --> Predict", done_status, cnt_tot_sfc, tr_cnt_tot_sfc,
                                                            #      " macro acc: ", tr_macro_acc, " micro acc: ",tr_micro_acc, tr_count_micro_pr)
                                                else:
                                                    pred_cnt_tot_sfc +=1
                                                    # make a prediction
                                                    yhat = ridge_model.predict([X])
                                                    # summarize prediction
                                                    priority_level = yhat[0]
                                                    #print("priority_level: ", priority_level, type(priority_level))
                                                    #classifing the priority levels into three classess.
                                                    if 0.8 <= priority_level and priority_level <= 1.0: #between 0.7 - 1.0
                                                        priority_class = 'high' 
                                                        cnt_High_pr +=1
                                                    elif 0.4 <= priority_level and priority_level <= 0.7:  #between 0.4 - 0.7
                                                        priority_class = 'medium'
                                                        cnt_Medium_pr +=1
                                                    else: 
                                                        priority_class = 'low' #between 0.0 - 0.3
                                                        cnt_Low_pr +=1
                                                    sfc_priority.append(priority_level)
                                                    global_services.priority_update(episode, sfc, vnf_per_service, 
                                                                                    priority_level, priority_class)
                                                    if priority_level == 1:
                                                        macro_priority_level = 1 #int(truncate(priority_level, 0))
                                                        micro_priority_level = 0 #round((truncate(priority_level, 1)- macro_priority_level),2)
                                                    else:
                                                        macro_priority_level = int(truncate(priority_level*10, 0))
                                                        micro_priority_level = int(truncate(priority_level*100, 1))- macro_priority_level*10
                                                        #micro_env_pr_level = round(micro_env_pr_level,2)
                                                    if macro_priority_level == macro_env_pr_level:
                                                        #first level clearance
                                                        pred_count_macro_pr += 1
                                                        epi_pred_count_macro_pr += 1
                                                        if micro_priority_level == micro_env_pr_level:
                                                            pred_count_micro_pr += 1
                                                            epi_pred_count_micro_pr += 1
                                                    pred_macro_acc =round(((pred_count_macro_pr/pred_cnt_tot_sfc)*100),2)
                                                    pred_micro_acc =round(((pred_count_micro_pr/pred_cnt_tot_sfc)*100),2)
                                                    if (pred_macro_acc/100) >= acc_theshold_limit:
                                                        done_status = "Done"
                                                    if overall_sfc == cnt_tot_sfc:
                                                        pred_macro_acc =round(((pred_count_macro_pr/pred_cnt_tot_sfc)*100),2)
                                                        pred_micro_acc =round(((pred_count_micro_pr/pred_cnt_tot_sfc)*100),2)
                                            else: 
                                                priority_level = arv_per_sfc[0][1]['env_priority']
                                            #print("SFC: ", sfc, "priority_level: ", priority_level)
                                            pr_sfc_end = time.perf_counter()
                                            expected_wt_B = round((pr_sfc_end - pr_sfc_start), 6) # step2
                                            #print("expected_wt_B: ", expected_wt_B)
                                            epi_expected_wt_B[sfc] = expected_wt_B
                                else:           
                                    pass
                                
                                if switch_scheduling is True:
                                    #print("switch_scheduling: ", switch_scheduling)
                                    #print("episode: ", episode)
                                    sch_sfc_start = time.perf_counter()
                                    if sch_model == 'DDPG':
                                        print("sch_model: ", sch_model)
                                        sch_obs = []
                                        threshold_priority_level    = arv_per_sfc[0][1]['priority_level']
                                        threshold_reliability       = arv_per_sfc[0][1]['reliability']
                                        threshold_theo_wating_time  = arv_per_sfc[0][1]['theo_wating_time']
                                        norm_waitingtime            = arv_per_sfc[0][1]['norm_waiting_time']
                                        threshold_reliability       = arv_per_sfc[0][1]['reliability']
                                        threshold_evnpriority_level = arv_per_sfc[0][1]['env_priority']
                                        threshold_int_flowtype      = arv_per_sfc[0][1]['int_flow_behaviour']
                                        current_starving_sfc = None
                                        intersec_sfc = None
                                        starving_sfc_index = None
                                        if threshold_priority_level <= 0.2:
                                            current_starving_sfc = sfc
                                            ar_starving_sfc.append(sfc)
                                            
                                        traffic_input_dataset,  traffic_output_dataset = [], []
                                        class_traffic_classifer = traffic_classifer(traffic_input_dataset, 
                                                                                    traffic_output_dataset, 
                                                                                    episode, cluster_testing)
                                        
                                        classifier_datasets = [None, None, None, None]
                                        if switch_clustering is True:
                                            #print( "==>", switch_clustering, classifier_training_status)
                                            mu, std = 100, 300
                                            pred_traffic_num_sfc_requested = abs(random.normal(mu, std, 1))
                                            while pred_traffic_num_sfc_requested < 1:
                                                pred_traffic_num_sfc_requested = abs(random.normal(mu, std, 1))
                                            pred_num =  abs(int(pred_traffic_num_sfc_requested))
                                            pred_f = FBM(n  = int(pred_num), hurst=0.7, length=length, method='daviesharte')
                                            pred_fbm_sample = abs(pred_f.fbm())
                                            pred_fgn_sample = pred_f.fgn()
                                            pred_t_values   = pred_f.times() #stationary traffic
                                            pred_traffic_packet_per_sfc = sum(pred_t_values) #load
                                            
                                            predict_dataset = [threshold_priority_level, 
                                                               threshold_int_flowtype, pred_num, 
                                                               pred_traffic_packet_per_sfc]
                                            #print("==> predict_dataset", predict_dataset)
                                            #print("classifier_models: ", classifier_models)
                                            Classifier_training = class_traffic_classifer.classifier_models(classifier_models, 
                                                                                                            classifier_datasets,
                                                                                                            predict_dataset,
                                                                                                            classifier_training_status,
                                                                                                            Switch_classifier_info)
                                            
                                            #print("Classifier_training: ", Classifier_training)
                                            #classifier_measurements[clusterModel] = Classifier_training
                                            traffic_pred = Classifier_training[4][0]
                                            #print(traffic_pred)
                                        else: 
                                            traffic_pred = -1
                                            
                                        #print("traffic_pred: ", traffic_pred)   
                                        #updating in global level
                                        global_services.classifier_class_update(episode, sfc, vnf_per_service, 
                                                                                traffic_pred, pred_num,
                                                                                pred_traffic_packet_per_sfc
                                                                               )   
                                        
                                        sch_obs = [threshold_theo_wating_time, 
                                                   threshold_reliability,
                                                   threshold_priority_level, 
                                                   traffic_pred
                                                  ]
                                        sch_rank = sch_agent.choose_action(sch_obs, sch_n_state)
                                        temp_my_dict[sfc] = sch_rank
                                        sorted_sfc_sch = dict(sorted(temp_my_dict.items(), 
                                                                    key=lambda item: item[1], 
                                                                    reverse = True))
                                        sorted_sch_list = list(sorted_sfc_sch.keys())
                                        
                                        if ar_starving_sfc is None or not ar_starving_sfc:
                                            pass
                                        else:
                                            intersec_sfc = set(sorted_sch_list).intersection([current_starving_sfc])
                                            if intersec_sfc is None or not intersec_sfc:
                                                pass
                                            else:
                                                starving_sfc_index = sorted_sch_list.index(current_starving_sfc) 
                                                
                                        if cnt_sfc_arrival == len(sfc_arrival_num[cnt_poisson_dist])-1:
                                            done_sch = True
                                            next_sch_obs = [#sfc, 
                                                            #threshold_evnpriority_level, 
                                                            #threshold_priority_level, 
                                                            threshold_theo_wating_time, 
                                                            threshold_reliability,
                                                            threshold_priority_level,
                                                            traffic_pred
                                                           ]
                                        else:
                                            next_sfc = cnt_sfc_arrival + 1
                                            next_threshold_priority_level    = arr_sfc_de[next_sfc][0][0][1]['priority_level']
                                            next_threshold_reliability       = arr_sfc_de[next_sfc][0][0][1]['reliability']
                                            next_threshold_theo_wating_time  = arr_sfc_de[next_sfc][0][0][1]['theo_wating_time']
                                            next_threshold_evnpriority_level = arr_sfc_de[next_sfc][0][0][1]['env_priority']
                                            next_threshold_int_flowtype      = arr_sfc_de[next_sfc][0][0][1]['int_flow_behaviour']
                                            traffic_pred = -1
                                            next_sch_obs = [next_threshold_theo_wating_time, 
                                                            next_threshold_reliability,
                                                            next_threshold_evnpriority_level, 
                                                            traffic_pred] 
                                        sch_next_state = next_sch_obs
                                        #Reward function
                                        A_cost = (threshold_priority_level*reward_pts) + (norm_waitingtime*reward_pts)
                                        link_cost = 0 #(1/sum(ar_link_bw))*reward_pts
                                        cpu_cost  = 0 #(1/sum(ar_sfc_cpu))*reward_pts
                                        reliability_cost = threshold_reliability*reward_pts
                                        B_cost = link_cost + cpu_cost + reliability_cost
                                        decay = 0.1
                                        alpha = 1
                                        
                                        pos_sfc = starving_sfc_index
                                        if pos_sfc is None:
                                            starvation_rate = 0
                                            pass
                                        else:
                                            starvation_rate = alpha*pow(decay,pos_sfc)
                                        S_cost = starvation_rate*reward_pts
                                        sch_rewardfunc = A_cost + B_cost + S_cost
                                        sch_agent.store_transition(sch_obs, 
                                                                   sch_rank, 
                                                                   sch_rewardfunc, 
                                                                   sch_next_state, 
                                                                   done_sch, 
                                                                   sfc)
                                        sch_agent.learn()
                                        sch_sfc_end = time.perf_counter()

                                    elif sch_model == 'FIFO': 
                                        sorted_sch_list = sfc_arrival_num[cnt_poisson_dist] 
                                        sch_sfc_end = time.perf_counter()
                                    elif sch_model == 'WFQ':
                                        if arv_per_sfc[0][1]['priority_class'] == 'high':
                                            h_pr_queue.append(sfc)
                                        elif arv_per_sfc[0][1]['priority_class'] == 'medium':
                                            m_pr_queue.append(sfc)
                                        else: 
                                            l_pr_queue.append(sfc)
                                        sorted_sch_list = sfc_arrival_num[cnt_poisson_dist]
                                        sch_sfc_end = time.perf_counter()
                                    else:
                                        #print(sch_model)
                                        #print("arv_per_sfc: ", arv_per_sfc[0][1])
                                        temp_my_dict[sfc] = arv_per_sfc[0][1]['priority_level']
                                        #print(temp_my_dict)
                                        sorted_sfc_sch = dict(sorted(temp_my_dict.items(), 
                                                                    key=lambda item: item[1], 
                                                                    reverse = True))
                                        sorted_sch_list = list(sorted_sfc_sch.keys())
                                        sch_sfc_end = time.perf_counter()
                                        #print("sorted_sch_list: ", sorted_sch_list)
                                cnt_sfc_arrival += 1
                            cnt_poisson_dist += 1
                            #print(sch_model)
                            #print("sorted_sch_list: ", sorted_sch_list)
                            
                            # Adaptive scheduling
                            if sch_model == 'DDPG':
                                temp_sfc_deploy = []
                                for w in sorted_sch_list:
                                    temp_var = []
                                    for q, p in arr_sfc_de:
                                        if q[0][1]['sfc'] == w:
                                            temp_var = [q,p]
                                            temp_sfc_deploy.append(temp_var)
                                        else:
                                            pass
                            elif sch_model == 'FIFO': 
                                temp_sfc_deploy = arr_sfc_de
                            elif sch_model == 'WFQ':
                                temp_sfc_deploy = []
                                wfq_sch_sfc_start = time.perf_counter()
                                sorted_sch_list = wfq_sch(arr_sfc_de, h_pr_queue, m_pr_queue, l_pr_queue)
                                wfq_sch_sfc_end = time.perf_counter()
                                for w in sorted_sch_list:
                                    temp_var = []
                                    for q, p in arr_sfc_de:
                                        if q[0][1]['sfc'] == w:
                                            temp_var = [q,p]
                                            temp_sfc_deploy.append(temp_var)
                                        else:
                                            pass
                            else: #priority-based!!
                                temp_sfc_deploy = []
                                for w in sorted_sch_list:
                                    temp_var = []
                                    for q, p in arr_sfc_de:
                                        if q[0][1]['sfc'] == w:
                                            temp_var = [q,p]
                                            temp_sfc_deploy.append(temp_var)
                                        else:
                                            pass
                            
                            if sch_model == 'WFQ':
                                wfq_sch_sfc = wfq_sch_sfc_end - wfq_sch_sfc_start
                            else:
                                wfq_sch_sfc =0
                            
                            sch_sfc_wt = (sch_sfc_end-sch_sfc_start)+wfq_sch_sfc
                            #print("sch_sfc_wt: ", sch_sfc_wt)
                            temp_val = [std_value for i in (range(env.network_len_nodes))]
                            prob_val = [float(i)/sum(temp_val) for i in temp_val]
                            # services will arrive here.
                            num_sfc_per_arrival = 0
                            sfc_waited_time =  0
                            
                            # edited on 15th sept 2022
                            #print("sorted_sch_list: ", sorted_sch_list)
                            for t_sfc in sorted_sch_list:
                                ar_sfc_current_waitingtime[t_sfc] = sfc_waited_time
                            temp_sorted_sch_list = sorted_sch_list.copy()
                            ar_sorted_sch_list.append(sorted_sch_list)
                            #print("ar_sfc_current_waitingtime: ", ar_sfc_current_waitingtime)
                            #print("overall_expected_wt_B: ", overall_expected_wt_B)
                            #print("ar_sfc_current_waitingtime: ", ar_sfc_current_waitingtime)
                            for per_sfc, per_link in temp_sfc_deploy:
                                done  = False
                                sfc   = per_sfc[0][1]['sfc'] 
                                epi_temp_sfc_arrived.append(sfc)
                                arr_traffic_packet_per_sfc = []
                                min_samples = min_samples
                                dict_traffic_packet_per_sfc = {}
                                #print("")
                                #print("SFC", sfc)
                                sfc_priority_level =  per_sfc[0][1]['priority_level']
                                sfc_priority_class =  per_sfc[0][1]['priority_class']
                                sfc_int_flow       =  per_sfc[0][1]['int_flow_behaviour']
                                traffic_N = 0 #None
                                traffic_load = 0 #None
                                temp_s = 0
                                temp_pr_ft = [sfc, sfc_priority_level, sfc_int_flow, traffic_N, traffic_load]
                                traffic_transition_dict[sfc] = temp_pr_ft
                                #print("traffic_transition_dict: ", traffic_transition_dict)
                                cnt_tot_sfc += 1
                                
                                num_of_vl = per_link[0][2]['numvl']
                                decompmvnf = False
                                res_exh = False
                                #checking if the deploying sfc is within the waiting threshold
                                #sfc_wt = per_sfc[0][1]['norm_waiting_time'] #theo_wating_time'
                                sfc_wt = per_sfc[0][1]['theo_wating_time']
                                sfc_rel = per_sfc[0][1]['reliability'] #reliability_factor
                                sfc_rel_factor = per_sfc[0][1]['reliability_factor']
                                sfc_start = time.perf_counter() 
                                traffic_pred = per_sfc[0][1]['classifier_class']
                                traffic_load = per_sfc[0][1]['traffic_load']
                                epi_traffic_class_gen[sfc] = traffic_pred
                                epi_predicted_traffic_classes_arr.append(traffic_pred)
                                epi_sfc_pr_traffic_class_all[sfc] = [sfc_priority_level, sfc_priority_class, traffic_pred]
                                
                                if sfc_priority_class == 'low'and traffic_load == 1:
                                    epi_cnt_low_pr_traffic_1 += 1

                                if ar_sfc_current_waitingtime[sfc] >= sfc_wt:
                                    #print("Starved: Deployment time", sfc_deployment_time)
                                    cnt_sfc_starved_before_serving += 1
                                    sfc_priority_class =  per_sfc[0][1]['priority_class']
                                    sfc_starved_before_serving.append(sfc)
                                    sfc_Prclass_starved_before_serving.append(sfc_priority_class)
                                    #print("expired SFC:", sfc, "sfc_priority_class:", sfc_priority_class) 
                                    #print("did SFC starve??", sfc)
                                    epi_unemb_sfc.append(sfc)
                                    emb_status = 'starved'
                                    sfc_deployment_time = sfc_deployment_time#
                                    global_services.emb_update(episode, sfc, vnf_per_service, emb_status, 
                                                               sfc_deployment_time)   
                                else:
                                    sfc_priority_class =  per_sfc[0][1]['priority_class']
                                    #microvnf
                                    micro_requested_services, micro_requested_nodes = [], []
                                    vnf_per_service = vnf_per_service
                                    #Estimating the number of VNF persent per sfc
                                    state_space_size = vnf_per_service
                                    link_space_size = num_of_vl
                                    action_space_size = env.network_len_nodes
                                    actions_per_iteration,accpeted_action = [], []
                                    update_actionspace, updated_path_perSFC  = [], []
                                    reward_per_current_states, rewardlink_per_current_states = [], []
                                    new_reward_per_current_states = []
                                    reward_per_service, rewardDelay_per_current_states= [], []
                                    sum_reward_per_service, score_per_service =[], []
                                    micro_vnf_id, global_mVNF_action = [], []
                                    global_updated_path_perVNF = []
                                    my_dict = {}
                                    dict_node_avail = {}
                                    sorted_list = []
                                    rewards = 0
                                    reward_action = 0
                                    global_decomp = []
                                    emb_mVNFID = []
                                    #Problem-solving algorithm starts
                                    count_vnf, count_updated_path_perSFC = 0, 0
                                    count_VNF, sum_count_mvnf_persfc = 0,0
                                    count_mvnf_persfc = []
                                    done_microvnf = False
                                    for node, available_res in env.network_nodes:
                                        my_dict[node] = available_res['emb_vnf']
                                        sorted_action = dict(sorted(my_dict.items(), key=lambda item: item[1], reverse = False))
                                    keys_sorted_action = sorted_action.keys()
                                    action_space = list(map(int,keys_sorted_action))
                                    up_actionspace = action_space
                                    temp_sorted_action = sorted_action
                                    vals_sorted_action = sorted_action.values()
                                    if sum(vals_sorted_action) == 0:
                                        break
                                    norm_vals_sorted_action = [float(i)/sum(vals_sorted_action) for i in vals_sorted_action]
                                    #resource got exhusted
                                    sum_resouces = sum(list(vals_sorted_action))
                                    nodal_avail = round(sum_resouces/tot_vnf_network, 3)
                                    #Reachitechture
                                    catalog_mVNF= {}
                                    for node in original_actionspace:
                                        catalog_mVNF[node] = []
                                        temp= {}  
                                    # checking the existence of the mVNF                                    
                                    temp_accepted_action = {}
                                    for node in original_actionspace:
                                        temp_accepted_action[node] = []
                                    rewards_persfc = []
                                    for vnf,requested_res in per_sfc:
                                        vnf_start = time.perf_counter_ns()
                                        done_vnf = False
                                        vnf_cnt +=1
                                        global_services.reset(episode, sfc, vnf)
                                        updated_path_perVNF = []
                                        local_rewardVNF = 0
                                        count_mvnf = 0
                                        reward_vnfs = []                                   
                                        if not action_space:
                                            res_exh = True
                                            sum_resouces = sum(list(vals_sorted_action))
                                            break     
                                        temp_val[action] = temp_val[action] + pref_node_val
                                        prob_val = [float(i)/sum(temp_val) for i in temp_val]
                                        temp_vals_sorted_action = vals_sorted_action
                                        temp_norm_vals_sorted_action = norm_vals_sorted_action
                                        prev_action = action
                                        prev_vnf_res_req = vnf_res_req
                                        vnf_res_req = requested_res['cpu_req']
                                        pr_level = requested_res['priority_level']
                                        sum_resouces = sum(list(vals_sorted_action))
                                        nodal_avail = round(sum_resouces/tot_vnf_network, 3)                                     
                                        decompose_criteria = decomp_identifier(vnf_res_req, nodal_avail, max_vnf_resource, 
                                                                               episode, sfc, vnf, vnf_per_service)
                                        #Decomposition check 1: based on VNF resource
                                        if vnf_res_req > threshold_decompose:
                                            decompose_candidate = decompose_criteria.potential_vnf(threshold_decompose, 
                                                                                                   done_potential_vnf = False)
                                            done_decompvnf = decompose_criteria.done_potential_vnf                                
                                            global_services.update_vnf(episode,sfc, vnf,decompose_candidate, done_decompvnf)
                                            decomp= decompose_criteria.decomp
                                            num_micro_vnf = decompose_criteria.num_micro_vnf
                                            if done_decompvnf == False:
                                                pass
                                        #Decomposition check 2: based on nodal availability 
                                        elif nodal_avail < threshold_nodal_decomp:
                                            decompose_candidate = decompose_criteria.nodal_availability(done_nodal = False)
                                            done_decompvnf = decompose_criteria.done_nodal
                                            global_services.update_vnf(episode,sfc, vnf,decompose_candidate, done_decompvnf)
                                            decomp = decompose_criteria.decomp
                                            num_micro_vnf = decompose_criteria.num_micro_vnf
                                            unique_id = decompose_criteria.unique_id
                                            if unique_id:
                                                micro_vnf_id.append(unique_id)
                                            if done_decompvnf == False:
                                                pass
                                        else:
                                            pass
                                        reward_per_current_vnfstates = []
                                        rewardlink_per_current_vnfstates = []
                                        microvnf_accpeted_action = [] 
                                        t_microvnf_accpeted_action = []
                                        micro_const = microvnf_construction(max_vnf_resource = max_vnf_resource, 
                                                                            episode = episode, 
                                                                            sfc = sfc, vnf = vnf, cpu_req = vnf_res_req, 
                                                                            n =vnf_per_service , val =0, 
                                                                            pr_level =pr_level, sfc_rel = sfc_rel, 
                                                                            traffic_pred = traffic_pred)
                                        micro_vnf, micro_vnf_res = -1, -1
                                        vnf_link_resources = per_link
                                        if requested_res['microvnf_done_status'] == True:
                                            done_mVNF = True
                                            #print("==> miroserives")
                                            decomp =[]
                                            global_decomp.append(decomp)
                                            if decomp is None or not decomp:
                                                    break
                                            emb_micro = micro_const.microvnf_agent(episode = episode, 
                                                                                   sfc = sfc, 
                                                                                   vnf = vnf, 
                                                                                   decomp = decomp , 
                                                                                   n_actions = env.network_len_nodes, 
                                                                                   lr = learning_rate, 
                                                                                   epsilon = epsilon,
                                                                                   mem_size = mem_size,
                                                                                   batch_size = batch_size, 
                                                                                   vnf_cnt = vnf_cnt,
                                                                                   gamma        = gamma,
                                                                                   eps_dec      = eps_dec,
                                                                                   eps_min      = eps_min,
                                                                                   replace      = replace,
                                                                                   state_memory = state_memory,
                                                                                   action_memory = action_memory,
                                                                                   reward_memory = reward_memory,
                                                                                   next_state_memory = next_state_memory,
                                                                                   terminal_memory = terminal_memory,
                                                                                   Neurons_per_HiddenLayer = Neurons_per_HiddenLayer,
                                                                                   drop_prob = drop_prob,
                                                                                   chkpt_dir = 'models/',
                                                                                   algo = 'DDQNAgent',
                                                                                   env_name = 'BtEurope',
                                                                                   count_VNF = vnf,
                                                                                   done = done,
                                                                                   amount_local_reward = amount_local_reward,
                                                                                   reward_per_current_vnfstates = reward_per_current_vnfstates,
                                                                                   microvnf_accpeted_action = microvnf_accpeted_action, 
                                                                                   accpeted_action = accpeted_action, 
                                                                                   updated_path_perVNF = updated_path_perVNF,
                                                                                   reward_per_current_states = reward_per_current_states,
                                                                                   nodal_avail = nodal_avail,
                                                                                   sum_resouces = sum_resouces,
                                                                                   state_space_size = state_space_size, 
                                                                                   threshold_delay = threshold_delay,
                                                                                   rewardlink_per_current_states = rewardlink_per_current_states,
                                                                                   vnf_link_resources = vnf_link_resources, 
                                                                                   temp_accepted_action = temp_accepted_action,
                                                                                   original_actionspace = original_actionspace, #added om 07032022
                                                                                   mvf_t_max = mvf_t_max, 
                                                                                   mvf_t_min = mvf_t_min, 
                                                                                   mvf_old_t_current = mvf_old_t_current, 
                                                                                   new_reward_per_current_states = new_reward_per_current_states,
                                                                                   traffic_pred = traffic_pred,
                                                                                   traffic_load = traffic_load
                                                                                  )
                                            count_VNF = emb_micro[0]
                                            reward_per_current_states = emb_micro[1]
                                            rewardlink_per_current_states = emb_micro[2]
                                            accpeted_action = emb_micro[3]
                                            updated_path_perVNF = emb_micro[4]
                                            vnf_cnt = emb_micro[5] -1
                                        else:
                                            #print("here")
                                            #print("sfc:", sfc, "traffic_pred", traffic_pred, classifier_training_status)
                                            
                                            vals_sorted_action = list(vals_sorted_action)
                                            micro_vnf, micro_vnf_res = -1, -1
                                            #print(traffic_pred)
                                            observation   = [episode,sfc, vnf,vnf_res_req, pr_level, 
                                                             micro_vnf, micro_vnf_res, traffic_pred]
                                            state_dim     = [len(observation)]
                                            done_mVNF     = False
                                            action_space = up_actionspace
                                            action = agent.choose_action(observation,
                                                                         prob_val,
                                                                         temp_val, 
                                                                         original_actionspace)
                                            vnf_end = time.perf_counter_ns()
                                            vnf_exutime = vnf_end - vnf_start
                                            if not action in up_actionspace:
                                                reward_action_not = -1000
                                            else:
                                                reward_action_not = 0    
                                            state = vnf 
                                            subjected_to = vnf_constraints(amount_local_reward, vnf_res_req,nodal_avail,
                                                                           max_vnf_resource, episode, sfc, vnf, vnf_per_service,
                                                                           sum_resouces, accpeted_action, switch_decomp)
                                            local_rewardVNF = subjected_to.constraint_1(action)
                                            if local_rewardVNF == amount_local_reward:
                                                pref_node_val = 10 #amount_local_reward #amount_local_reward #1
                                            else:
                                                pref_node_val = 0
                                                
                                            # Modifying Reward Function
                                            for node, available_res in env.network_nodes:   
                                                dict_node_avail[node] = (available_res['emb_vnf'] / tot_vnf_per_core)
                                                
                                            # Part 1: Reward based on the quality of Nodes selected
                                            reward_Node_based = local_rewardVNF*dict_node_avail[str(action)]
                                            reward_vnfs.append(reward_Node_based)  
                                            
                                            # Part 2: Reward based on the Priority service
                                            reward_pr = local_rewardVNF*priority_level
                                            reward_vnfs.append(reward_pr)
                                            
                                            # Part 3: Reward based on the sfc_rel
                                            reward_rel = local_rewardVNF*sfc_rel
                                            reward_vnfs.append(reward_rel)
                                            
                                            # Part 4: Reward based on the traffic_pred
                                            #print("traffic_pred", traffic_pred)
                                            if traffic_load == None:
                                                traffic_load = 0
                                            else:
                                                traffic_load = traffic_load

                                            if local_rewardVNF == 1000:
                                                reward_traffic_pred = local_rewardVNF*(traffic_pred*traffic_load)
                                                #print("reward_traffic_pred:", reward_traffic_pred, traffic_pred, local_rewardVNF)
                                            else: 
                                                if traffic_pred == -1:
                                                    reward_traffic_pred = -(local_rewardVNF*(traffic_pred*traffic_load))
                                                else:
                                                    reward_traffic_pred = (local_rewardVNF*(traffic_pred*traffic_load))
                                            #print("reward_traffic_pred:", reward_traffic_pred, traffic_pred, local_rewardVNF)
                                            reward_vnfs.append(reward_traffic_pred)
                                            
                                            # Part 5: Reward based on scheduling delay
                                            t_current = vnf_exutime
                                            if t_max is None or  t_current >= t_max: t_max = t_current
                                            if (t_current < t_max and t_min == 0) or (t_current <= t_min): t_min = t_current
                                            elif (t_current >= old_t_current and t_min == 0):t_min = old_t_current
                                            norm_time = 1 - Normalization(t_current, t_max, t_min)
                                            time_reward = local_rewardVNF*norm_time
                                            reward_vnfs.append(time_reward)
                                            old_t_current = t_current
                                            total_reward_vnf = round(sum(reward_vnfs), 4)
                                            new_reward_per_current_states.append(total_reward_vnf)
                                            if local_rewardVNF != -1000:
                                                local_rewardVNF = local_rewardVNF
                                                reward_per_current_states.append(local_rewardVNF)
                                            else:
                                                local_rewardVNF = local_rewardVNF 
                                                if switch_decomp is False:
                                                    decomp = None
                                                    decomp = []
                                                else: 
                                                    decomp = subjected_to.decomp
                                                    decomp = decomp    
                                                global_decomp.append(decomp)
                                                decompmvnf = True
                                                if decomp is None or not decomp:
                                                    global_reward = 0
                                                    #print("==> no decomp possible")
                                                    local_reward = local_rewardVNF
                                                    rewards = total_reward_vnf + global_reward
                                                    agent.store_transition(observation, action, 
                                                                           rewards, next_state, 
                                                                           done, vnf_cnt)
                                                    agent.learn()
                                                    epsilon = agent.epsilon
                                                    break
                                                else:
                                                    #print("VNF:", vnf)
                                                    #print("decomp")
                                                    local_rewardVNF = 1000 
                                                    unique_id = decomp[2]
                                                    if unique_id:
                                                        micro_vnf_id.append(unique_id)    
                                                for item in catalog_mVNF.items():
                                                    #print("item", item)
                                                    mVNFID_temp = []
                                                    for k in item[1]:
                                                        repeat_mVNFid = {e for e in unique_id if e == k}
                                                        mVNFID_temp = temp_accepted_action[item[0]]
                                                        if(repeat_mVNFid):
                                                            if k in mVNFID_temp:
                                                                mVNFID_temp = mVNFID_temp
                                                            else:
                                                                mVNFID_temp = mVNFID_temp + [k]
                                                        temp_accepted_action[item[0]] = mVNFID_temp
                                                count_mvnf = len(unique_id)
                                                count_mvnf_persfc.append(count_mvnf)
                                                prev_maction = False
                                                emb_micro = micro_const.microvnf_agent(episode = episode, 
                                                                   sfc = sfc, 
                                                                   vnf = vnf, 
                                                                   decomp = decomp , 
                                                                   n_actions = env.network_len_nodes, 
                                                                   lr = learning_rate, 
                                                                   epsilon = epsilon,
                                                                   mem_size = mem_size,
                                                                   batch_size = batch_size, 
                                                                   vnf_cnt = vnf_cnt,
                                                                   gamma        = gamma,
                                                                   eps_dec      = eps_dec,
                                                                   eps_min      = eps_min,
                                                                   replace      = replace,
                                                                   state_memory = state_memory,
                                                                   action_memory = action_memory,
                                                                   reward_memory = reward_memory,
                                                                   next_state_memory = next_state_memory,
                                                                   terminal_memory = terminal_memory,
                                                                   Neurons_per_HiddenLayer = Neurons_per_HiddenLayer,
                                                                   drop_prob =drop_prob,
                                                                   chkpt_dir = 'models/',
                                                                   algo = 'DDQNAgent',
                                                                   env_name = 'BtEurope',
                                                                   count_VNF = vnf,
                                                                   done = done,
                                                                   amount_local_reward = amount_local_reward,
                                                                   reward_per_current_vnfstates = reward_per_current_vnfstates,
                                                                   microvnf_accpeted_action = microvnf_accpeted_action, 
                                                                   t_microvnf_accpeted_action = t_microvnf_accpeted_action, 
                                                                   accpeted_action = accpeted_action, 
                                                                   updated_path_perVNF = updated_path_perVNF,
                                                                   reward_per_current_states = reward_per_current_states,
                                                                   nodal_avail = nodal_avail,
                                                                   sum_resouces = sum_resouces,
                                                                   state_space_size = state_space_size, 
                                                                   threshold_delay = threshold_delay,
                                                                   rewardlink_per_current_states = rewardlink_per_current_states,
                                                                   vnf_link_resources = vnf_link_resources,
                                                                   temp_accepted_action = temp_accepted_action,
                                                                   prob_val = prob_val,
                                                                   temp_val = temp_val,
                                                                   original_actionspace = original_actionspace, #added on 07032022
                                                                   prev_maction = prev_maction,
                                                                   min_pr = min_pr, #added on 13th April 2022
                                                                   max_pr = max_pr,  #added on 13th April 2022
                                                                   mvf_t_max = mvf_t_max, 
                                                                   mvf_t_min = mvf_t_min, 
                                                                   mvf_old_t_current = mvf_old_t_current, 
                                                                   new_reward_per_current_states = new_reward_per_current_states,
                                                                   traffic_pred = traffic_pred,
                                                                   traffic_load = traffic_load
                                                                   )
                                                mvf_t_max = emb_micro[7]
                                                mvf_t_min = emb_micro[8]
                                                new_reward_per_current_states = emb_micro[10]
                                                empt = []
                                                temp_count = 0
                                                for key in  microvnf_accpeted_action:
                                                    if type(key) is list:
                                                        pass
                                                    else:
                                                        empt = [unique_id[temp_count]]
                                                        catalog_mVNF[key] = unique_id[temp_count]  
                                                        if key in temp.keys():
                                                            empt = empt + temp[key]  
                                                        catalog_mVNF[key] = empt       
                                                        temp = dict(sorted(catalog_mVNF.items(), 
                                                                           key=lambda item: item[1], 
                                                                           reverse = False)) 
                                                    temp_count +=1
                                                count_VNF = emb_micro[0]
                                                #print("count_VNF: ", count_VNF)
                                                reward_per_current_states = emb_micro[1]
                                                rewardlink_per_current_states = emb_micro[2]
                                                accpeted_action = emb_micro[3]
                                                updated_path_perVNF = emb_micro[4]
                                                vnf_cnt = emb_micro[5]-1
                                                global_mVNF_action.append(microvnf_accpeted_action)
                                                global_updated_path_perVNF.append(updated_path_perVNF)    
                                            #Constraint 3 and 4: : checking for the available resouce for Links#
                                            vnf_link_resources = per_link 
                                            env_networkgraph   = env.network_graph
                                            env_networkedges   = env.network_edges
                                            subjected_to.constraint_3(state_space_size, 
                                                                   vnf_link_resources,
                                                                   env_networkgraph, 
                                                                   env_networkedges, 
                                                                   threshold_delay, 
                                                                   updated_path_perSFC
                                                                   )
                                            rewardlink_per_current_states = subjected_to.local_reward_link
                                            updated_path_perSFC = subjected_to.updated_path_perSFC
                                            #Next state
                                            next_state = vnf+1
                                            if next_state >= state_space_size:
                                                next_state = 0
                                                next_traffic_pred = -1
                                                next_pr_level = round(rand.uniform(0,1), 2)

                                                if len(sorted_sch_list) == num_sfc_per_arrival+1: 
                                                    next_sfc = max(sorted_sch_list)+1
                                                else: 
                                                    next_sfc = sorted_sch_list[num_sfc_per_arrival+1]

                                                next_observation = [a, next_sfc,next_state,vnf_res_req, next_pr_level, 
                                                                    micro_vnf, micro_vnf_res, next_traffic_pred]
                                            else: 
                                                next_state = next_state
                                                next_observation = [a, sfc,next_state,vnf_res_req, pr_level, 
                                                                    micro_vnf, micro_vnf_res, traffic_pred]
                                            state = observation 
                                            next_state = next_observation
                                        #Reward Function
                                        global_reward = 0
                                        if (sum(reward_per_current_states) == amount_local_reward*state_space_size) and \
                                            (sum(rewardlink_per_current_states) == amount_local_reward*link_space_size):
                                            done = True
                                            deployed_sfc_per_episode.append(sfc)
                                            deployed_priority_per_episode.append(pr_level)
                                            vnf_node_resources = per_sfc 
                                            vnf_link_resources = per_link 
                                            reward_func.reward_vnf(global_reward,
                                                                   rewardDelay_per_current_states, 
                                                                   rewardlink_per_current_states, 
                                                                   actions_per_iteration,
                                                                   accpeted_action,
                                                                   vnf_node_resources, 
                                                                   count_vnf, 
                                                                   updated_path_perSFC,
                                                                   vnf_link_resources, 
                                                                   count_updated_path_perSFC,
                                                                   count_success,
                                                                   global_decomp,
                                                                   emb_mVNFID,
                                                                   switch_reconst,
                                                                   global_updated_path_perVNF)
                                            count_updated_path_perSFC = reward_func.count_updated_path_perSFC
                                            count_VNF = reward_func.count_vnf
                                            global_reward = reward_func.global_reward
                                        local_reward = local_rewardVNF
                                        rewards = total_reward_vnf + global_reward
                                        rewards_persfc.append(rewards)
                                        if done_mVNF == True:
                                            pass
                                        else:
                                            #storing the transistions
                                            agent.store_transition(state, action, rewards, next_state, done, vnf_cnt)
                                            #learning:  providing state, action, reward and next state to the NN for learning.
                                            agent.learn()
                                            epsilon = agent.epsilon
                                    sum_rewards_persfc = sum(rewards_persfc)
                                    rewards_perepi.append(sum_rewards_persfc)
                                    
                                    if state_space_size == count_VNF and link_space_size == count_updated_path_perSFC:
                                        if decompmvnf is True:
                                            count_succes_decompvnf +=1
                                        sum_count_mvnf_persfc = sum(count_mvnf_persfc)
                                        count_mvnf_perepi.append(sum_count_mvnf_persfc)
                                        count_success += 1
                                        count_accpeted +=1
                                        #print("sfc: ",sfc, " priority: ", sfc_priority_level , " predc class: ", traffic_pred)
                                        epi_sfc_pr_traffic_class_emb[sfc] = [sfc_priority_level, sfc_priority_class, traffic_pred]
                                    
                                        
                                        if sfc_priority_class == "high":
                                            cnt_High_pr_emb += 1
                                            if traffic_pred == 1:
                                                cnt_High_traffic1_emb += 1
                                            else:
                                                cnt_High_traffic0_emb += 1
                                                
                                        elif sfc_priority_class == "medium":
                                            cnt_Medium_pr_emb += 1
                                            if traffic_pred == 1:
                                                cnt_Medium_traffic1_emb += 1
                                            else:
                                                cnt_Medium_traffic0_emb += 1
                                        else:
                                            cnt_Low_pr_emb += 1
                                            if traffic_pred == 1:
                                                cnt_Low_traffic1_emb += 1
                                            else: 
                                                cnt_Low_traffic0_emb += 1
                                                
                                            
                                        epi_emb_sfc.append(sfc) #array of embedded SFCs
                                        if sfc_rel >= 0.7:
                                            epi_emb_sfc_h_rel.append(sfc)
                                            if sfc_priority_class == 'high':
                                                cnt_rel += 1
                                        #print(cnt_rel)
                                        
                                        # update emd status: added 0n 11jan 2023
                                        emb_status = True                                        
                                    else:
                                        # update emd status: added 0n 11jan 2023
                                        epi_unemb_sfc.append(sfc)
                                        emb_status = False
                                        count_unsucces_decompvnf += 1
                                        count_mvnf_perepi.append(0)
                                    if res_exh is True:
                                        res_exhausted +=1
                                    num_sfc_per_arrival +=1
                                    
                                    sfc_end = time.perf_counter() #_ns()
                                    sfc_deployment_time = round(sfc_end-sfc_start,6) # time take by the agent to deploy the agent
                                    #print("SFC: ", sfc, sfc_deployment_time)
                                    
                                    # update emd status: added 0n 11jan 2023
                                    global_services.emb_update(episode, sfc, vnf_per_service, emb_status, 
                                                               sfc_deployment_time)

                                    if switch_sfc_expiring_check == True:
                                        for t_sfc in temp_sorted_sch_list: 
                                            if sfc == t_sfc:
                                                expected_wt_C = 0
                                                ar_sfc_current_waitingtime[sfc] += expected_wt_C
                                            else: 
                                                expected_wt_C = sfc_deployment_time
                                                ar_sfc_current_waitingtime[t_sfc] += expected_wt_C
                                        #print("expected_wt_C: ", expected_wt_C)
                                        temp_sorted_sch_list.remove(sfc)
                                    #else: #commented on 15th sept 2022
                                     #   pass # commented on 15th sept 2022
                                    #print(ar_sfc_current_waitingtime)   
                                    network_overall_accepted_action.append(accpeted_action)
                                    count_rejected = len(epi_temp_sfc_arrived) - count_accpeted
                                    count_failed   = len(epi_temp_sfc_arrived) - count_success 
                                    
                                    #episode_pr_rewardfunc.append(sfc_pr_rewardfunc)
                                    #print("sfc_priority: ", sfc_priority)
                                    episode_priority.append(sfc_priority)
                                    #episode_envpriority.append(sfc_envpriority)
                                    episode_priority_err.append(sfc_priority_err)   
                        
                            #print("epi_expected_wt_B: ", epi_expected_wt_B)
                            #print("episode", episode)
                            #overall_expected_wt_B[episode] = epi_expected_wt_B
                            #print("overall_expected_wt_B: ", overall_expected_wt_B)
                        #print("epi_traffic_class_gen: ", epi_traffic_class_gen, len(epi_traffic_class_gen))
                        #print("epi_predicted_traffic_classes_arr: ", epi_predicted_traffic_classes_arr)
                        count_class_0_predicted_traffic = epi_predicted_traffic_classes_arr.count(0)
                        count_class_1_predicted_traffic = epi_predicted_traffic_classes_arr.count(1)
                        #print(count_class_0_predicted_traffic, count_class_1_predicted_traffic)
                        epi_predicted_traffic_classes_dict[0] = count_class_0_predicted_traffic
                        epi_predicted_traffic_classes_dict[1] = count_class_1_predicted_traffic
                        #print("epi_sfc_pr_traffic_class_emb: ", epi_sfc_pr_traffic_class_emb)
                        
                        overall_epi_predicted_traffic_classes_dict[a] =  epi_predicted_traffic_classes_dict   
                        overall_epi_sfc_pr_traffic_class_emb[a] = epi_sfc_pr_traffic_class_emb 
                        overall_epi_sfc_pr_traffic_class_all[a] = epi_sfc_pr_traffic_class_all
                             
                        overall_epi_traffic_class_gen[a] = epi_traffic_class_gen
                        overall_ar_sorted_sch_list.append(ar_sorted_sch_list)
                        epi_cnt_sfc_starved_before_serving.append(cnt_sfc_starved_before_serving)
                        epi_cnt_sfc_Prclass_starved_before_serving.append(cnt_sfc_starved_before_serving)
                        overall_cnt_sfc_starved_before_serving.append(sfc_starved_before_serving)
                        overall_cnt_sfc_Prclass_starved_before_serving.append(sfc_Prclass_starved_before_serving)
                        
                        overall_epi_cnt_High_pr_emb.append(cnt_High_pr_emb)
                        overall_epi_cnt_Medium_pr_emb.append(cnt_Medium_pr_emb)
                        overall_epi_cnt_Low_pr_emb.append(cnt_Low_pr_emb)
                        
                        epi_pr_emb["H"] = cnt_High_pr_emb
                        epi_pr_emb["M"] = cnt_Medium_pr_emb
                        epi_pr_emb["L"] = cnt_Low_pr_emb
                        
                        overall_epi_pr_emb.append(epi_pr_emb)
                        overall_unemb_sfc.append(epi_unemb_sfc)
                        overall_emb_sfc.append(epi_emb_sfc)
                        overall_emb_sfc_h_rel.append(epi_emb_sfc_h_rel)
                        
                        overall_epi_cnt_High_traffic0_emb.append(cnt_High_traffic0_emb)                      
                        overall_epi_cnt_Medium_traffic0_emb.append(cnt_Medium_traffic0_emb)
                        overall_epi_cnt_Low_traffic0_emb.append(cnt_Low_traffic0_emb)
                        overall_epi_cnt_High_traffic1_emb.append(cnt_High_traffic1_emb) 
                        overall_epi_cnt_Medium_traffic1_emb.append(cnt_Medium_traffic1_emb) 
                        overall_epi_cnt_Low_traffic1_emb.append(cnt_Low_traffic1_emb)
                        overall_epi_cnt_low_pr_traffic_1.append(epi_cnt_low_pr_traffic_1)
                        
                    

                        #print("overall_epi_pr_emb: ", overall_epi_pr_emb)
                        if len(overall_sfc_priority) != len(global_services.vNetwork_service_details_er):
                            overall_sfc_priority.append(sfc_priority)
                            epi_cnt_High_pr.append(cnt_High_pr)
                            epi_cnt_Medium_pr.append(cnt_Medium_pr)
                            epi_cnt_Low_pr.append(cnt_Low_pr)
                            epi_pr["H"] = cnt_High_pr
                            epi_pr["M"] = cnt_Medium_pr
                            epi_pr["L"] = cnt_Low_pr
                            #print("epi_pr: ", epi_pr)
                            #print("cnt_High_pr: ", cnt_High_pr, "cnt_Medium_pr: ", cnt_Medium_pr, "cnt_Low_pr: ", cnt_Low_pr)
                            overall_epi_pr.append(epi_pr)
                            status_cnt_ridge = False
                            copy_overall_epi_pr = overall_epi_pr.copy()
                        else:
                            status_cnt_ridge = True
                            copy_overall_episode_envpriority = overall_episode_envpriority.copy()
                            #print("copy_overall_epi_pr: ",copy_overall_epi_pr)
                            #print("XXXX")
                        
                        overall_episode_priority.append(episode_priority)
                        if sch_model == 'DDPG':  
                            overall_episode_envpriority.append(sfc_envpriority)
                        else: 
                            copy_overall_episode_envpriority = overall_episode_envpriority.copy()
                            
                        overall_episode_priority_err.append(episode_priority_err)
                        overall_deployed_sfc_per_episode.append(deployed_sfc_per_episode)
                        overall_deployed_priority_per_episode.append(deployed_priority_per_episode)
                        sum_rewards_perepi = sum(rewards_perepi)
                        network_overall_sum_rewards_perepi.append(sum_rewards_perepi)
                        sum_count_mvnf_perepi = sum(count_mvnf_perepi)
                        network_overall_sum_mvnf.append(sum_count_mvnf_perepi)
                        network_overall_count_mvnf.append(count_mvnf_perepi)
                        network_overall_accepted.append(count_success)
                        network_overall_rejected.append(count_failed)
                        tot_decompsfc = (count_succes_decompvnf + count_unsucces_decompvnf) 
                        undecomp_accepted = num_sfc-tot_decompsfc
                        undecomp_rejected = tot_decompsfc
                        monolithic_deploy_success.append(undecomp_accepted)
                        monolithic_deploy_rejected.append(undecomp_rejected)
                        overall_res_exhausted.append(res_exhausted)
                        network_overall_tot_decompsfc.append(tot_decompsfc)
                        network_overall_count_succes_decompvnf.append(count_succes_decompvnf)
                        network_overall_count_unsucces_decompvnf.append(count_unsucces_decompvnf)
                        overall_epi_traffic_packets_dict[a] = epi_traffic_packets_dict
                        overall_epi_traffic_load_dict[a] = epi_traffic_load_dict
                        
                        if switch_priorty == True:
                            if pr_model == 'DDPG' or pr_model == 'DDQN': 
                                pass
                            else:
                                if done_status == "Train":
                                    cnt_macro_pr = tr_count_macro_pr
                                    acc = round(tr_macro_acc,2)
                                    epi_cnt_macro_pr = epi_tr_count_macro_pr
                                    epi_acc = round((epi_cnt_macro_pr/num_sfc)*100,2)
                                    epi_tr_correct_perc.append(epi_acc)
                                elif done_status == "Eval":
                                    cnt_macro_pr = eval_count_macro_pr
                                    acc = eval_macro_acc
                                    epi_cnt_macro_pr = epi_eval_count_macro_pr
                                    epi_acc = round((epi_cnt_macro_pr/num_sfc)*100,2)
                                else:
                                    cnt_macro_pr = pred_count_macro_pr
                                    acc = pred_macro_acc
                                    epi_cnt_macro_pr = epi_pred_count_macro_pr
                                    epi_acc = round((epi_cnt_macro_pr/num_sfc)*100,2)
                                    epi_pred_correct_perc.append(epi_acc)
                        else:
                            pass

                        if switch_clustering is True:
                            #pass
                            if a%num_episode==0:
                                pass
                                #clusterplot_val = class_traffic_clustering.clusteringplot(epi_clustering_status, 
                                #                                                          traffic_input_dataset, 
                                #                                                          cluster_label_val)

                          
                        print("*************")
                        print("cnt_sfc_arrival: ", cnt_sfc_arrival)
                        print("Total Services for Episode",a,"is",len(epi_temp_sfc_arrived), "Num accepted",count_success,"Num rejected",count_failed)
                        #print("Episode",a," Done Status: ", done_status," count_macro_pr: ", cnt_macro_pr," Epi count_macro_pr: ", epi_cnt_macro_pr, "Epi Acc: ", epi_acc, acc)
                        #print('Mean MAE: %.3f (%.3f)' % (m(scores), std(scores)))
                        #print("Episode",a," count_macro_pr: ", count_macro_pr, " Episode count macro: ", epi_cnt_macro_pr)
                        count_numVNF_perNode = []
                        count_num_mVNF_perNode = []
                        for ac in [i for i in range(env.network_len_nodes)]:
                            count =0
                            count_mVNF = 0
                            for key in network_overall_accepted_action:
                                for action in key:
                                    if type(action) is list:
                                        for maction in action:
                                            if maction is ac:
                                                count_mVNF += 1
                                    else:
                                        if action is ac:
                                            count +=1
                            count_numVNF_perNode.append(count)
                            count_num_mVNF_perNode.append(count_mVNF)
                        overall_count_numVNF_perNode.append(count_numVNF_perNode)
                        overall_count_num_mVNF_perNode.append(count_num_mVNF_perNode)
                        qq = list(env.network_graph.nodes(data=True))
                        remain_cpu = []
                        for key, val in qq:
                            cpu = val['emb_vnf']
                            #print(key, cpu)
                            remain_cpu.append(cpu)
                        sum_remain_cpu = sum(remain_cpu)
                        #remaining BW
                        residual_bw_res = []
                        for key in env.network_edges: 
                            residual_bw_res.append(key[2]['bw'])
                        overall_remain_cpu.append(sum_remain_cpu)
                        overall_remain_bw.append(residual_bw_res)
                        #print("epsilon", round(epsilon, 2))
                        network_overall_epsilon.append(round(epsilon, 2))
                        a += 1
                    
                    #print("overall_epi_traffic_class_gen[a]:", overall_epi_traffic_class_gen)    
                    
                    from statistics import mean 
                    mean_network_overall_accepted = mean(network_overall_accepted)
                    end = time.time()  
                    runtime = end-start
                    runtime = round(runtime,2)
                    print("runtime",runtime)
                    topology_name = str('Netrail')

                    overall_res  = []
                    sum_overall_res = 0
                    for per_episode in global_services.vNetwork_service_details_er:
                        res_perepi = []
                        sum_res_perepi = 0
                        for per_sfc, per_link in per_episode:
                            res_persfc = []
                            sum_res_persfc = 0
                            #print(per_sfc)
                            for vnf, val in per_sfc:
                                res_persfc.append(val['cpu_req'])
                            #print(res_persfc)  
                            sum_res_persfc = sum(res_persfc)
                            #print(sum_res_persfc)
                            res_perepi.append(sum_res_persfc)
                        #print(res_perepi)
                        sum_res_perepi = sum(res_perepi)
                        #print(sum_res_perepi)
                        overall_res.append(sum_res_perepi)
                    sum_overall_res = sum(overall_res)

                    upper_bound = round(tot_avail_res_network/min(overall_res), 2)
                    lower_bound = round(tot_avail_res_network/max(overall_res),2)

                    theo_bound  = [lower_bound, upper_bound]
                    #print(theo_bound)

                    Bound_Cal = ['overall_res: ' +str(overall_res), 'sum_overall_res: ' +str(sum_overall_res),\
                         'theo_bound: '  +str(theo_bound), 'min: ' +str(min(overall_res)), 'max: ' +str(max(overall_res))             
                        ]
                    #print(Bound_Cal)
                    
                    Epsilon = 1
                    overall_vnf = (vnf_per_service*num_sfc)*num_episode
                    Information = 'Information: '
                    experience_replay = ['Experience Replay:',('Memory Size: '+str(mem_size)), ('Batch Size: '+ str(batch_size)), ('Replace: '+ str(replace))]
                    topologyname = ['Tolpology:'+ topology_name, ('Node:'+ str(env.network_len_nodes)), ('Link:' + str(env.len_tot_links))]
                    execution = [('Episode: '+ str(num_episode)), ('SFC: ' + str(num_sfc)), ('VNF per SFC: '+ str(vnf_per_service))]
                    Info = [Information] + [Algorithm]+ [topologyname] + [Opimizer] + [execution]
                    #print(Info)    
                    Parameters = 'Parameters: '
                    NN_para = ['Neural Network: '+ ('Neuron units: '+ str(Neurons_per_HiddenLayer)), 'Hidden Layer: '+str(Hidden_layer),\
                               'Drop Probability: '+ str(drop_prob)]
                    learn_para = ['Learning :'+ ('Learning Rate: ' + str(learning_rate)), ('Epsilon: '+ str(Epsilon)), ('Gamma: '+ str(gamma)),\
                                 ('Epsilon Dec: '+ str(eps_dec)), 'Epsilon Min: '+ str(eps_min), Opimizer]
                    RL_para = ['RL rewards function: ' + ('Reward: ' +str(amount_local_reward)) , ('Penalty: ' + str(amount_local_penalty))]

                    Decomp = ['Decomp: ' +'switch_reconst: ' +str(switch_reconst),'switch_decomp: ' + str(switch_decomp),\
                              'n_state: ' + str(n_state), ' with No Constraint 2 '
                             ]
                    para = [Parameters] + [NN_para] + [learn_para]  + [experience_replay] + [RL_para] + [Decomp]
                    Topology_resource_init = ['Topology Resouce Initilization: '+ ('Latency: ' + str(threshold_delay)),\
                                              'CPU core: ' + str(core_cpu), 'VNF per Core: ' + str(num_vnf_per_core),\
                                              'Total VNF per core' + str(tot_vnf_per_core)]
                    Nodal_outage_para = ['Nodal Outage: ' + 'Outage Percentage: '+str(nodal_outage_percentage),\
                                         'No of nodes outaged: ' + str(round_nodal_outage)]
                    # Result
                    computation_time = 'Runtime: '+str(runtime)+ ' seconds'
                    results = ['Results: ' + computation_time, 
                               'Total Generated VNF: ' + str(overall_vnf),'Mean of Overall VNF: '+ str(mean_network_overall_accepted),\
                               'SAR: '+str(network_overall_accepted), 'Monolithic_deploy_success: ' + str(monolithic_deploy_success),\
                               'Monolithic_deploy_rejected: ' + str(monolithic_deploy_rejected),\
                               'Network_overall_tot_decompsfc: ' + str(network_overall_tot_decompsfc),\
                               'Network_overall_count_succes_decompvnf: ' + str(network_overall_count_succes_decompvnf),\
                               'network_overall_count_unsucces_decompvnf: ' + str(network_overall_count_unsucces_decompvnf),\
                               'network_overall_sum_mvnf: ' + str(network_overall_sum_mvnf),\
                               'network_overall_count_mvnf: ' +str(network_overall_count_mvnf),\
                               'network_overall_sum_rewards_perepi: '+str(network_overall_sum_rewards_perepi),\
                               'network_overall_epsilon: '+ str(network_overall_epsilon)
                              ]
                    #print(results)
                    resource_residual = ['Resource Residual: ' + 'VNF residual: '+ str(overall_remain_cpu) +\
                                         'Link Residual: '+ str(overall_remain_bw) + \
                                         #'VNF Distribution: '+str(Network_count_numVNF_perNode),\
                                         'overall_count_numVNF_perNode: ' + str(overall_count_numVNF_perNode),\
                                         'overall_count_num_mVNF_perNode: ' + str(overall_count_num_mVNF_perNode),\
                                         'overall_res_exhausted: '+ str(overall_res_exhausted)
                                        ]
                    priority = [' Priority: ' + 'pr_HL: ' +str(pr_HL) + \
                                ' overall_episode_pr_rewardfunc: ' + str(overall_episode_pr_rewardfunc)+\
                                ' overall_sfc_priority: ' +str(overall_sfc_priority)+\
                                ' overall_episode_envpriority: '+ str(overall_episode_envpriority)+\
                                ' copy_overall_episode_envpriority: ' + str(copy_overall_episode_envpriority)+\
                                ' epi_cnt_sfc_starved_before_serving: ' + str( epi_cnt_sfc_starved_before_serving)+\
                                ' overall_cnt_sfc_starved_before_serving: ' + str(overall_cnt_sfc_starved_before_serving)+\
                                ' epi_cnt_sfc_Prclass_starved_before_serving: ' + str( epi_cnt_sfc_Prclass_starved_before_serving)+\
                                ' overall_cnt_sfc_Prclass_starved_before_serving: ' + str(overall_cnt_sfc_Prclass_starved_before_serving)+\
                                ' copy_overall_epi_pr: ' + str(copy_overall_epi_pr)+\
                                ' overall_epi_pr: ' + str(overall_epi_pr)+\
                                ' overall_epi_pr_emb: ' + str(overall_epi_pr_emb)
                                #'over_count_micro_pr:' +str(over_count_micro_pr)
                               ]
                    
                    #print("overall_emb_sfc: ", overall_emb_sfc, " overall_emb_sfc_h_rel:", overall_emb_sfc_h_rel)
                    #print("overall_ar_sorted_sch_list: ", overall_ar_sorted_sch_list)
                    embeded_sfc = ['Emb_SFC: ' +\
                                   'overall_emb_sfc: '+ str(overall_emb_sfc) +\
                                   'overall_UNemb_sfc: '+ str(overall_unemb_sfc) +\
                                   'overall_emb_sfc_h_rel:' + str(overall_emb_sfc_h_rel) +\
                                   'overall_ar_sorted_sch_list: ' +str(overall_ar_sorted_sch_list) +\
                                   'overall_epi_cnt_High_pr_emb: ' + str(overall_epi_cnt_High_pr_emb) +\
                                   'overall_epi_cnt_Medium_pr_emb: ' + str(overall_epi_cnt_Medium_pr_emb) +\
                                   'overall_epi_cnt_Low_pr_emb: ' +str(overall_epi_cnt_Low_pr_emb)
                                  ]
                    

                    
                    traffic_part = ['Traffic: '+ '_clustermodel_' +str(Switch_cluster_info) + '_classifier_' +str(Switch_classifier_info)+\
                                    #'TRAINING: overall_epi_traffic_load_dict: ' + str(overall_epi_traffic_load_dict) +\
                                    #'TRAINING: overall_epi_traffic_packets_dict: '+str(overall_epi_traffic_packets_dict) +\
                                    #' ==> classifier_measurements: ' +str(classifier_measurements) +\
                                    ' ==> overall_epi_traffic_class_gen: '+ str(overall_epi_traffic_class_gen) +\
                                    ' ==> overall_epi_predicted_traffic_classes_dict'+ str(overall_epi_predicted_traffic_classes_dict) +\
                                    ' ==> overall_epi_sfc_pr_traffic_class_emb'+ str(overall_epi_sfc_pr_traffic_class_emb) +\
                                    ' ==> overall_epi_sfc_pr_traffic_class_all' + str(overall_epi_sfc_pr_traffic_class_all) +\
                                    ' ==> overall_epi_cnt_High_traffic0_emb: ' + str(overall_epi_cnt_High_traffic0_emb) +\
                                    ' ==> overall_epi_cnt_Medium_traffic0_emb: ' + str(overall_epi_cnt_Medium_traffic0_emb) +\
                                    ' ==> overall_epi_cnt_Low_traffic0_emb: ' + str(overall_epi_cnt_Low_traffic0_emb) +\
                                    ' ==> overall_epi_cnt_High_traffic1_emb: ' + str(overall_epi_cnt_High_traffic1_emb) +\
                                    ' ==> overall_epi_cnt_Medium_traffic1_emb: ' +str( overall_epi_cnt_Medium_traffic1_emb) +\
                                    ' ==> overall_epi_cnt_Low_traffic1_emb: ' +str(overall_epi_cnt_Low_traffic1_emb) +\
                                    ' overall_epi_cnt_low_pr_traffic_1: ' +str(overall_epi_cnt_low_pr_traffic_1)
                                     ]

                    #print("traffic_part: ",traffic_part)
                    data = [Info] + [para] + [Topology_resource_init] + [Nodal_outage_para] + [computation_time] + [results] + \
                           [resource_residual] + [extra_note] + [Bound_Cal] + [priority]+[embeded_sfc] +[traffic_part]              
                    #print(data)
                    
                    dt = datetime.now()
                    # getting the timestamp
                    ts = datetime.timestamp(dt)
                    # convert to datetime
                    date_time = datetime.fromtimestamp(ts)
                    # convert timestamp to string in dd-mm-yyyy HH:MM:SS
                    str_date_time = date_time.strftime("%d-%m-%Y-%H-%M-%S")
                    #print(str_date_time)
                    
                    
                    fname = str(trial)+'_'+ str(topology_name)+\
                            '_cpucore_' + str(core_cpu)+'_num_vnf_per_core_'+\
                            str(num_vnf_per_core) +\
                            '_mean_' + str(round(mean_network_overall_accepted,3))+\
                            '_sfc_' + str(num_sfc)+\
                            '_epi_'+str(num_episode)+ '_runtime_' +str(runtime) +\
                            '_clustermodel_' +str(Switch_cluster_info) +\
                            '_classifier_' +str(Switch_classifier_info)+\
                            '_switch_sfc_expiring_check_' +str(switch_sfc_expiring_check) +\
                            str(str_date_time)
                    
                           # '_switch_reconst_' +str(switch_reconst) +\
                            #'_switch_decomp_' + str(switch_decomp) + \
                            #'_switch_priorty_' + str(switch_priorty)+ \
                            #'_pr_model_'+ str(pr_model)+\
                            #'_sch_model_'+ str(sch_model)
                            
                    print(fname)
                    #f = open('Data_storage/epi_2000_100sfc_agglo_DT/'+fname,"a")
                    #f.write(str(data))
                    #f.close()
                    #print("XXXXXXXXXXXXXXXXX")#

        if len(overall_sfc_priority) != len(global_services.vNetwork_service_details_er):
            status_cnt_ridge = False
        else:
            status_cnt_ridge = True
            #print("XXXX")
        #print("overall_ar_sorted_sch_list: ",overall_ar_sorted_sch_list)   
        #print("overall_epi_pr: ", overall_epi_pr)
        #print("")
        #print("Copy_overall_epi_pr: ", copy_overall_epi_pr)
        #print("overall_epi_pr_emb: ", overall_epi_pr_emb)
        #print("overall_sfc_priority: ", overall_sfc_priority)
        sch_model_overall_cnt_sfc_starved_before_serving[sch_model] = overall_cnt_sfc_starved_before_serving    
        sch_model_overall_cnt_sfc_Prclass_starved_before_serving[sch_model] = overall_cnt_sfc_Prclass_starved_before_serving 
        #print(sch_model_overall_cnt_sfc_starved_before_serving)
        #print(sch_model_overall_cnt_sfc_Prclass_starved_before_serving)
        

Trial_DDQN_100__Netrail__cpucore_12_num_vnf_per_core_4_saving_services
XXXXXXXXXXXXXXXXX

Start Time: 1701024600.6999483
activate agent DDQN
DDQN Linear agent
ridge
DDPG agent

Episode:  0
sfc_poisson:  [51]
-> 51
sfc_poisson:  [55]
-> 55
sfc_poisson:  [47]
-> 47
sfc_poisson:  [53]
-> 100
sfc_poisson:  [58]
-> 105
sfc_poisson:  [63]
-> 110
sfc_poisson:  [53]
-> 100
sfc_poisson:  [53]
-> 100
sfc_poisson:  [57]
-> 104
sfc_poisson:  [48]
-> 95
sfc_poisson:  [55]
-> 102
sfc_poisson:  [56]
-> 103
sfc_poisson:  [57]
-> 104
sfc_poisson:  [58]
-> 105
sfc_poisson:  [58]
-> 105
sfc_poisson:  [53]
-> 100
sfc_poisson:  [48]
-> 95
sfc_poisson:  [53]
-> 100
sfc_poisson:  [61]
-> 108
sfc_poisson:  [44]
-> 91
sfc_poisson:  [38]
-> 85
sfc_poisson:  [48]
-> 95
sfc_poisson:  [42]
-> 89
sfc_poisson:  [31]
-> 78
sfc_poisson:  [36]
-> 83
sfc_poisson:  [54]
-> 101
sfc_poisson:  [44]
-> 91
sfc_poisson:  [58]
-> 105
sfc_poisson:  [36]
-> 83
sfc_poisson:  [59]
-> 106
sfc_poisson:  [48]
-> 95
sfc_poisson:  [45]


<Figure size 432x288 with 0 Axes>

## Without HD services

In [ ]:
print(fname)
fname = "Netrail_cpucore_12_4_mean_19.626_sfc_100_epi_2000_runtime_73311.93_clustermodel_['brich']_classifier_Support Vector Machines_switch_sfc_expiring_check_False01-04-2023"
#"Netrail_12_8_100epi_cluster_brich_classfier_Support Vector Machine_admission-control_false_mean_40.615"

#f = open('Data_storage/without_admission_controller/'+fname,"a")
#f.write(str(data))
#f.close()
#print("XXXXXXXXXXXXXXXXX")#

In [ ]:


switch_clustering = False

if __name__ == '__main__': 
    sch_model_overall_cnt_sfc_starved_before_serving, sch_model_overall_cnt_sfc_Prclass_starved_before_serving = {},{}
    overall_sfc_arrival_detail,  overall_sfc_arrival_num = [],[]
    overall_sfc_priority, copy_overall_epi_pr, overall_episode_envpriority  = [], [],[]
    copy_overall_episode_envpriority = []
    #sclassifier_models ={}
    
    #saving the arrival services
    saving_services = global_services.vNetwork_service_details_er
    #print("saving_services: ", saving_services)
    f_name = str(trial)+'_'+ str('_Netrail_')+\
            '_cpucore_' + str(arr_core_cpu[0])+'_num_vnf_per_core_'+\
            str(arr_num_vnf_per_core[0]) + str('_saving_services')
    print(f_name)
    #f = open('Data_storage/epi_2000_100sfc_agglo_DT/'+f_name,"a")
    #f.write(str(saving_services))
    #f.close()
    print("XXXXXXXXXXXXXXXXX")
        
    status_cnt_ridge = False
    for sch_model in ar_sch_model:
        #print("______________________")
        #print("sch_model: ", sch_model, status_cnt_ridge)
        sch_model_overall_cnt_sfc_starved_before_serving[sch_model] = None
        sch_model_overall_cnt_sfc_Prclass_starved_before_serving [sch_model] = None
        env = substrate_network()
        reward_func = Reward_Function()
        temp_vals_sorted_action, temp_norm_vals_sorted_action = [], []
        min_pr,max_pr = 0, 6
        t_max, t_min, old_t_current = None, 0, 0
        mvf_t_max, mvf_t_min, mvf_old_t_current = None, 0, 0
        switch_or_run_once = False
        overall_emb_sfc, overall_emb_sfc_h_rel = [], []
        overall_unemb_sfc = []
        overall_ar_sorted_sch_list =[]

        # NODAL OUTAGE
        for nodal_outage_percentage in arr_nodal_outage_percentage:
            nodal_outage = (env.network_len_nodes*nodal_outage_percentage)/100
            round_nodal_outage = round(nodal_outage)

            # CPU CORE
            for core_cpu in arr_core_cpu:

                #VNF PER CPU CORE
                for num_vnf_per_core in arr_num_vnf_per_core:
                    tot_vnf_per_core = core_cpu*num_vnf_per_core
                    tot_vnf_network = tot_vnf_per_core*env.network_len_nodes
                    pr_epsilon, epsilon = 1, 1
                    eps_history,network_overall_accepted_action = [],[]
                    network_overall_accepted, network_overall_rejected = [],[]
                    monolithic_deploy_success, monolithic_deploy_rejected = [], []
                    overall_res_exhausted = []
                    network_overall_count_mvnf, network_overall_sum_mvnf = [],[]
                    network_overall_tot_decompsfc, network_overall_count_succes_decompvnf,  = [], []
                    network_overall_count_unsucces_decompvnf, overall_remain_cpu, overall_remain_bw = [], [], []
                    overall_count_numVNF_perNode, overall_count_num_mVNF_perNode = [], []
                    network_overall_epsilon, network_overall_sum_rewards_perepi = [], []
                    overall_episode_priority,  overall_episode_priority_err= [], []
                    overall_episode_pr_rewardfunc = []
                    over_count_macro_pr, over_count_micro_pr = [], []
                    overall_epi_pr,overall_epi_pr_emb = [],[]
                    overall_epi_traffic_packets_dict, overall_epi_traffic_load_dict = {}, {}
                    start = time.time()
                    print("")
                    print("Start Time:",start)
                    #overall runtime these needs to be initiated
                    n_state = [8]
                    state_memory      = np.zeros((mem_size, *n_state), dtype = np.float32) # size of mem size X input shape
                    action_memory     = np.zeros(mem_size, dtype = np.int64)
                    reward_memory     = np.zeros(mem_size, dtype = np.float32)
                    next_state_memory = np.zeros((mem_size, *n_state), dtype = np.float32)
                    terminal_memory   = np.zeros(mem_size, dtype = np.bool)
                    vnf_cnt = 0
                    temp_vals_sorted_action, temp_norm_vals_sorted_action = [], [] 
                    if model == 'DDQN':
                        print("activate agent DDQN")
                        #agent
                        agent = LinearDDQNAgent(n_state = n_state, 
                                                n_actions = env.network_len_nodes, 
                                                lr = learning_rate, 
                                                epsilon = epsilon,
                                                mem_size = mem_size,
                                                batch_size= batch_size,
                                                vnf_cnt = vnf_cnt,
                                                gamma =gamma,
                                                eps_dec =eps_dec,
                                                eps_min =eps_min,
                                                replace = replace,
                                                model = model,
                                                state_memory = state_memory,
                                                action_memory = action_memory,
                                                reward_memory = reward_memory,
                                                next_state_memory = next_state_memory,
                                                terminal_memory = terminal_memory,
                                                Neurons_per_HiddenLayer = Neurons_per_HiddenLayer,
                                                drop_prob =drop_prob,
                                                chkpt_dir = 'models/',
                                                algo = 'DDQNAgent',
                                                env_name = 'BtEurope')
                    elif model == 'DQN':
                        agent = LinearDQNAgent(n_state = n_state, 
                                               n_actions = env.network_len_nodes, 
                                               epsilon = epsilon, 
                                               lr = learning_rate, 
                                               vnf_cnt = vnf_cnt, 
                                               Neurons_per_HiddenLayer = Neurons_per_HiddenLayer,
                                               drop_prob =drop_prob,
                                               gamma =gamma,
                                               eps_dec =eps_dec,
                                               eps_min =eps_min, 
                                               model = model,
                                               algo = None, 
                                               env_name = None,
                                               chkpt_dir ='tmp/dqn')
                    #added on 01032022 Heuristic model
                    else:
                        print("Activate Heuristic model")
                        agent = HeuAgent(n_state = n_state, 
                                         n_actions = env.network_len_nodes)  
                        
                    cnt_ridge = 0
                    # Priority
                    if switch_priorty is True:
                        pr_mem_size = 5000
                        pr_n_state = [3]  #  waiting time, delay, jitter, Packetloss,
                        pr_state_input = 'waiting time, delay, jitter, Packetloss'
                        # [episode, sfc, flow behaviour, delay, jitter, Packetloss, reliability(removed), wating time]
                        pr_n_actions = 1
                        pr_state_memory      = np.zeros((pr_mem_size, *pr_n_state), dtype = np.float32) # size of mem size X input shape
                        pr_reward_memory     = np.zeros(pr_mem_size, dtype = np.float32)
                        pr_next_state_memory = np.zeros((pr_mem_size, *pr_n_state), dtype = np.float32)
                        pr_terminal_memory   = np.zeros(pr_mem_size, dtype = np.bool)
                        sfc_cnt = 0
                        if pr_model == 'DDPG':
                            print("pr_model: ",pr_model)
                            pr_action_memory = np.zeros((pr_mem_size, pr_n_actions), dtype = np.float32)
                            pr_agent = DDPG_Agent(pr_n_state       = pr_n_state, 
                                                  pr_n_actions     = pr_n_actions,
                                                  alpha            = alpha, 
                                                  beta             = beta,
                                                  tau              = tau, 
                                                  sfc_cnt          = sfc_cnt,
                                                  mem_size         = mem_size,
                                                  batch_size       = batch_size,
                                                  gamma            = gamma,
                                                  pr_model         = pr_model,
                                                  pr_state_memory  = pr_state_memory,
                                                  pr_action_memory = pr_action_memory,
                                                  pr_reward_memory = pr_reward_memory,
                                                  pr_next_state_memory = pr_next_state_memory,
                                                  pr_terminal_memory = pr_terminal_memory,
                                                  Neurons_per_HiddenLayer = Neurons_per_HiddenLayer,
                                                  chkpt_dir = 'models/',
                                                  algo = 'DDPGAgent',
                                                  env_name = 'Netrail')
                        elif pr_model == 'DDQN':
                            print(pr_model)
                            pr_action_memory = np.zeros((pr_mem_size), dtype = np.float32)
                            pr_agent = PR_DDQNAgent(pr_n_state       = pr_n_state,
                                                 pr_n_actions     = pr_n_actions,
                                                 lr               = learning_rate, 
                                                 pr_epsilon       = pr_epsilon,
                                                 mem_size         = mem_size,
                                                 batch_size       = batch_size,
                                                 sfc_cnt          = sfc_cnt,
                                                 gamma            = gamma,
                                                 eps_dec          = eps_dec,
                                                 eps_min          = eps_min,
                                                 replace          = replace,
                                                 pr_model         = pr_model,
                                                 pr_state_memory  = pr_state_memory,
                                                 pr_action_memory = pr_action_memory,
                                                 pr_reward_memory = pr_reward_memory,
                                                 pr_next_state_memory = pr_next_state_memory,
                                                 pr_terminal_memory = pr_terminal_memory,
                                                 Neurons_per_HiddenLayer = Neurons_per_HiddenLayer,
                                                 drop_prob =drop_prob,
                                                 chkpt_dir = 'models/',
                                                 algo = 'PR_DDQNAgent',
                                                 env_name = 'Netrail')    
                        elif pr_model == 'ridge':
                            print(pr_model)
                            #Lasso model
                            alpha = 0.0001
                            input_train_data,target_train_data = [], [] 
                            input_test_data, target_test_data  = [], []
                            input_train_mem     = np.zeros((mem_size, *pr_n_state), dtype = np.float32)
                            target_train_mem    = np.zeros(mem_size, dtype = np.float32)
                            input_test_mem      = np.zeros((mem_size, *pr_n_state), dtype = np.float32)
                            target_test_mem     = np.zeros(mem_size, dtype = np.float32)
                            # define model
                            ridge_model = Ridge(alpha=alpha, normalize = True) #Lasso(alpha=alpha, normalize = True)
                            cv = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeat, random_state=1)
                            done_status = "Train"
                        else: 
                            #print("here")
                            print(pr_model)
                            hidden_neuron = 20
                            input_train_data,target_train_data = [], [] 
                            input_test_data, target_test_data  = [], []
                            pr_MLP_model = MLP(pr_n_state = pr_n_state, 
                                               pr_n_actions = pr_n_actions, 
                                               hidden_neuron = hidden_neuron)
                            input_train_mem     = np.zeros((mem_size, *pr_n_state), dtype = np.float)
                            target_train_mem    = np.zeros(mem_size, dtype = np.float)
                            input_test_mem      = np.zeros((mem_size, *pr_n_state), dtype = np.float)
                            target_test_mem     = np.zeros(mem_size, dtype = np.float)
                            done_status = "Train"
                            
                    
                    # Admission control: Scheduling 
                    if switch_scheduling is True:
                        sch_mem_size = 5000
                        sch_n_state = [4]  #  waiting time, delay, jitter, Packetloss,
                        sch_state_input = 'waiting time, reliability, priority'
                        # [episode, sfc, flow behaviour, delay, jitter, Packetloss, reliability(removed), wating time]
                        sch_n_actions = 1
                        sch_n_tempsfc = [10]
                        sch_state_memory      = np.zeros((sch_mem_size,*sch_n_state), dtype = np.float32) # size of mem size X input shape
                        #print(sch_state_memory)
                        sch_reward_memory     = np.zeros(sch_mem_size, dtype = np.float32)
                        sch_next_state_memory = np.zeros((sch_mem_size, *sch_n_state), dtype = np.float32)
                        sch_terminal_memory   = np.zeros(sch_mem_size, dtype = np.bool)
                        sfc_cnt = 0
                        if sch_model == 'DDPG':
                            #print("sch_model", sch_model)
                            sch_action_memory     = np.zeros((sch_mem_size, sch_n_actions), dtype = np.float32)
                            sch_agent = DDPG_Agent(sch_n_state      = sch_n_state, 
                                                   sch_n_actions    = sch_n_actions,
                                                   alpha            = alpha, 
                                                   beta             = beta,
                                                   tau              = tau, 
                                                   sfc_cnt          = sfc_cnt,
                                                   mem_size         = mem_size,
                                                   batch_size       = batch_size,
                                                   gamma            = gamma,
                                                   sch_model        = sch_model,
                                                   sch_state_memory  = sch_state_memory,
                                                   sch_action_memory = sch_action_memory,
                                                   sch_reward_memory = sch_reward_memory,
                                                   sch_next_state_memory = sch_next_state_memory,
                                                   sch_terminal_memory = sch_terminal_memory,
                                                   Neurons_per_HiddenLayer = Neurons_per_HiddenLayer,
                                                   chkpt_dir = 'models/',
                                                   algo = 'DDPGAgent',
                                                   env_name = 'Netrail')
                        else:
                            pass

                    a = 0
                    cnt_tot_sfc , tr_cnt_tot_sfc, eval_cnt_tot_sfc, pred_cnt_tot_sfc= 0, 0, 0, 0
                    count_macro_pr, count_micro_pr = 0, 0
                    tr_count_macro_pr, tr_count_micro_pr = 0, 0
                    eval_count_macro_pr, eval_count_micro_pr = 0, 0
                    pred_count_macro_pr, pred_count_micro_pr = 0, 0
                    tr_mem_cnt, test_mem_cnt = 0, 0
                    input_train_traffic_mem_cnt, test_traffic_mem_cnt = 0, 0
                    epi_tr_correct_perc,  epi_pred_correct_perc = [], []
                    overall_deployed_sfc_per_episode, overall_deployed_priority_per_episode = [], []
                    epi_cnt_High_pr, epi_cnt_Medium_pr, epi_cnt_Low_pr = [], [], []
                    epi_cnt_High_pr_emb, epi_cnt_Medium_pr_emb, epi_cnt_Low_pr_emb = [], [], []
                    overall_cnt_sfc_starved_before_serving, overall_cnt_sfc_Prclass_starved_before_serving = [], []
                    epi_cnt_sfc_starved_before_serving, epi_cnt_sfc_Prclass_starved_before_serving = [], []
                    epi_expected_wt_B = {}
                    

                    for per_episode in global_services.vNetwork_service_details_er: 
                        episode = a
                        #print("")
                        #print("Episode: ", episode)
                        action, vnf_res_req = False, False
                        pref_node_val = False
                        cnt_macro_pr, epi_cnt_macro_pr = 0,0 
                        cnt_High_pr, cnt_Medium_pr, cnt_Low_pr = 0, 0, 0
                        cnt_High_pr_emb, cnt_Medium_pr_emb, cnt_Low_pr_emb = 0, 0, 0
                        epi_pr = {"H": None,"M": None, "L": None}
                        epi_pr_emb = {"H": None,"M": None, "L": None}
                        cnt_sfc_starved_before_serving = 0
                        cnt_rel = 0
                        epi_classification_grp_acc = {}
                        kmeans_runtime, Aggo_runtime, birch_runtime, GM_runtime = [],[],[],[]
                        traffic_input_dataset = []
                        sfc_starved_before_serving, sfc_Prclass_starved_before_serving = [], []
                        nodal_actionspace, removed_nodes = [], []
                        epi_emb_sfc, epi_emb_sfc_h_rel = [], []
                        traffic_sfc_arrival = []
                        traffic_transition_dict, cluster_label_val = {}, {}
                        arr_traffic_packet_per_epi = []
                        epi_unemb_sfc = []
                        epi_temp_sfc_arrived = []
                        
                        episode_priority, episode_priority_err, episode_envpriority, episode_pr_rewardfunc = [], [], [], []
                        actionspace = [i for i in range(env.network_len_nodes)]
                        original_actionspace = actionspace
                        # to remove certain number of the nodes from the network when outage happens
                        for key in range(round_nodal_outage):
                            random_action = random.choice(actionspace)
                            actionspace.remove(random_action)
                            removed_nodes.append(random_action)                 
                        #----Resource Initialization for the network Network----#
                        env.substrate_node_init(removed_nodes, tot_vnf_per_core)
                        env.substrate_link_init()
                        if switch_priorty is True and pr_model == 'DDPG':
                            pr_agent.noise.reset()
                        else:
                            pass
                        #---- End of Initialization of Resource ----#
                        count_accpeted, count_rejected, count_success,count_failed  = 0,0,0,0
                        reward_per_episode, sum_reward_per_episode = [], []
                        deployed_sfc_per_episode, deployed_priority_per_episode = [], []
                        micro_vnf_collection = []
                        tot_decompsfc = 0
                        undecomp_accepted = 0
                        res_exhausted = 0
                        count_mvnf_perepi =[]
                        sum_count_mvnf_perepi =0
                        sum_rewards_perepi = 0
                        count_succes_decompvnf, count_unsucces_decompvnf = 0, 0
                        nodal_avail = 1 #at the start of the episode the network will be fully available.
                        rewards_perepi = []
                        std_value = 1
                        
                        # Priority!
                        epi_tr_count_macro_pr,   epi_tr_count_micro_pr   = 0, 0
                        epi_eval_count_macro_pr, epi_eval_count_micro_pr = 0, 0
                        epi_pred_count_macro_pr, epi_pred_count_micro_pr = 0, 0
                        
                        # arrival distriution of SFC using Poisson dis.
                        if len(overall_sfc_arrival_num) != len(global_services.vNetwork_service_details_er):
                            arr_sfc_dist = []
                            class_arr_sfc_dist = arrival_sfc(num_sfc, arr_sfc_dist)
                            arr_sfc_dist = list(class_arr_sfc_dist.sfc_poisson_dist())
                            #print("==> SFC dist: ",arr_sfc_dist)
                            sfc_arrival_detail, sfc_arrival_num = [], []
                            arr_sfc_detail = class_arr_sfc_dist.sfc_details(per_episode, 
                                                                            arr_sfc_dist, 
                                                                            sfc_arrival_detail, 
                                                                            sfc_arrival_num)
                            overall_sfc_arrival_detail.append(sfc_arrival_detail)
                            overall_sfc_arrival_num.append(sfc_arrival_num)
                            #print("sfc_arrival_num: ", sfc_arrival_num)

                        else: 
                            pass
                        #print("Length of overall_sfc_arrival_num: ",len(overall_sfc_arrival_num))
                        #print("overall_sfc_arrival_num: ", overall_sfc_arrival_num)
                        sfc_arrival_detail = overall_sfc_arrival_detail[episode]
                        sfc_arrival_num = overall_sfc_arrival_num[episode]
                        sfc_priority, sfc_priority_err, sfc_envpriority, sfc_pr_rewardfunc = [], [], [], []
                        done_pr   = False
                        ar_sfc_current_waitingtime = {}
                        ar_sorted_sch_list =[]
                        cnt_poisson_dist = 0
                        for arr_sfc_de in sfc_arrival_detail:
                            ar_sfc_start = time.perf_counter() # step 1
                            cnt_sfc_arrival = 0
                            temp_my_dict = {}
                            ar_starving_sfc = []
                            #print("")
                            #print("cnt_poisson_dist: ", cnt_poisson_dist, " arr num: ", sfc_arrival_num[cnt_poisson_dist])
                            if sch_model == 'WFQ':
                                h_pr_queue, m_pr_queue, l_pr_queue = [], [], []
                            if not sfc_arrival_num[cnt_poisson_dist]:
                                sorted_sch_list =[]
                            ar_sfc_end = time.perf_counter() #step 1
                            expected_wt_A = round(ar_sfc_end - ar_sfc_start,6)

                            for arv_per_sfc, arv_per_link in arr_sfc_de:
                                sfc = arv_per_sfc[0][1]['sfc']
                                #print("arv_per_sfc: ", arv_per_sfc)
                                #print("SFC: ", sfc)
                                ar_link_bw = []
                                for link in arv_per_link:
                                    temp_bw = link[2]['bw']
                                    ar_link_bw.append(temp_bw)
                                ar_sfc_cpu = []
                                for temp_sfc in arv_per_sfc:
                                    temp_cpu = temp_sfc[1]['cpu_req']
                                    ar_sfc_cpu.append(temp_cpu)     
                                # Priority for Arriving SFC
                                env_priority = None
                                env_priority = arv_per_sfc[0][1]['env_priority']
                                env_priority = env_priority - env_priority % 0.01
                                done_sch = False
                                A_cost, B_cost, S_cost = 0,0,0
                                reward_pts = 1000
                                #print("SFC: ", sfc)
                                if status_cnt_ridge is False:
                                    #print("pr_model: ", pr_model)
                                    if switch_priorty is True:
                                            pr_sfc_start = time.perf_counter()
                                            threshold_latency          = arv_per_sfc[0][1]['norm_delay'] 
                                            threshold_jitter           = arv_per_sfc[0][1]['norm_jitter']     
                                            threshold_packetloss       = arv_per_sfc[0][1]['norm_packetloss'] 
                                            threshold_theo_wating_time = arv_per_sfc[0][1]['norm_waiting_time'] 
                                            
                                            # creating priority observation
                                            pr_obs = [threshold_latency, threshold_jitter, threshold_packetloss]
                                            if pr_model == 'ridge':
                                                X = pr_obs
                                                Y = [env_priority]
                                                sfc_envpriority.append(env_priority)
                                                env_pr_level = truncate(Y[0],2)
                                                if env_pr_level == 1:
                                                    macro_env_pr_level = int(truncate(env_pr_level, 0))
                                                    micro_env_pr_level = round((truncate(env_pr_level, 1)- macro_env_pr_level),2)
                                                else:
                                                    macro_env_pr_level = int(truncate(env_pr_level*10, 0))
                                                    micro_env_pr_level = int(truncate(env_pr_level*100, 1))- macro_env_pr_level*10
                                                if done_status == "Train": 
                                                    tr_cnt_tot_sfc += 1
                                                    train_dataset = prepare_train_data(input_train_mem, target_train_mem, 
                                                                                       input_train_data, target_train_data,
                                                                                       X, Y, 
                                                                                       tr_mem_cnt, tr_mem_size, tr_batch_size)
                                                    input_train_data = train_dataset[0]
                                                    target_train_data = train_dataset[1]
                                                    tr_mem_cnt = train_dataset[2]
                                                    #print("train_dataset: ", train_dataset)
                                                    if tr_cnt_tot_sfc < tr_batch_size:
                                                        temp_priority_level = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
                                                        priority_level  = random.uniform(0,1) #np.random.choice(temp_priority_level)
                                                        #print("priority_level: ", priority_level, type(priority_level))
                                                    else:
                                                        # evaluate model
                                                        scores = cross_val_score(ridge_model, input_train_data, target_train_data, 
                                                                                 scoring='neg_mean_absolute_error', 
                                                                                 cv=cv, n_jobs= 1)
                                                        # force scores to be positive
                                                        scores = absolute(scores)
                                                        # fit model
                                                        ridge_model.fit(input_train_data, target_train_data)
                                                        # make a prediction
                                                        yhat = ridge_model.predict([X])
                                                        # summarize prediction
                                                        priority_level = yhat[0]
                                                        #print("priority_level: ", priority_level, type(priority_level))

                                                    #classifing the priority levels into three classess.
                                                    if 0.8<=priority_level and priority_level<=1.0:
                                                        priority_class = 'high'
                                                        cnt_High_pr +=1
                                                    elif 0.4<=priority_level and priority_level<=0.7:
                                                        priority_class = 'medium'
                                                        cnt_Medium_pr +=1
                                                    else: 
                                                        priority_class = 'low'   
                                                        cnt_Low_pr +=1
                                                    sfc_priority.append(priority_level)
                                                    global_services.priority_update(episode, sfc, 
                                                                                    vnf_per_service, 
                                                                                    priority_level, 
                                                                                    priority_class)
                                                    if priority_level == 1:
                                                        macro_priority_level = 1 #int(truncate(priority_level, 0))
                                                        micro_priority_level = 0 #round((truncate(priority_level, 1)- macro_priority_level),2)
                                                    else:
                                                        macro_priority_level = int(truncate(priority_level*10, 0))
                                                        micro_priority_level = int(truncate(priority_level*100, 1))- macro_priority_level*10
                                                    if macro_priority_level == macro_env_pr_level:
                                                        #first level clearance
                                                        tr_count_macro_pr += 1
                                                        epi_tr_count_macro_pr += 1
                                                        if micro_priority_level == micro_env_pr_level:
                                                            tr_count_micro_pr += 1
                                                            epi_tr_count_micro_pr += 1
                                                    tr_macro_acc = round(((tr_count_macro_pr/tr_cnt_tot_sfc)*100), 2)
                                                    tr_micro_acc = round(((tr_count_micro_pr/tr_cnt_tot_sfc)*100), 2)
                                                    if a == 0:
                                                        pass
                                                    else:
                                                        if (tr_macro_acc/100) >= acc_theshold_limit:
                                                            #done_status == "Train"
                                                            done_status = "Predict"
                                                            #print("Train --> Predict", done_status, cnt_tot_sfc, tr_cnt_tot_sfc,
                                                            #      " macro acc: ", tr_macro_acc, " micro acc: ",tr_micro_acc, tr_count_micro_pr)
                                                else:
                                                    pred_cnt_tot_sfc +=1
                                                    # make a prediction
                                                    yhat = ridge_model.predict([X])
                                                    # summarize prediction
                                                    priority_level = yhat[0]
                                                    #print("priority_level: ", priority_level, type(priority_level))
                                                    #classifing the priority levels into three classess.
                                                    if 0.8 <= priority_level and priority_level <= 1.0: #between 0.7 - 1.0
                                                        priority_class = 'high' 
                                                        cnt_High_pr +=1
                                                    elif 0.4 <= priority_level and priority_level <= 0.7:  #between 0.4 - 0.7
                                                        priority_class = 'medium'
                                                        cnt_Medium_pr +=1
                                                    else: 
                                                        priority_class = 'low' #between 0.0 - 0.3
                                                        cnt_Low_pr +=1
                                                    sfc_priority.append(priority_level)
                                                    global_services.priority_update(episode, sfc, vnf_per_service, 
                                                                                    priority_level, priority_class)
                                                    if priority_level == 1:
                                                        macro_priority_level = 1 #int(truncate(priority_level, 0))
                                                        micro_priority_level = 0 #round((truncate(priority_level, 1)- macro_priority_level),2)
                                                    else:
                                                        macro_priority_level = int(truncate(priority_level*10, 0))
                                                        micro_priority_level = int(truncate(priority_level*100, 1))- macro_priority_level*10
                                                        #micro_env_pr_level = round(micro_env_pr_level,2)
                                                    if macro_priority_level == macro_env_pr_level:
                                                        #first level clearance
                                                        pred_count_macro_pr += 1
                                                        epi_pred_count_macro_pr += 1
                                                        if micro_priority_level == micro_env_pr_level:
                                                            pred_count_micro_pr += 1
                                                            epi_pred_count_micro_pr += 1
                                                    pred_macro_acc =round(((pred_count_macro_pr/pred_cnt_tot_sfc)*100),2)
                                                    pred_micro_acc =round(((pred_count_micro_pr/pred_cnt_tot_sfc)*100),2)
                                                    if (pred_macro_acc/100) >= acc_theshold_limit:
                                                        done_status = "Done"
                                                    if overall_sfc == cnt_tot_sfc:
                                                        pred_macro_acc =round(((pred_count_macro_pr/pred_cnt_tot_sfc)*100),2)
                                                        pred_micro_acc =round(((pred_count_micro_pr/pred_cnt_tot_sfc)*100),2)
                                            else: 
                                                priority_level = arv_per_sfc[0][1]['env_priority']
                                            #print("SFC: ", sfc, "priority_level: ", priority_level)
                                            pr_sfc_end = time.perf_counter()
                                            expected_wt_B = round((pr_sfc_end - pr_sfc_start), 6) # step2
                                            #print("expected_wt_B: ", expected_wt_B)
                                            epi_expected_wt_B[sfc] = expected_wt_B
                                else:           
                                    pass
                                
                                if switch_scheduling is True:
                                    #print("switch_scheduling: ", switch_scheduling)
                                    #print("episode: ", episode)
                                    sch_sfc_start = time.perf_counter()
                                    if sch_model == 'DDPG':
                                        #print("sch_model: ", sch_model)
                                        sch_obs = []
                                        threshold_priority_level    = arv_per_sfc[0][1]['priority_level']
                                        threshold_reliability       = arv_per_sfc[0][1]['reliability']
                                        threshold_theo_wating_time  = arv_per_sfc[0][1]['theo_wating_time']
                                        norm_waitingtime            = arv_per_sfc[0][1]['norm_waiting_time']
                                        threshold_reliability       = arv_per_sfc[0][1]['reliability']
                                        threshold_evnpriority_level = arv_per_sfc[0][1]['env_priority']
                                        threshold_int_flowtype      = arv_per_sfc[0][1]['int_flow_behaviour']
                                        current_starving_sfc = None
                                        intersec_sfc = None
                                        starving_sfc_index = None
                                        if threshold_priority_level <= 0.2:
                                            current_starving_sfc = sfc
                                            ar_starving_sfc.append(sfc)
                                            
                                        traffic_input_dataset,  traffic_output_dataset = [], []
                                        class_traffic_classifer = traffic_classifer(traffic_input_dataset, 
                                                                                    traffic_output_dataset, 
                                                                                    episode, cluster_testing)
                                        
                                        classifier_datasets = [None, None, None, None]
                                        if switch_clustering is True:
                                            #print( "==>", switch_clustering, classifier_training_status)
                                            mu, std = 100, 300
                                            pred_traffic_num_sfc_requested = abs(random.normal(mu, std, 1))
                                            while pred_traffic_num_sfc_requested < 1:
                                                pred_traffic_num_sfc_requested = abs(random.normal(mu, std, 1))
                                            pred_num =  abs(int(pred_traffic_num_sfc_requested))
                                            pred_f = FBM(n  = int(pred_num), hurst=0.7, length=length, method='daviesharte')
                                            pred_fbm_sample = abs(pred_f.fbm())
                                            pred_fgn_sample = pred_f.fgn()
                                            pred_t_values   = pred_f.times() #stationary traffic
                                            pred_traffic_packet_per_sfc = sum(pred_t_values) #load
                                            
                                            predict_dataset = [threshold_priority_level, 
                                                               threshold_int_flowtype, pred_num, 
                                                               pred_traffic_packet_per_sfc]
                                            #print("==> predict_dataset", predict_dataset)
                                            #print("classifier_models: ", classifier_models)
                                            Classifier_training = class_traffic_classifer.classifier_models(classifier_models, 
                                                                                                            classifier_datasets,
                                                                                                            predict_dataset,
                                                                                                            classifier_training_status)
                                            
                                            #print("Classifier_training: ", Classifier_training)
                                            #classifier_measurements[clusterModel] = Classifier_training
                                            traffic_pred = Classifier_training[4][0]
                                            #print(traffic_pred)
                                        else: 
                                            #print("no clustering")
                                            traffic_pred = -1
                                            pred_num = -1
                                            pred_traffic_packet_per_sfc = -1
                                            
                                        #print("traffic_pred: ", traffic_pred)   
                                        #updating in global level
                                        global_services.classifier_class_update(episode, sfc, vnf_per_service, 
                                                                                traffic_pred, pred_num,
                                                                                pred_traffic_packet_per_sfc
                                                                               )   
                                        
                                        sch_obs = [threshold_theo_wating_time, 
                                                   threshold_reliability,
                                                   threshold_priority_level, 
                                                   traffic_pred
                                                  ]
                                        sch_rank = sch_agent.choose_action(sch_obs, sch_n_state)
                                        temp_my_dict[sfc] = sch_rank
                                        sorted_sfc_sch = dict(sorted(temp_my_dict.items(), 
                                                                    key=lambda item: item[1], 
                                                                    reverse = True))
                                        sorted_sch_list = list(sorted_sfc_sch.keys())
                                        
                                        if ar_starving_sfc is None or not ar_starving_sfc:
                                            pass
                                        else:
                                            intersec_sfc = set(sorted_sch_list).intersection([current_starving_sfc])
                                            if intersec_sfc is None or not intersec_sfc:
                                                pass
                                            else:
                                                starving_sfc_index = sorted_sch_list.index(current_starving_sfc) 
                                                
                                        if cnt_sfc_arrival == len(sfc_arrival_num[cnt_poisson_dist])-1:
                                            done_sch = True
                                            next_sch_obs = [#sfc, 
                                                            #threshold_evnpriority_level, 
                                                            #threshold_priority_level, 
                                                            threshold_theo_wating_time, 
                                                            threshold_reliability,
                                                            threshold_priority_level,
                                                            traffic_pred
                                                           ]
                                        else:
                                            next_sfc = cnt_sfc_arrival + 1
                                            next_threshold_priority_level    = arr_sfc_de[next_sfc][0][0][1]['priority_level']
                                            next_threshold_reliability       = arr_sfc_de[next_sfc][0][0][1]['reliability']
                                            next_threshold_theo_wating_time  = arr_sfc_de[next_sfc][0][0][1]['theo_wating_time']
                                            next_threshold_evnpriority_level = arr_sfc_de[next_sfc][0][0][1]['env_priority']
                                            next_threshold_int_flowtype      = arr_sfc_de[next_sfc][0][0][1]['int_flow_behaviour']
                                            traffic_pred = -1
                                            next_sch_obs = [next_threshold_theo_wating_time, 
                                                            next_threshold_reliability,
                                                            next_threshold_evnpriority_level, 
                                                            traffic_pred] 
                                        sch_next_state = next_sch_obs
                                        #Reward function
                                        A_cost = (threshold_priority_level*reward_pts) + (norm_waitingtime*reward_pts)
                                        link_cost = 0 #(1/sum(ar_link_bw))*reward_pts
                                        cpu_cost  = 0 #(1/sum(ar_sfc_cpu))*reward_pts
                                        reliability_cost = threshold_reliability*reward_pts
                                        B_cost = link_cost + cpu_cost + reliability_cost
                                        decay = 0.1
                                        alpha = 1
                                        
                                        pos_sfc = starving_sfc_index
                                        if pos_sfc is None:
                                            starvation_rate = 0
                                            pass
                                        else:
                                            starvation_rate = alpha*pow(decay,pos_sfc)
                                        S_cost = starvation_rate*reward_pts
                                        sch_rewardfunc = A_cost + B_cost + S_cost
                                        sch_agent.store_transition(sch_obs, 
                                                                   sch_rank, 
                                                                   sch_rewardfunc, 
                                                                   sch_next_state, 
                                                                   done_sch, 
                                                                   sfc)
                                        sch_agent.learn()
                                        sch_sfc_end = time.perf_counter()

                                    elif sch_model == 'FIFO': 
                                        sorted_sch_list = sfc_arrival_num[cnt_poisson_dist] 
                                        sch_sfc_end = time.perf_counter()
                                    elif sch_model == 'WFQ':
                                        if arv_per_sfc[0][1]['priority_class'] == 'high':
                                            h_pr_queue.append(sfc)
                                        elif arv_per_sfc[0][1]['priority_class'] == 'medium':
                                            m_pr_queue.append(sfc)
                                        else: 
                                            l_pr_queue.append(sfc)
                                        sorted_sch_list = sfc_arrival_num[cnt_poisson_dist]
                                        sch_sfc_end = time.perf_counter()
                                    else:
                                        #print(sch_model)
                                        #print("arv_per_sfc: ", arv_per_sfc[0][1])
                                        temp_my_dict[sfc] = arv_per_sfc[0][1]['priority_level']
                                        #print(temp_my_dict)
                                        sorted_sfc_sch = dict(sorted(temp_my_dict.items(), 
                                                                    key=lambda item: item[1], 
                                                                    reverse = True))
                                        sorted_sch_list = list(sorted_sfc_sch.keys())
                                        sch_sfc_end = time.perf_counter()
                                        #print("sorted_sch_list: ", sorted_sch_list)
                                cnt_sfc_arrival += 1
                            cnt_poisson_dist += 1
                            #print(sch_model)
                            #print("sorted_sch_list: ", sorted_sch_list)
                            
                            # Adaptive scheduling
                            if sch_model == 'DDPG':
                                temp_sfc_deploy = []
                                for w in sorted_sch_list:
                                    temp_var = []
                                    for q, p in arr_sfc_de:
                                        if q[0][1]['sfc'] == w:
                                            temp_var = [q,p]
                                            temp_sfc_deploy.append(temp_var)
                                        else:
                                            pass
                            elif sch_model == 'FIFO': 
                                temp_sfc_deploy = arr_sfc_de
                            elif sch_model == 'WFQ':
                                temp_sfc_deploy = []
                                wfq_sch_sfc_start = time.perf_counter()
                                sorted_sch_list = wfq_sch(arr_sfc_de, h_pr_queue, m_pr_queue, l_pr_queue)
                                wfq_sch_sfc_end = time.perf_counter()
                                for w in sorted_sch_list:
                                    temp_var = []
                                    for q, p in arr_sfc_de:
                                        if q[0][1]['sfc'] == w:
                                            temp_var = [q,p]
                                            temp_sfc_deploy.append(temp_var)
                                        else:
                                            pass
                            else: #priority-based!!
                                temp_sfc_deploy = []
                                for w in sorted_sch_list:
                                    temp_var = []
                                    for q, p in arr_sfc_de:
                                        if q[0][1]['sfc'] == w:
                                            temp_var = [q,p]
                                            temp_sfc_deploy.append(temp_var)
                                        else:
                                            pass
                            
                            if sch_model == 'WFQ':
                                wfq_sch_sfc = wfq_sch_sfc_end - wfq_sch_sfc_start
                            else:
                                wfq_sch_sfc =0
                            
                            sch_sfc_wt = (sch_sfc_end-sch_sfc_start)+wfq_sch_sfc
                            #print("sch_sfc_wt: ", sch_sfc_wt)
                            temp_val = [std_value for i in (range(env.network_len_nodes))]
                            prob_val = [float(i)/sum(temp_val) for i in temp_val]
                            # services will arrive here.
                            num_sfc_per_arrival = 0
                            sfc_waited_time =  0
                            
                            # edited on 15th sept 2022
                            #print("sorted_sch_list: ", sorted_sch_list)
                            for t_sfc in sorted_sch_list:
                                ar_sfc_current_waitingtime[t_sfc] = sfc_waited_time
                            temp_sorted_sch_list = sorted_sch_list.copy()
                            ar_sorted_sch_list.append(sorted_sch_list)
                            #print("ar_sfc_current_waitingtime: ", ar_sfc_current_waitingtime)
                            #print("overall_expected_wt_B: ", overall_expected_wt_B)
                            #print("ar_sfc_current_waitingtime: ", ar_sfc_current_waitingtime)
                            for per_sfc, per_link in temp_sfc_deploy:
                                done  = False
                                sfc   = per_sfc[0][1]['sfc'] 
                                epi_temp_sfc_arrived.append(sfc)
                                arr_traffic_packet_per_sfc = []
                                min_samples = min_samples
                                dict_traffic_packet_per_sfc = {}
                                #print("")
                                #print("SFC", sfc)
                                sfc_priority_level =  per_sfc[0][1]['priority_level']
                                sfc_int_flow       =  per_sfc[0][1]['int_flow_behaviour']
                                traffic_N = 0 #None
                                traffic_load = 0 #None
                                temp_s = 0
                                temp_pr_ft = [sfc, sfc_priority_level, sfc_int_flow, traffic_N, traffic_load]
                                traffic_transition_dict[sfc] = temp_pr_ft
                                #print("traffic_transition_dict: ", traffic_transition_dict)
                                cnt_tot_sfc += 1

                                num_of_vl = per_link[0][2]['numvl']
                                decompmvnf = False
                                res_exh = False
                                #checking if the deploying sfc is within the waiting threshold
                                #sfc_wt = per_sfc[0][1]['norm_waiting_time'] #theo_wating_time'
                                sfc_wt = per_sfc[0][1]['theo_wating_time']
                                sfc_rel = per_sfc[0][1]['reliability'] #reliability_factor
                                sfc_rel_factor = per_sfc[0][1]['reliability_factor']
                                sfc_start = time.perf_counter() 
                                traffic_pred = per_sfc[0][1]['classifier_class']
                                traffic_load = per_sfc[0][1]['traffic_load']
                                
                                if ar_sfc_current_waitingtime[sfc] >= sfc_wt:
                                    #print("Starved: Deployment time", sfc_deployment_time)
                                    cnt_sfc_starved_before_serving += 1
                                    sfc_priority_class =  per_sfc[0][1]['priority_class']
                                    sfc_starved_before_serving.append(sfc)
                                    sfc_Prclass_starved_before_serving.append(sfc_priority_class)
                                    #print("expired SFC:", sfc, "sfc_priority_class:", sfc_priority_class) 
                                    #print("did SFC starve??", sfc)
                                    epi_unemb_sfc.append(sfc)
                                    emb_status = 'starved'
                                    sfc_deployment_time = sfc_deployment_time#
                                    global_services.emb_update(episode, sfc, vnf_per_service, emb_status, 
                                                               sfc_deployment_time)   
                                else:
                                    sfc_priority_class =  per_sfc[0][1]['priority_class']
                                    #microvnf
                                    micro_requested_services, micro_requested_nodes = [], []
                                    vnf_per_service = vnf_per_service
                                    #Estimating the number of VNF persent per sfc
                                    state_space_size = vnf_per_service
                                    link_space_size = num_of_vl
                                    action_space_size = env.network_len_nodes
                                    actions_per_iteration,accpeted_action = [], []
                                    update_actionspace, updated_path_perSFC  = [], []
                                    reward_per_current_states, rewardlink_per_current_states = [], []
                                    new_reward_per_current_states = []
                                    reward_per_service, rewardDelay_per_current_states= [], []
                                    sum_reward_per_service, score_per_service =[], []
                                    micro_vnf_id, global_mVNF_action = [], []
                                    global_updated_path_perVNF = []
                                    my_dict = {}
                                    dict_node_avail = {}
                                    sorted_list = []
                                    rewards = 0
                                    reward_action = 0
                                    global_decomp = []
                                    emb_mVNFID = []
                                    #Problem-solving algorithm starts
                                    count_vnf, count_updated_path_perSFC = 0, 0
                                    count_VNF, sum_count_mvnf_persfc = 0,0
                                    count_mvnf_persfc = []
                                    done_microvnf = False
                                    for node, available_res in env.network_nodes:
                                        my_dict[node] = available_res['emb_vnf']
                                        sorted_action = dict(sorted(my_dict.items(), key=lambda item: item[1], reverse = False))
                                    keys_sorted_action = sorted_action.keys()
                                    action_space = list(map(int,keys_sorted_action))
                                    up_actionspace = action_space
                                    temp_sorted_action = sorted_action
                                    vals_sorted_action = sorted_action.values()
                                    if sum(vals_sorted_action) == 0:
                                        break
                                    norm_vals_sorted_action = [float(i)/sum(vals_sorted_action) for i in vals_sorted_action]
                                    #resource got exhusted
                                    sum_resouces = sum(list(vals_sorted_action))
                                    nodal_avail = round(sum_resouces/tot_vnf_network, 3)
                                    #Reachitechture
                                    catalog_mVNF= {}
                                    for node in original_actionspace:
                                        catalog_mVNF[node] = []
                                        temp= {}  
                                    # checking the existence of the mVNF                                    
                                    temp_accepted_action = {}
                                    for node in original_actionspace:
                                        temp_accepted_action[node] = []
                                    rewards_persfc = []
                                    for vnf,requested_res in per_sfc:
                                        vnf_start = time.perf_counter_ns()
                                        done_vnf = False
                                        vnf_cnt +=1
                                        global_services.reset(episode, sfc, vnf)
                                        updated_path_perVNF = []
                                        local_rewardVNF = 0
                                        count_mvnf = 0
                                        reward_vnfs = []                                   
                                        if not action_space:
                                            res_exh = True
                                            sum_resouces = sum(list(vals_sorted_action))
                                            break     
                                        temp_val[action] = temp_val[action] + pref_node_val
                                        prob_val = [float(i)/sum(temp_val) for i in temp_val]
                                        temp_vals_sorted_action = vals_sorted_action
                                        temp_norm_vals_sorted_action = norm_vals_sorted_action
                                        prev_action = action
                                        prev_vnf_res_req = vnf_res_req
                                        vnf_res_req = requested_res['cpu_req']
                                        pr_level = requested_res['priority_level']
                                        sum_resouces = sum(list(vals_sorted_action))
                                        nodal_avail = round(sum_resouces/tot_vnf_network, 3)                                     
                                        decompose_criteria = decomp_identifier(vnf_res_req, nodal_avail, max_vnf_resource, 
                                                                               episode, sfc, vnf, vnf_per_service)
                                        #Decomposition check 1: based on VNF resource
                                        if vnf_res_req > threshold_decompose:
                                            decompose_candidate = decompose_criteria.potential_vnf(threshold_decompose, 
                                                                                                   done_potential_vnf = False)
                                            done_decompvnf = decompose_criteria.done_potential_vnf                                
                                            global_services.update_vnf(episode,sfc, vnf,decompose_candidate, done_decompvnf)
                                            decomp= decompose_criteria.decomp
                                            num_micro_vnf = decompose_criteria.num_micro_vnf
                                            if done_decompvnf == False:
                                                pass
                                        #Decomposition check 2: based on nodal availability 
                                        elif nodal_avail < threshold_nodal_decomp:
                                            decompose_candidate = decompose_criteria.nodal_availability(done_nodal = False)
                                            done_decompvnf = decompose_criteria.done_nodal
                                            global_services.update_vnf(episode,sfc, vnf,decompose_candidate, done_decompvnf)
                                            decomp = decompose_criteria.decomp
                                            num_micro_vnf = decompose_criteria.num_micro_vnf
                                            unique_id = decompose_criteria.unique_id
                                            if unique_id:
                                                micro_vnf_id.append(unique_id)
                                            if done_decompvnf == False:
                                                pass
                                        else:
                                            pass
                                        reward_per_current_vnfstates = []
                                        rewardlink_per_current_vnfstates = []
                                        microvnf_accpeted_action = [] 
                                        t_microvnf_accpeted_action = []
                                        micro_const = microvnf_construction(max_vnf_resource = max_vnf_resource, 
                                                                            episode = episode, 
                                                                            sfc = sfc, vnf = vnf, cpu_req = vnf_res_req, 
                                                                            n =vnf_per_service , val =0, 
                                                                            pr_level =pr_level, sfc_rel = sfc_rel, 
                                                                            traffic_pred = traffic_pred)
                                        micro_vnf, micro_vnf_res = -1, -1
                                        vnf_link_resources = per_link
                                        if requested_res['microvnf_done_status'] == True:
                                            done_mVNF = True
                                            #print("==> miroserives")
                                            decomp =[]
                                            global_decomp.append(decomp)
                                            if decomp is None or not decomp:
                                                    break
                                            emb_micro = micro_const.microvnf_agent(episode = episode, 
                                                                                   sfc = sfc, 
                                                                                   vnf = vnf, 
                                                                                   decomp = decomp , 
                                                                                   n_actions = env.network_len_nodes, 
                                                                                   lr = learning_rate, 
                                                                                   epsilon = epsilon,
                                                                                   mem_size = mem_size,
                                                                                   batch_size = batch_size, 
                                                                                   vnf_cnt = vnf_cnt,
                                                                                   gamma        = gamma,
                                                                                   eps_dec      = eps_dec,
                                                                                   eps_min      = eps_min,
                                                                                   replace      = replace,
                                                                                   state_memory = state_memory,
                                                                                   action_memory = action_memory,
                                                                                   reward_memory = reward_memory,
                                                                                   next_state_memory = next_state_memory,
                                                                                   terminal_memory = terminal_memory,
                                                                                   Neurons_per_HiddenLayer = Neurons_per_HiddenLayer,
                                                                                   drop_prob = drop_prob,
                                                                                   chkpt_dir = 'models/',
                                                                                   algo = 'DDQNAgent',
                                                                                   env_name = 'BtEurope',
                                                                                   count_VNF = vnf,
                                                                                   done = done,
                                                                                   amount_local_reward = amount_local_reward,
                                                                                   reward_per_current_vnfstates = reward_per_current_vnfstates,
                                                                                   microvnf_accpeted_action = microvnf_accpeted_action, 
                                                                                   accpeted_action = accpeted_action, 
                                                                                   updated_path_perVNF = updated_path_perVNF,
                                                                                   reward_per_current_states = reward_per_current_states,
                                                                                   nodal_avail = nodal_avail,
                                                                                   sum_resouces = sum_resouces,
                                                                                   state_space_size = state_space_size, 
                                                                                   threshold_delay = threshold_delay,
                                                                                   rewardlink_per_current_states = rewardlink_per_current_states,
                                                                                   vnf_link_resources = vnf_link_resources, 
                                                                                   temp_accepted_action = temp_accepted_action,
                                                                                   original_actionspace = original_actionspace, #added om 07032022
                                                                                   mvf_t_max = mvf_t_max, 
                                                                                   mvf_t_min = mvf_t_min, 
                                                                                   mvf_old_t_current = mvf_old_t_current, 
                                                                                   new_reward_per_current_states = new_reward_per_current_states,
                                                                                   traffic_pred = traffic_pred,
                                                                                   traffic_load = traffic_load
                                                                                  )
                                            count_VNF = emb_micro[0]
                                            reward_per_current_states = emb_micro[1]
                                            rewardlink_per_current_states = emb_micro[2]
                                            accpeted_action = emb_micro[3]
                                            updated_path_perVNF = emb_micro[4]
                                            vnf_cnt = emb_micro[5] -1
                                        else:
                                            #print("here")
                                            #print("sfc:", sfc, "traffic_pred", traffic_pred, classifier_training_status)
                                            vals_sorted_action = list(vals_sorted_action)
                                            micro_vnf, micro_vnf_res = -1, -1
                                            observation   = [episode,sfc, vnf,vnf_res_req, pr_level, 
                                                             micro_vnf, micro_vnf_res, traffic_pred]
                                            state_dim     = [len(observation)]
                                            done_mVNF     = False
                                            action_space = up_actionspace
                                            action = agent.choose_action(observation,
                                                                         prob_val,
                                                                         temp_val, 
                                                                         original_actionspace)
                                            vnf_end = time.perf_counter_ns()
                                            vnf_exutime = vnf_end - vnf_start
                                            if not action in up_actionspace:
                                                reward_action_not = -1000
                                            else:
                                                reward_action_not = 0    
                                            state = vnf 
                                            subjected_to = vnf_constraints(amount_local_reward, vnf_res_req,nodal_avail,
                                                                           max_vnf_resource, episode, sfc, vnf, vnf_per_service,
                                                                           sum_resouces, accpeted_action, switch_decomp)
                                            local_rewardVNF = subjected_to.constraint_1(action)
                                            if local_rewardVNF == amount_local_reward:
                                                pref_node_val = 10 #amount_local_reward #amount_local_reward #1
                                            else:
                                                pref_node_val = 0
                                                
                                            # Modifying Reward Function
                                            for node, available_res in env.network_nodes:   
                                                dict_node_avail[node] = (available_res['emb_vnf'] / tot_vnf_per_core)
                                                
                                            # Part 1: Reward based on the quality of Nodes selected
                                            reward_Node_based = local_rewardVNF*dict_node_avail[str(action)]
                                            reward_vnfs.append(reward_Node_based)  
                                            
                                            # Part 2: Reward based on the Priority service
                                            reward_pr = local_rewardVNF*priority_level
                                            reward_vnfs.append(reward_pr)
                                            
                                            # Part 3: Reward based on the sfc_rel
                                            reward_rel = local_rewardVNF*sfc_rel
                                            reward_vnfs.append(reward_rel)
                                            
                                            # Part 4: Reward based on the traffic_pred
                                            if traffic_load == None:
                                                traffic_load = 0
                                            else:
                                                traffic_load = traffic_load

                                            if local_rewardVNF == 1000:
                                                reward_traffic_pred = local_rewardVNF*(traffic_pred*traffic_load)
                                                #print("reward_traffic_pred:", reward_traffic_pred, traffic_pred, local_rewardVNF)
                                            else: 
                                                if traffic_pred == -1:
                                                    reward_traffic_pred = -(local_rewardVNF*(traffic_pred*traffic_load))
                                                else:
                                                    reward_traffic_pred = (local_rewardVNF*(traffic_pred*traffic_load))
                                            #print("reward_traffic_pred:", reward_traffic_pred, traffic_pred, local_rewardVNF)
                                            reward_vnfs.append(reward_traffic_pred)
                                            
                                            # Part 5: Reward based on scheduling delay
                                            t_current = vnf_exutime
                                            if t_max is None or  t_current >= t_max: t_max = t_current
                                            if (t_current < t_max and t_min == 0) or (t_current <= t_min): t_min = t_current
                                            elif (t_current >= old_t_current and t_min == 0):t_min = old_t_current
                                            norm_time = 1 - Normalization(t_current, t_max, t_min)
                                            time_reward = local_rewardVNF*norm_time
                                            reward_vnfs.append(time_reward)
                                            old_t_current = t_current
                                            total_reward_vnf = round(sum(reward_vnfs), 4)
                                            new_reward_per_current_states.append(total_reward_vnf)
                                            if local_rewardVNF != -1000:
                                                local_rewardVNF = local_rewardVNF
                                                reward_per_current_states.append(local_rewardVNF)
                                            else:
                                                local_rewardVNF = local_rewardVNF 
                                                if switch_decomp is False:
                                                    decomp = None
                                                    decomp = []
                                                else: 
                                                    decomp = subjected_to.decomp
                                                    decomp = decomp    
                                                global_decomp.append(decomp)
                                                decompmvnf = True
                                                if decomp is None or not decomp:
                                                    global_reward = 0
                                                    #print("==> no decomp possible")
                                                    local_reward = local_rewardVNF
                                                    rewards = total_reward_vnf + global_reward
                                                    agent.store_transition(observation, action, 
                                                                           rewards, next_state, 
                                                                           done, vnf_cnt)
                                                    agent.learn()
                                                    epsilon = agent.epsilon
                                                    break
                                                else:
                                                    #print("VNF:", vnf)
                                                    #print("decomp")
                                                    local_rewardVNF = 1000 
                                                    unique_id = decomp[2]
                                                    if unique_id:
                                                        micro_vnf_id.append(unique_id)    
                                                for item in catalog_mVNF.items():
                                                    #print("item", item)
                                                    mVNFID_temp = []
                                                    for k in item[1]:
                                                        repeat_mVNFid = {e for e in unique_id if e == k}
                                                        mVNFID_temp = temp_accepted_action[item[0]]
                                                        if(repeat_mVNFid):
                                                            if k in mVNFID_temp:
                                                                mVNFID_temp = mVNFID_temp
                                                            else:
                                                                mVNFID_temp = mVNFID_temp + [k]
                                                        temp_accepted_action[item[0]] = mVNFID_temp
                                                count_mvnf = len(unique_id)
                                                count_mvnf_persfc.append(count_mvnf)
                                                prev_maction = False
                                                emb_micro = micro_const.microvnf_agent(episode = episode, 
                                                                   sfc = sfc, 
                                                                   vnf = vnf, 
                                                                   decomp = decomp , 
                                                                   n_actions = env.network_len_nodes, 
                                                                   lr = learning_rate, 
                                                                   epsilon = epsilon,
                                                                   mem_size = mem_size,
                                                                   batch_size = batch_size, 
                                                                   vnf_cnt = vnf_cnt,
                                                                   gamma        = gamma,
                                                                   eps_dec      = eps_dec,
                                                                   eps_min      = eps_min,
                                                                   replace      = replace,
                                                                   state_memory = state_memory,
                                                                   action_memory = action_memory,
                                                                   reward_memory = reward_memory,
                                                                   next_state_memory = next_state_memory,
                                                                   terminal_memory = terminal_memory,
                                                                   Neurons_per_HiddenLayer = Neurons_per_HiddenLayer,
                                                                   drop_prob =drop_prob,
                                                                   chkpt_dir = 'models/',
                                                                   algo = 'DDQNAgent',
                                                                   env_name = 'BtEurope',
                                                                   count_VNF = vnf,
                                                                   done = done,
                                                                   amount_local_reward = amount_local_reward,
                                                                   reward_per_current_vnfstates = reward_per_current_vnfstates,
                                                                   microvnf_accpeted_action = microvnf_accpeted_action, 
                                                                   t_microvnf_accpeted_action = t_microvnf_accpeted_action, 
                                                                   accpeted_action = accpeted_action, 
                                                                   updated_path_perVNF = updated_path_perVNF,
                                                                   reward_per_current_states = reward_per_current_states,
                                                                   nodal_avail = nodal_avail,
                                                                   sum_resouces = sum_resouces,
                                                                   state_space_size = state_space_size, 
                                                                   threshold_delay = threshold_delay,
                                                                   rewardlink_per_current_states = rewardlink_per_current_states,
                                                                   vnf_link_resources = vnf_link_resources,
                                                                   temp_accepted_action = temp_accepted_action,
                                                                   prob_val = prob_val,
                                                                   temp_val = temp_val,
                                                                   original_actionspace = original_actionspace, #added on 07032022
                                                                   prev_maction = prev_maction,
                                                                   min_pr = min_pr, #added on 13th April 2022
                                                                   max_pr = max_pr,  #added on 13th April 2022
                                                                   mvf_t_max = mvf_t_max, 
                                                                   mvf_t_min = mvf_t_min, 
                                                                   mvf_old_t_current = mvf_old_t_current, 
                                                                   new_reward_per_current_states = new_reward_per_current_states,
                                                                   traffic_pred = traffic_pred,
                                                                   traffic_load = traffic_load
                                                                   )
                                                mvf_t_max = emb_micro[7]
                                                mvf_t_min = emb_micro[8]
                                                new_reward_per_current_states = emb_micro[10]
                                                empt = []
                                                temp_count = 0
                                                for key in  microvnf_accpeted_action:
                                                    if type(key) is list:
                                                        pass
                                                    else:
                                                        empt = [unique_id[temp_count]]
                                                        catalog_mVNF[key] = unique_id[temp_count]  
                                                        if key in temp.keys():
                                                            empt = empt + temp[key]  
                                                        catalog_mVNF[key] = empt       
                                                        temp = dict(sorted(catalog_mVNF.items(), 
                                                                           key=lambda item: item[1], 
                                                                           reverse = False)) 
                                                    temp_count +=1
                                                count_VNF = emb_micro[0]
                                                #print("count_VNF: ", count_VNF)
                                                reward_per_current_states = emb_micro[1]
                                                rewardlink_per_current_states = emb_micro[2]
                                                accpeted_action = emb_micro[3]
                                                updated_path_perVNF = emb_micro[4]
                                                vnf_cnt = emb_micro[5]-1
                                                global_mVNF_action.append(microvnf_accpeted_action)
                                                global_updated_path_perVNF.append(updated_path_perVNF)    
                                            #Constraint 3 and 4: : checking for the available resouce for Links#
                                            vnf_link_resources = per_link 
                                            env_networkgraph   = env.network_graph
                                            env_networkedges   = env.network_edges
                                            subjected_to.constraint_3(state_space_size, 
                                                                   vnf_link_resources,
                                                                   env_networkgraph, 
                                                                   env_networkedges, 
                                                                   threshold_delay, 
                                                                   updated_path_perSFC
                                                                   )
                                            rewardlink_per_current_states = subjected_to.local_reward_link
                                            updated_path_perSFC = subjected_to.updated_path_perSFC
                                            #Next state
                                            next_state = vnf+1
                                            if next_state >= state_space_size:
                                                next_state = 0
                                                traffic_pred = -1
                                                next_pr_level = round(rand.uniform(0,1), 2)

                                                if len(sorted_sch_list) == num_sfc_per_arrival+1: 
                                                    next_sfc = max(sorted_sch_list)+1
                                                else: 
                                                    next_sfc = sorted_sch_list[num_sfc_per_arrival+1]

                                                next_observation = [a, next_sfc,next_state,vnf_res_req, 
                                                                    next_pr_level, micro_vnf, micro_vnf_res, traffic_pred]
                                            else: 
                                                next_state = next_state
                                                next_observation = [a, sfc,next_state,vnf_res_req, pr_level, 
                                                                    micro_vnf, micro_vnf_res, traffic_pred]
                                            state = observation 
                                            next_state = next_observation
                                        #Reward Function
                                        global_reward = 0
                                        if (sum(reward_per_current_states) == amount_local_reward*state_space_size) and \
                                            (sum(rewardlink_per_current_states) == amount_local_reward*link_space_size):
                                            done = True
                                            deployed_sfc_per_episode.append(sfc)
                                            deployed_priority_per_episode.append(pr_level)
                                            vnf_node_resources = per_sfc 
                                            vnf_link_resources = per_link 
                                            reward_func.reward_vnf(global_reward,
                                                                   rewardDelay_per_current_states, 
                                                                   rewardlink_per_current_states, 
                                                                   actions_per_iteration,
                                                                   accpeted_action,
                                                                   vnf_node_resources, 
                                                                   count_vnf, 
                                                                   updated_path_perSFC,
                                                                   vnf_link_resources, 
                                                                   count_updated_path_perSFC,
                                                                   count_success,
                                                                   global_decomp,
                                                                   emb_mVNFID,
                                                                   switch_reconst,
                                                                   global_updated_path_perVNF)
                                            count_updated_path_perSFC = reward_func.count_updated_path_perSFC
                                            count_VNF = reward_func.count_vnf
                                            global_reward = reward_func.global_reward
                                        local_reward = local_rewardVNF
                                        rewards = total_reward_vnf + global_reward
                                        rewards_persfc.append(rewards)
                                        if done_mVNF == True:
                                            pass
                                        else:
                                            #storing the transistions
                                            agent.store_transition(state, action, rewards, next_state, done, vnf_cnt)
                                            #learning:  providing state, action, reward and next state to the NN for learning.
                                            agent.learn()
                                            epsilon = agent.epsilon
                                    sum_rewards_persfc = sum(rewards_persfc)
                                    rewards_perepi.append(sum_rewards_persfc)
                                    
                                    if state_space_size == count_VNF and link_space_size == count_updated_path_perSFC:
                                        if decompmvnf is True:
                                            count_succes_decompvnf +=1
                                        sum_count_mvnf_persfc = sum(count_mvnf_persfc)
                                        count_mvnf_perepi.append(sum_count_mvnf_persfc)
                                        count_success += 1
                                        count_accpeted +=1
                                        if sfc_priority_class == "high":
                                            cnt_High_pr_emb += 1
                                        elif sfc_priority_class == "medium":
                                            cnt_Medium_pr_emb += 1
                                        else:
                                            cnt_Low_pr_emb += 1
                                            
                                        epi_emb_sfc.append(sfc) #array of embedded SFCs
                                        if sfc_rel >= 0.7:
                                            epi_emb_sfc_h_rel.append(sfc)
                                            if sfc_priority_class == 'high':
                                                cnt_rel += 1
                                        #print(cnt_rel)
                                        
                                        # update emd status: added 0n 11jan 2023
                                        emb_status = True                                        
                                    else:
                                        # update emd status: added 0n 11jan 2023
                                        epi_unemb_sfc.append(sfc)
                                        emb_status = False
                                        count_unsucces_decompvnf += 1
                                        count_mvnf_perepi.append(0)
                                    if res_exh is True:
                                        res_exhausted +=1
                                    num_sfc_per_arrival +=1
                                    
                                    sfc_end = time.perf_counter() #_ns()
                                    sfc_deployment_time = round(sfc_end-sfc_start,6) # time take by the agent to deploy the agent
                                    #print("SFC: ", sfc, sfc_deployment_time)
                                    
                                    # update emd status: added 0n 11jan 2023
                                    global_services.emb_update(episode, sfc, vnf_per_service, emb_status, 
                                                               sfc_deployment_time)

                                    if switch_sfc_expiring_check == True:
                                        for t_sfc in temp_sorted_sch_list: 
                                            if sfc == t_sfc:
                                                expected_wt_C = 0
                                                ar_sfc_current_waitingtime[sfc] += expected_wt_C
                                            else: 
                                                expected_wt_C = sfc_deployment_time
                                                ar_sfc_current_waitingtime[t_sfc] += expected_wt_C
                                        #print("expected_wt_C: ", expected_wt_C)
                                        temp_sorted_sch_list.remove(sfc)
                                    #else: #commented on 15th sept 2022
                                     #   pass # commented on 15th sept 2022
                                    #print(ar_sfc_current_waitingtime)   
                                    network_overall_accepted_action.append(accpeted_action)
                                    count_rejected = len(epi_temp_sfc_arrived) - count_accpeted
                                    count_failed   = len(epi_temp_sfc_arrived) - count_success 
                                    #episode_pr_rewardfunc.append(sfc_pr_rewardfunc)
                                    #print("sfc_priority: ", sfc_priority)
                                    episode_priority.append(sfc_priority)
                                    #episode_envpriority.append(sfc_envpriority)
                                    episode_priority_err.append(sfc_priority_err)   
                        
                            #print("epi_expected_wt_B: ", epi_expected_wt_B)
                            #print("episode", episode)
                            #overall_expected_wt_B[episode] = epi_expected_wt_B
                            #print("overall_expected_wt_B: ", overall_expected_wt_B)
                        
                        overall_ar_sorted_sch_list.append(ar_sorted_sch_list)
                        epi_cnt_sfc_starved_before_serving.append(cnt_sfc_starved_before_serving)
                        epi_cnt_sfc_Prclass_starved_before_serving.append(cnt_sfc_starved_before_serving)
                        overall_cnt_sfc_starved_before_serving.append(sfc_starved_before_serving)
                        overall_cnt_sfc_Prclass_starved_before_serving.append(sfc_Prclass_starved_before_serving)
                        epi_cnt_High_pr_emb.append(cnt_High_pr_emb)
                        epi_cnt_Medium_pr_emb.append(cnt_Medium_pr_emb)
                        epi_cnt_Low_pr_emb.append(cnt_Low_pr_emb)
                        epi_pr_emb["H"] = cnt_High_pr_emb
                        epi_pr_emb["M"] = cnt_Medium_pr_emb
                        epi_pr_emb["L"] = cnt_Low_pr_emb
                        overall_epi_pr_emb.append(epi_pr_emb)
                        overall_unemb_sfc.append(epi_unemb_sfc)
                        overall_emb_sfc.append(epi_emb_sfc)
                        overall_emb_sfc_h_rel.append(epi_emb_sfc_h_rel)

                        #print("overall_epi_pr_emb: ", overall_epi_pr_emb)
                        if len(overall_sfc_priority) != len(global_services.vNetwork_service_details_er):
                            overall_sfc_priority.append(sfc_priority)
                            epi_cnt_High_pr.append(cnt_High_pr)
                            epi_cnt_Medium_pr.append(cnt_Medium_pr)
                            epi_cnt_Low_pr.append(cnt_Low_pr)
                            epi_pr["H"] = cnt_High_pr
                            epi_pr["M"] = cnt_Medium_pr
                            epi_pr["L"] = cnt_Low_pr
                            #print("epi_pr: ", epi_pr)
                            #print("cnt_High_pr: ", cnt_High_pr, "cnt_Medium_pr: ", cnt_Medium_pr, "cnt_Low_pr: ", cnt_Low_pr)
                            overall_epi_pr.append(epi_pr)
                            status_cnt_ridge = False
                            copy_overall_epi_pr = overall_epi_pr.copy()
                        else:
                            status_cnt_ridge = True
                            copy_overall_episode_envpriority = overall_episode_envpriority.copy()
                            #print("copy_overall_epi_pr: ",copy_overall_epi_pr)
                            #print("XXXX")
                        
                        overall_episode_priority.append(episode_priority)
                        if sch_model == 'DDPG':  
                            overall_episode_envpriority.append(sfc_envpriority)
                        else: 
                            copy_overall_episode_envpriority = overall_episode_envpriority.copy()
                            
                        overall_episode_priority_err.append(episode_priority_err)
                        overall_deployed_sfc_per_episode.append(deployed_sfc_per_episode)
                        overall_deployed_priority_per_episode.append(deployed_priority_per_episode)
                        sum_rewards_perepi = sum(rewards_perepi)
                        network_overall_sum_rewards_perepi.append(sum_rewards_perepi)
                        sum_count_mvnf_perepi = sum(count_mvnf_perepi)
                        network_overall_sum_mvnf.append(sum_count_mvnf_perepi)
                        network_overall_count_mvnf.append(count_mvnf_perepi)
                        network_overall_accepted.append(count_success)
                        network_overall_rejected.append(count_failed)
                        tot_decompsfc = (count_succes_decompvnf + count_unsucces_decompvnf) 
                        undecomp_accepted = num_sfc-tot_decompsfc
                        undecomp_rejected = tot_decompsfc
                        monolithic_deploy_success.append(undecomp_accepted)
                        monolithic_deploy_rejected.append(undecomp_rejected)
                        overall_res_exhausted.append(res_exhausted)
                        network_overall_tot_decompsfc.append(tot_decompsfc)
                        network_overall_count_succes_decompvnf.append(count_succes_decompvnf)
                        network_overall_count_unsucces_decompvnf.append(count_unsucces_decompvnf)
                        overall_epi_traffic_packets_dict[a] = epi_traffic_packets_dict
                        overall_epi_traffic_load_dict[a] = epi_traffic_load_dict
                        
                        if switch_priorty == True:
                            if pr_model == 'DDPG' or pr_model == 'DDQN': 
                                pass
                            else:
                                if done_status == "Train":
                                    cnt_macro_pr = tr_count_macro_pr
                                    acc = round(tr_macro_acc,2)
                                    epi_cnt_macro_pr = epi_tr_count_macro_pr
                                    epi_acc = round((epi_cnt_macro_pr/num_sfc)*100,2)
                                    epi_tr_correct_perc.append(epi_acc)
                                elif done_status == "Eval":
                                    cnt_macro_pr = eval_count_macro_pr
                                    acc = eval_macro_acc
                                    epi_cnt_macro_pr = epi_eval_count_macro_pr
                                    epi_acc = round((epi_cnt_macro_pr/num_sfc)*100,2)
                                else:
                                    cnt_macro_pr = pred_count_macro_pr
                                    acc = pred_macro_acc
                                    epi_cnt_macro_pr = epi_pred_count_macro_pr
                                    epi_acc = round((epi_cnt_macro_pr/num_sfc)*100,2)
                                    epi_pred_correct_perc.append(epi_acc)
                        else:
                            pass

                        if switch_clustering is True:
                            #pass
                            if a%num_episode==0:
                                pass
                                #clusterplot_val = class_traffic_clustering.clusteringplot(epi_clustering_status, 
                                #                                                          traffic_input_dataset, 
                                #                                                          cluster_label_val)

                        if episode == (num_episode - 1):
                            #classifers_plots = class_traffic_classifer.classifier_plot(classifier_measurements, num_sfc, baseline)
                        
                            title = 'Runtime for cluster model'
                            #kmeans_runtime, Aggo_runtime, birch_runtime, GM_runtime
                            Y_axis_Rcluster = [sum(kmeans_runtime), sum(Aggo_runtime), sum(birch_runtime), sum(GM_runtime)]
                            #print("X_axis_Rcluster: ", X_axis_Rcluster)
                            X_axis_Rcluster = ['K-means', 'Agglomerative', 'BIRCH', 'GM']
                            fig = plt.figure()
                            plt.barh(X_axis_Rcluster, Y_axis_Rcluster, color = '#27408B', edgecolor ='#2E2E2E',hatch = '///') 
                            plt.ylabel("Cluster Modelling")
                            plt.xlabel("Runtime (ns)")
                            plt.grid()
                            plt.show()
                            
                            ##saving the figure
                            figure_title = str(title)+"_"+str(num_sfc)+"_"+str(episode)
                            #fig.savefig('Generated_plots/cluster/epi_2000_100sfc_agglo_DT/Plot_'+str(figure_title)+'.png')
                            #fig.savefig('Generated_plots/cluster/epi_2000_100sfc_agglo_DT/Plot_'+str(figure_title)+'.pdf', bbox_inches='tight')


                            #saving clustering data
                            cluster_data = ["Title: " + str(figure_title ) +\
                                            "X_axis_Rcluster: " + str(X_axis_Rcluster) +\
                                            "Y_axis_Rcluster: " + str(Y_axis_Rcluster)]

                            #fname = figure_title
                            #print(fname)
                            #f = open('Generated_plots/cluster/epi_2000_100sfc_agglo_DT/'+fname,"a")
                            #f.write(str(cluster_data))
                            #f.close()
                            #print("XXXXXXXXXXXXXXXXX")   
                        print("*************")
                        print("cnt_sfc_arrival: ", cnt_sfc_arrival)
                        print("Total Services for Episode",a,"is",len(epi_temp_sfc_arrived), "Num accepted",count_success,"Num rejected",count_failed)
                        #print("Episode",a," Done Status: ", done_status," count_macro_pr: ", cnt_macro_pr," Epi count_macro_pr: ", epi_cnt_macro_pr, "Epi Acc: ", epi_acc, acc)
                        #print('Mean MAE: %.3f (%.3f)' % (m(scores), std(scores)))
                        #print("Episode",a," count_macro_pr: ", count_macro_pr, " Episode count macro: ", epi_cnt_macro_pr)
                        count_numVNF_perNode = []
                        count_num_mVNF_perNode = []
                        for ac in [i for i in range(env.network_len_nodes)]:
                            count =0
                            count_mVNF = 0
                            for key in network_overall_accepted_action:
                                for action in key:
                                    if type(action) is list:
                                        for maction in action:
                                            if maction is ac:
                                                count_mVNF += 1
                                    else:
                                        if action is ac:
                                            count +=1
                            count_numVNF_perNode.append(count)
                            count_num_mVNF_perNode.append(count_mVNF)
                        overall_count_numVNF_perNode.append(count_numVNF_perNode)
                        overall_count_num_mVNF_perNode.append(count_num_mVNF_perNode)
                        qq = list(env.network_graph.nodes(data=True))
                        remain_cpu = []
                        for key, val in qq:
                            cpu = val['emb_vnf']
                            #print(key, cpu)
                            remain_cpu.append(cpu)
                        sum_remain_cpu = sum(remain_cpu)
                        #remaining BW
                        residual_bw_res = []
                        for key in env.network_edges: 
                            residual_bw_res.append(key[2]['bw'])
                        overall_remain_cpu.append(sum_remain_cpu)
                        overall_remain_bw.append(residual_bw_res)
                        #print("epsilon", round(epsilon, 2))
                        network_overall_epsilon.append(round(epsilon, 2))
                        a += 1
                        
                    from statistics import mean 
                    mean_network_overall_accepted = mean(network_overall_accepted)
                    end = time.time()  
                    runtime = end-start
                    runtime = round(runtime,2)
                    print("runtime",runtime)
                    topology_name = str('Netrail')

                    overall_res  = []
                    sum_overall_res = 0
                    for per_episode in global_services.vNetwork_service_details_er:
                        res_perepi = []
                        sum_res_perepi = 0
                        for per_sfc, per_link in per_episode:
                            res_persfc = []
                            sum_res_persfc = 0
                            #print(per_sfc)
                            for vnf, val in per_sfc:
                                res_persfc.append(val['cpu_req'])
                            #print(res_persfc)  
                            sum_res_persfc = sum(res_persfc)
                            #print(sum_res_persfc)
                            res_perepi.append(sum_res_persfc)
                        #print(res_perepi)
                        sum_res_perepi = sum(res_perepi)
                        #print(sum_res_perepi)
                        overall_res.append(sum_res_perepi)
                    sum_overall_res = sum(overall_res)

                    upper_bound = round(tot_avail_res_network/min(overall_res), 2)
                    lower_bound = round(tot_avail_res_network/max(overall_res),2)

                    theo_bound  = [lower_bound, upper_bound]
                    #print(theo_bound)

                    Bound_Cal = ['overall_res: ' +str(overall_res), 'sum_overall_res: ' +str(sum_overall_res),\
                         'theo_bound: '  +str(theo_bound), 'min: ' +str(min(overall_res)), 'max: ' +str(max(overall_res))             
                        ]
                    #print(Bound_Cal)
                    
                    Epsilon = 1
                    overall_vnf = (vnf_per_service*num_sfc)*num_episode
                    Information = 'Information: '
                    experience_replay = ['Experience Replay:',('Memory Size: '+str(mem_size)), ('Batch Size: '+ str(batch_size)), ('Replace: '+ str(replace))]
                    topologyname = ['Tolpology:'+ topology_name, ('Node:'+ str(env.network_len_nodes)), ('Link:' + str(env.len_tot_links))]
                    execution = [('Episode: '+ str(num_episode)), ('SFC: ' + str(num_sfc)), ('VNF per SFC: '+ str(vnf_per_service))]
                    Info = [Information] + [Algorithm]+ [topologyname] + [Opimizer] + [execution]
                    #print(Info)    
                    Parameters = 'Parameters: '
                    NN_para = ['Neural Network: '+ ('Neuron units: '+ str(Neurons_per_HiddenLayer)), 'Hidden Layer: '+str(Hidden_layer),\
                               'Drop Probability: '+ str(drop_prob)]
                    learn_para = ['Learning :'+ ('Learning Rate: ' + str(learning_rate)), ('Epsilon: '+ str(Epsilon)), ('Gamma: '+ str(gamma)),\
                                 ('Epsilon Dec: '+ str(eps_dec)), 'Epsilon Min: '+ str(eps_min), Opimizer]
                    RL_para = ['RL rewards function: ' + ('Reward: ' +str(amount_local_reward)) , ('Penalty: ' + str(amount_local_penalty))]

                    Decomp = ['Decomp: ' +'switch_reconst: ' +str(switch_reconst),'switch_decomp: ' + str(switch_decomp),\
                              'n_state: ' + str(n_state), ' with No Constraint 2 '
                             ]
                    para = [Parameters] + [NN_para] + [learn_para]  + [experience_replay] + [RL_para] + [Decomp]
                    Topology_resource_init = ['Topology Resouce Initilization: '+ ('Latency: ' + str(threshold_delay)),\
                                              'CPU core: ' + str(core_cpu), 'VNF per Core: ' + str(num_vnf_per_core),\
                                              'Total VNF per core' + str(tot_vnf_per_core)]
                    Nodal_outage_para = ['Nodal Outage: ' + 'Outage Percentage: '+str(nodal_outage_percentage),\
                                         'No of nodes outaged: ' + str(round_nodal_outage)]
                    # Result
                    computation_time = 'Runtime: '+str(runtime)+ ' seconds'
                    results = ['Results: ' + computation_time, 
                               'Total Generated VNF: ' + str(overall_vnf),'Mean of Overall VNF: '+ str(mean_network_overall_accepted),\
                               'SAR: '+str(network_overall_accepted), 'Monolithic_deploy_success: ' + str(monolithic_deploy_success),\
                               'Monolithic_deploy_rejected: ' + str(monolithic_deploy_rejected),\
                               'Network_overall_tot_decompsfc: ' + str(network_overall_tot_decompsfc),\
                               'Network_overall_count_succes_decompvnf: ' + str(network_overall_count_succes_decompvnf),\
                               'network_overall_count_unsucces_decompvnf: ' + str(network_overall_count_unsucces_decompvnf),\
                               'network_overall_sum_mvnf: ' + str(network_overall_sum_mvnf),\
                               'network_overall_count_mvnf: ' +str(network_overall_count_mvnf),\
                               'network_overall_sum_rewards_perepi: '+str(network_overall_sum_rewards_perepi),\
                               'network_overall_epsilon: '+ str(network_overall_epsilon)
                              ]
                    #print(results)
                    resource_residual = ['Resource Residual: ' + 'VNF residual: '+ str(overall_remain_cpu) +\
                                         'Link Residual: '+ str(overall_remain_bw) + \
                                         #'VNF Distribution: '+str(Network_count_numVNF_perNode),\
                                         'overall_count_numVNF_perNode: ' + str(overall_count_numVNF_perNode),\
                                         'overall_count_num_mVNF_perNode: ' + str(overall_count_num_mVNF_perNode),\
                                         'overall_res_exhausted: '+ str(overall_res_exhausted)
                                        ]
                    priority = ['Priority: ' + 'pr_HL: ' +str(pr_HL) + \
                                'overall_episode_pr_rewardfunc: ' + str(overall_episode_pr_rewardfunc)+\
                                ' overall_sfc_priority: ' +str(overall_sfc_priority)+\
                                ' overall_episode_envpriority: '+ str(overall_episode_envpriority)+\
                                ' copy_overall_episode_envpriority: ' + str(copy_overall_episode_envpriority)+\
                                ' epi_cnt_sfc_starved_before_serving: ' + str( epi_cnt_sfc_starved_before_serving)+\
                                ' overall_cnt_sfc_starved_before_serving: ' + str(overall_cnt_sfc_starved_before_serving)+\
                                ' epi_cnt_sfc_Prclass_starved_before_serving: ' + str( epi_cnt_sfc_Prclass_starved_before_serving)+\
                                ' overall_cnt_sfc_Prclass_starved_before_serving: ' + str(overall_cnt_sfc_Prclass_starved_before_serving)+\
                                ' copy_overall_epi_pr: ' + str(copy_overall_epi_pr)+\
                                ' overall_epi_pr: ' + str(overall_epi_pr)+\
                                ' overall_epi_pr_emb: ' + str(overall_epi_pr_emb)
                                #'over_count_micro_pr:' +str(over_count_micro_pr)
                               ]
                    
                    #print("overall_emb_sfc: ", overall_emb_sfc, " overall_emb_sfc_h_rel:", overall_emb_sfc_h_rel)
                    #print("overall_ar_sorted_sch_list: ", overall_ar_sorted_sch_list)
                    embeded_sfc = ['Emb_SFC: ' +\
                                   'overall_emb_sfc: '+ str(overall_emb_sfc) +\
                                   'overall_UNemb_sfc: '+ str(overall_unemb_sfc) +\
                                   'overall_emb_sfc_h_rel:' + str(overall_emb_sfc_h_rel) +\
                                   'overall_ar_sorted_sch_list: ' +str(overall_ar_sorted_sch_list)
                                  ]
                    
                    traffic_part = ['Traffic: '+ 'overall_epi_traffic_load_dict: ' + str(overall_epi_traffic_load_dict) +\
                                    'overall_epi_traffic_packets_dict: '+str(overall_epi_traffic_packets_dict) +\
                                    'classifier_measurements: ' +str(classifier_measurements)]

                    #print(embeded_sfc)
                    data = [Info] + [para] + [Topology_resource_init] + [Nodal_outage_para] + [computation_time] + [results] + \
                           [resource_residual] + [extra_note] + [Bound_Cal] + [priority]+[embeded_sfc] +[traffic_part]              
                    #print(data)
                    
                    
                    dt = datetime.now()
                    # getting the timestamp
                    ts = datetime.timestamp(dt)
                    # convert to datetime
                    date_time = datetime.fromtimestamp(ts)
                    # convert timestamp to string in dd-mm-yyyy HH:MM:SS
                    str_date_time = date_time.strftime("%d-%m-%Y-%H-%M-%S")
                    #print(str_date_time)
                    
                    fname = str(trial)+'_'+ str(topology_name)+\
                            '_cpucore_' + str(core_cpu)+'_num_vnf_per_core_'+\
                            str(num_vnf_per_core) +\
                            '_mean_' + str(round(mean_network_overall_accepted,3))+\
                            '_sfc_' + str(num_sfc)+\
                            '_epi_'+str(num_episode)+ '_runtime_' +str(runtime)+\
                            '_Clustering status_'+  str(switch_clustering)+\
                            '_switch_sfc_expiring_check_' +str(switch_sfc_expiring_check) +\
                            str(str_date_time)
                    
                           # '_switch_reconst_' +str(switch_reconst) +\
                            #'_switch_decomp_' + str(switch_decomp) + \
                            #'_switch_priorty_' + str(switch_priorty)+ \
                            #'_pr_model_'+ str(pr_model)+\
                            #'_sch_model_'+ str(sch_model)
                            
                    print(fname)
                    f = open('Data_storage/epi_2000_100sfc_agglo_DT/'+fname,"a")
                    f.write(str(data))
                    f.close()
                    print("XXXXXXXXXXXXXXXXX")

        if len(overall_sfc_priority) != len(global_services.vNetwork_service_details_er):
            status_cnt_ridge = False
        else:
            status_cnt_ridge = True
            #print("XXXX")
        #print("overall_ar_sorted_sch_list: ",overall_ar_sorted_sch_list)   
        #print("overall_epi_pr: ", overall_epi_pr)
        #print("")
        #print("Copy_overall_epi_pr: ", copy_overall_epi_pr)
        #print("overall_epi_pr_emb: ", overall_epi_pr_emb)
        #print("overall_sfc_priority: ", overall_sfc_priority)
        sch_model_overall_cnt_sfc_starved_before_serving[sch_model] = overall_cnt_sfc_starved_before_serving    
        sch_model_overall_cnt_sfc_Prclass_starved_before_serving[sch_model] = overall_cnt_sfc_Prclass_starved_before_serving 
        #print(sch_model_overall_cnt_sfc_starved_before_serving)
        #print(sch_model_overall_cnt_sfc_Prclass_starved_before_serving)



In [ ]:
print(fname)
fname = fname
#'00_Netrail_cpucore_12_num_vnf_per_core_8_mean_40.014_sfc_100_epi_2000_runtime_95484.37_Clustering status_False_switch_sfc_expiring_check_False25-03-2023'#fname #"Netrail_12_4_100epi_cluster_false_classfier_flase_admission-control_false"
f = open('Data_storage/without_admission_controller/'+fname,"a")
f.write(str(data))
f.close()
#print("XXXXXXXXXXXXXXXXX")#

In [ ]:
print("work now")